# Анализ трансляций национального медицинского исследовательского центра

## Постановка задачи


Национальный медицинский исследовательский центр Минздрава России (далее: НМИЦ) проводит научные семинары с выступлениями докладчиков. Доклады транслируются онлайн. Прослушать доклады онлайн можно при регистрации на соответствующей платформе. Всего было проведено восемь семинаров: 2 крупных мероприятия и шесть менее крупных семинаров. 


По итогам проведённых семинаров накопились данные о трансляциях. Данные представлены в датасетах: 

* "Все мероприятия 2" - логи просмотров трансляций зрителями;

* "Расписание выступлений 11" и "Расписание выступлений 12" - данные о прочитанных докладах двух крупных форумов;

* "Расписание_небольших_мероприятий 2" - данные о прочитанных докладах на прочих семинарах НМИЦ;

* "Словарь рубрик 11-12" - дополнительные данные о прочитанных докладах с двух крупных форумов.


Заказчик в лице НМИЦ ставит задачу проанализировать имеющиеся данные, сделать выводы о посещаемости вебинаров, выявить наиболее интересные темы, доклады, докладчиков и др., выявить активные (заинтересованные) группы зрителей трансляций, сформулировать выводы и рекомендации для будущих трансляций. 


❗ В соответствии с NDA в исследовании не приведено наименование исследовательского центра (НМИЦ), в данных изменены даты, имена докладчиков вымышленные, реальные названия докладов, сессий, направлений, форумов заменены условными обозначениями. В связи с такими заменами, часть преобразований либо отсутствует, либо не имеет логического обоснования и текстовой подсказки. Также, разумеется, формулировки выводов и рекомендаций максимально обезличены, сведены к минимуму. ❗

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import statistics
import datetime  
from datetime import datetime

import warnings
warnings.simplefilter('ignore')

## Обработка датасета с данными просмотров зрителей трансляций

#### Краткое описание операций при обработке датасета с просмотрами зрителей


Отсечение столбцов с излишней информацией.

Унификация и исправление ошибок в обозначении страны, региона, города, професии, специализации зрителя. Замена пропущенных значений в этих столбцах символом "-". 

Введение поправки +3 часа к значениям времени начала и окончания просмотра трансляции.

Введение вспомогательных столбцов с датой без часов и минут.

Рассмотрение строк без данных или частично без данных о зрителе. Удаление неинформативных строк - строк без каких либо данных о зрителе.

In [2]:
# Открытие и просмотр файла с записями просмотров

data_v = pd.read_csv('Все мероприятия 2.csv', sep=';')
pd.set_option('display.max_columns', 100)

data_v.head()

,UID,OID,BARCODE,COUNTRY,REGION,CITY,PROFESSION,SPECIALIZATION,"Общее время просомтра, мин",ПОТОК,Устройство,Оп.сис,Броузер,Время начала,Время окончания,Timestamp начала,Timestamp окончания,"Время просомтра, мин",Кол-во кликов,ID открытой сессии,ID закрытой сессии,IP,Unnamed: 22,SERVER SESSION ID,Статус открытия,Статус закрытия,Мероприятие
0,16573.0,626199.0,0809202204421616573,Россия,Амурская обл.,NaN,Врач,онкология: хирургия (огш),52.0,1.0,Десктоп,Windows,Chrome,13.03.2022 9:42,13.03.2022 9:44,1647153750,1647153857,2,NaN,NaN,vEg3gydkv6KzcG,NaN,NaN,vIsrJYwqtiP3UwjQ3gCiHPl6OS1eFpnu,PLAY_START,DESTROY_PAUSE,Мероприятие 1
1,16573.0,626199.0,0809202204421616573,Россия,Амурская обл.,NaN,Врач,онкология: хирургия (огш),52.0,1.0,Десктоп,Windows,Chrome,13.03.2022 9:51,13.03.2022 9:51,1647154294,1647154297,3 сек.,NaN,NaN,vEg3gydkv6KzcG,NaN,NaN,vIsrJYwqtiP3UwjQ3gCiHPl6OS1eFpnu,PLAY_RESTART_PAUSE,NaN,Мероприятие 1
2,16573.0,626199.0,0809202204421616573,Россия,Амурская обл.,NaN,Врач,онкология: хирургия (огш),52.0,1.0,Десктоп,Windows,Chrome,13.03.2022 9:51,13.03.2022 9:54,1647154302,1647154495,3,NaN,NaN,bquNyVcL7eCLeE,NaN,NaN,vIsrJYwqtiP3UwjQ3gCiHPl6OS1eFpnu,PLAY_START,DESTROY_STOP,Мероприятие 1
3,16573.0,626199.0,0809202204421616573,Россия,Амурская обл.,NaN,Врач,онкология: хирургия (огш),52.0,1.0,Десктоп,Windows,Chrome,13.03.2022 9:54,13.03.2022 10:09,1647154497,1647155385,15,NaN,NaN,bquNyVcL7eCLeE,NaN,NaN,vIsrJYwqtiP3UwjQ3gCiHPl6OS1eFpnu,PLAY_RESTART_STOP,DESTROY_PAUSE,Мероприятие 1
4,16573.0,626199.0,0809202204421616573,Россия,Амурская обл.,NaN,Врач,онкология: хирургия (огш),52.0,1.0,Десктоп,Windows,Chrome,13.03.2022 10:10,13.03.2022 10:14,1647155406,1647155697,5,NaN,NaN,bquNyVcL7eCLeE,NaN,NaN,vIsrJYwqtiP3UwjQ3gCiHPl6OS1eFpnu,PLAY_RESTART_PAUSE,DESTROY_PAUSE,Мероприятие 1


In [3]:
# Просмотр информации о данных в таблице

data_v.info(verbose=True, show_counts=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 154333 entries, 0 to 154332
Data columns (total 27 columns):
 #   Column                      Non-Null Count   Dtype  
---  ------                      --------------   -----  
 0   UID                         134779 non-null  float64
 1   OID                         135694 non-null  float64
 2   BARCODE                     135694 non-null  object 
 3   COUNTRY                     127206 non-null  object 
 4   REGION                      118648 non-null  object 
 5   CITY                        133047 non-null  object 
 6   PROFESSION                  133191 non-null  object 
 7   SPECIALIZATION              131793 non-null  object 
 8   Общее время просомтра, мин  135694 non-null  float64
 9   ПОТОК                       135694 non-null  float64
 10  Устройство                  135694 non-null  object 
 11  Оп.сис                      135693 non-null  object 
 12  Броузер                     135693 non-null  object 
 13  Время начала  

Описание базы данных, представленное заказчиком.


* UID - Уникальный идентификатор пользователя, не меняется

* OID - Уникальный идентификатор участника, меняется от мероприятия к мероприятию

* BARCODE -  это билет, уникален для каждого участника (он выдается на мероприятие один раз, по нему мы отслеживаем оффлайн участников)

* COUNTRY - страна

* REGION - регион

* CITY - город

* PROFESSION - профессия

* SPECIALIZATION - специализация

* Общее время просомтра, мин - Это суммарный расчет времени просмотра пользователя. Вообще всего по базе. Пока никакой практической пользы не несет

* ПОТОК - Это номер “потока” - куда транслировалась конкретная конференция, каждый день работает ограниченное количество потоков. Каждый день в этом потоке могут быть разные конференции в соответствии с расписанием

* Устройство - Тип устройства с которого был просмотр видеозаписи

* Оп.сис - Операционная система пользователя

* Броузер - Браузер через который смотрели запись

* Время начала - Дата и время в которое пользователь начал просмотр, конкретной сессии

* Время окончания - Дата и время в которое пользователь закончил просмотр в рамках данной сессии

* Timestamp начала - Дата и время в формате UNIX TIME начала подключения

* Timestamp окончания - Дата и время в формате UNIX TIME окончания просмотра

* Время просомтра, мин - Время просмотра, разница между датой и временем окончания просмора и начала просмотра конкретной сессии

* Кол-во кликов - Количество кликов для подтверждения просмотра трансляции. Необходим для оценки присутствия на мероприятии, если пользователь не нажал на всплывающее окно, то сессия подключения прерывается

* ID открытой сессии - 

* ID закрытой сессии - 

* IP - ip адрес с которого пользователь подключался к трансляции

* SERVER SESSION ID - Уникальный идентификатор сессии подключения. У нас пользователь может заходить одновременно с разных устройств и браузеров соответственно у него может быть одновременно несколько открытых сессий. Хотя, в один и тот же момент времени не должно такого быть, так как если человек открывает трансляцию в другом месте, то она должна автоматически обрывать предыдущую сессию.

* Статус открытия - 

* Статус закрытия - 

* Мероприятие - Название мероприятия


In [4]:
# Переименование столбцов

data_v = data_v.rename(columns={'UID':'user_id',
                           'OID':'o_id',
                           'BARCODE':'barcode',
                           'COUNTRY':'country',
                           'REGION':'region',
                           'CITY':'user_city',
                           'PROFESSION':'profession',
                           'SPECIALIZATION':'specialization',
                           'Общее время просомтра, мин':'total_time_min',
                           'ПОТОК':'potok',
                           'Устройство':'device',
                           'Оп.сис':'system',
                           'Броузер':'browser',
                           'Время начала':'user_date_start',
                           'Время окончания':'user_date_end',
                           'Timestamp начала':'user_timestamp_start',
                           'Timestamp окончания':'user_timestamp_end',
                           'Время просомтра, мин':'view_duration_min',
                           'Кол-во кликов':'click_cnt',
                           'ID открытой сессии':'opened_session_id',
                           'ID закрытой сессии':'closed_session_id',
                           'IP':'ip',
                           'SERVER SESSION ID':'server_session_id',
                           'Статус открытия':'status_opened',
                           'Статус закрытия':'status_closed',
                           'Мероприятие':'event'})

In [5]:
# Контроль выполнения операции

data_v.head(1)

,user_id,o_id,barcode,country,region,user_city,profession,specialization,total_time_min,potok,device,system,browser,user_date_start,user_date_end,user_timestamp_start,user_timestamp_end,view_duration_min,click_cnt,opened_session_id,closed_session_id,ip,Unnamed: 22,server_session_id,status_opened,status_closed,event
0,16573.0,626199.0,0809202204421616573,Россия,Амурская обл.,NaN,Врач,онкология: хирургия (огш),52.0,1.0,Десктоп,Windows,Chrome,13.03.2022 9:42,13.03.2022 9:44,1647153750,1647153857,2,NaN,NaN,vEg3gydkv6KzcG,NaN,NaN,vIsrJYwqtiP3UwjQ3gCiHPl6OS1eFpnu,PLAY_START,DESTROY_PAUSE,Мероприятие 1


Для текущего исследования в датасете с просмотрами присутствует излишняя информация. Часть данных отсекаю.

In [6]:
# Формирование укороченной версии датасета

data_users = data_v[['user_id', 'o_id', 'barcode', 'country', 'region', 'user_city', 'profession',
                   'specialization', 'potok', 'device', 'user_date_start', 'user_date_end',
                   'user_timestamp_start', 'user_timestamp_end', 'event']]

In [7]:
# Контроль выполнения операции

data_users.head(1)

,user_id,o_id,barcode,country,region,user_city,profession,specialization,potok,device,user_date_start,user_date_end,user_timestamp_start,user_timestamp_end,event
0,16573.0,626199.0,0809202204421616573,Россия,Амурская обл.,NaN,Врач,онкология: хирургия (огш),1.0,Десктоп,13.03.2022 9:42,13.03.2022 9:44,1647153750,1647153857,Мероприятие 1


In [8]:
# Подсчёт числа уникальных зрителей

data_users['user_id'].nunique()

3190

In [9]:
# Подсчёт числа уникальных o_id

data_users['o_id'].nunique()

3945

In [10]:
# Подсчёт числа уникальных билетов/штрих-кодов

data_users['barcode'].nunique()

4811

In [11]:
# Вывод списка стран зрителей

data_users['country'].unique()

array(['Россия', nan, 'Молдова', 'Казахстан', 'Беларусь', 'Австралия',
       'undefined', 'Украина', 'Армения', 'Кыргызстан', 'Узбекистан',
       'Латвия', 'Германия', 'Россия ', 'Таджикистан', 'Китай',
       'Азербайджан', '1', 'Выберите', 'Россич'], dtype=object)

In [12]:
# Вывод подозрительных значений

# data_users.query('country == "undefined"')
# data_users.query('country == "1"')
# data_users.query('country == "Выберите"')

# data_users.query('country == "undefined"')['region'].unique()
# data_users.query('country == "undefined"')['city'].unique()

In [13]:
# Замена неверных значений

data_users['country'] = data_users['country'].replace('undefined', np.nan)
data_users['country'] = data_users['country'].replace('Россия ', 'Россия')
data_users['country'] = data_users['country'].replace('Россич', 'Россия')
data_users['country'] = data_users['country'].replace('1', np.nan)
data_users['country'] = data_users['country'].replace('Выберите', np.nan)

In [14]:
# Восстановление пропущенных значений

data_users.loc[data_users.loc[:,'user_city'] == 'Санкт-Петербург', 'country'] = 'Россия'
data_users.loc[data_users.loc[:,'user_city'] == 'Москва', 'country'] = 'Россия'
data_users.loc[data_users.loc[:,'user_city'] == 'Краснодар', 'country'] = 'Россия'

In [15]:
# Замена пропущенных значений 

data_users['country'] = data_users['country'].fillna('-')

In [16]:
# Проверка выполнения операций

data_users['country'].unique()

array(['Россия', '-', 'Молдова', 'Казахстан', 'Беларусь', 'Австралия',
       'Украина', 'Армения', 'Кыргызстан', 'Узбекистан', 'Латвия',
       'Германия', 'Таджикистан', 'Китай', 'Азербайджан'], dtype=object)

In [17]:
# Вывод списка регионов зрителей

data_users['region'].unique()

array(['Амурская обл.', nan, 'Удмуртия', 'Молдова', 'Дагестан',
       'Павлодарская обл.', 'Санкт-Петербург и область',
       'Владимирская обл.', 'Белгородская обл.', 'Ростовская обл.',
       'Москва и Московская обл.', 'Коми', 'Красноярский край',
       'Вологодская обл.', 'Новосибирская обл.', 'Гродненская обл.',
       'Кемеровская обл.', 'Башкортостан(Башкирия)', 'Минская обл.',
       'Кабардино-Балкария', 'Омская область', 'Крым (Республика)',
       'Рязанская обл.', 'Калининградская обл.', 'Виктория',
       'Нижегородская (Горьковская)', 'Брестская обл.', 'Калмыкия',
       'Татарстан', 'Астраханская обл.', 'Саратовская обл.',
       'Воронежская обл.', 'Ярославская обл.', 'Чувашия',
       'Краснодарский край', 'Северная Осетия-Алания',
       'Архангельская обл.', 'Хоринск', 'Бурятия', 'Пензенская обл.',
       'Саха (Якутия)', 'Самарская обл.', 'Тюменская обл.',
       'Волгоградская обл.', 'Брянская обл.', 'Челябинская обл.',
       'undefined', 'Луганская обл.', 'Пер

In [18]:
# Подсчёт числа уникальных регионов зрителей

data_users['region'].nunique()

130

In [19]:
# Вывод подозрительных значений

# data_users.query('region == "undefined"')['city'].unique()
# data_users.query('region == "1"')
# data_users.query('region == "Выберите"')
# data_users.query('region == " "')
# data_users.query('region == "Казахстан"')
# data_users.query('region == "Виктория"')

In [20]:
# Восстановление пропущенных значений

data_users.loc[data_users.loc[:,'user_city'] == 'Пермь', 'region'] = 'Пермский край'
data_users.loc[data_users.loc[:,'user_city'] == 'Ярославль', 'region'] = 'Ярославская обл.'
data_users.loc[data_users.loc[:,'user_city'] == 'Хоринск', 'region'] = 'Бурятия'

data_users.loc[data_users.loc[:,'user_city'] == 'Шымкент', 'region'] = 'Чимкентская обл.'
data_users.loc[data_users.loc[:,'user_city'] == 'Волгоград', 'region'] = 'Волгоградская обл.'
data_users.loc[data_users.loc[:,'user_city'] == 'Калининград', 'region'] = 'Калининградская обл.'

data_users.loc[data_users.loc[:,'user_city'] == 'Москва', 'region'] = 'Московская обл.' 
data_users.loc[data_users.loc[:,'user_city'] == 'Дзержинский', 'region'] = 'Московская обл.' 
data_users.loc[data_users.loc[:,'user_city'] == 'Раменское', 'region'] = 'Московская обл.' 

data_users.loc[data_users.loc[:,'user_city'] == 'Краснодар', 'region'] = 'Краснодарский край'
data_users.loc[data_users.loc[:,'user_city'] == 'Санкт-Петербург', 'region'] = 'Ленинградская обл.'
data_users.loc[data_users.loc[:,'user_city'] == 'Хабаровск', 'region'] = 'Хабаровский край'

data_users.loc[data_users.loc[:,'user_city'] == 'Азов', 'region'] = 'Ростовская обл.'
data_users.loc[data_users.loc[:,'user_city'] == 'Тольятти', 'region'] = 'Самарская обл.'
data_users.loc[data_users.loc[:,'user_city'] == 'Биробиджан', 'region'] = 'Еврейская АО'

data_users.loc[data_users.loc[:,'user_city'] == 'Омск', 'region'] = 'Омская обл.'
data_users.loc[data_users.loc[:,'user_city'] == 'Барнаул', 'region'] = 'Алтайский край'
data_users.loc[data_users.loc[:,'user_city'] == 'Новокузнецк', 'region'] = 'Кемеровская обл.'

data_users.loc[data_users.loc[:,'user_city'] == 'Петрозаводск', 'region'] = 'Республика Карелия'
data_users.loc[data_users.loc[:,'user_city'] == 'Архангельск', 'region'] = 'Архангельская обл.'
data_users.loc[data_users.loc[:,'user_city'] == 'Астрахань', 'region'] = 'Астраханская обл.'


In [21]:
# Словарь для замены значений

dict_1 = {'Пензенская':'Пензенская обл.', 'Москва и Московская обл.':'Московская обл.',
          'Санкт-Петербург и область':'Ленинградская обл.',
          'Ленинградская область':'Ленинградская обл.',
          'Нижегородская (Горьковская)':'Нижегородская обл.',
          'Рязанская область ':'Рязанская обл.', 'Башкортостан(Башкирия)':'Башкортостан',
          'Омская область':'Омская обл.', 'Санкт-Петербург':'Ленинградская обл.',
          'Республика Башкортостан':'Башкортостан', 'Ульяновская':'Ульяновская обл.',
          'undefined':'-', '1':'-', 'Выберите':'-'}


In [22]:
# Замена значений

# data_users['region'] = data_users['region'].map(dict_1)

data_users = data_users.replace({"region":dict_1})

In [23]:
# Замена пропущенных значений 

data_users['region'] = data_users['region'].fillna('-')

In [24]:
# Проверка выполнения операций

data_users['region'].nunique()

115

In [25]:
# Подсчёт числа уникальных городов зрителей

data_users['user_city'].nunique()

257

In [26]:
# Вывод уникальных городов зрителей

data_users['user_city'].unique()

array([nan, 'Ижевск', 'Кишинев', 'Махачкала', 'Павлодар',
       'Санкт-Петербург', 'Тирасполь', 'Гусь Хрустальный', 'Белгород',
       'Ростов-на-Дону', 'Москва', 'Сыктывкар', 'Зеленогорск', 'Вологда',
       'Новосибирск', 'Гродно', 'Кемерово', 'Уфа', 'Минск', 'Нальчик',
       'Омск', 'Симферополь', 'Рязань', 'Череповец', 'еее', 'Калининград',
       'Балларат', 'Нижний Новгород', 'Брест', 'Элиста', 'Казань',
       'Астрахань', 'Энгельс', 'Воронеж', 'Ярославль', 'Чебоксары',
       'Краснодар', 'Владикавказ', 'Архангельск', 'Хоринск', 'Улан-Удэ',
       'Сочи', 'Пенза', 'Якутск', 'Самара', 'Тюмень', 'Волгоград',
       'Брянск', 'Челябинск', 'undefined', 'Луганск', 'вар', 'Пермь',
       'Томск', 'Сургут', 'Армавир', 'Ванино', 'Муром', 'Великие Луки',
       'Раменское', 'Тетюши', 'Чимкент', 'Палласовка', 'Ереван',
       'Красноярск', 'Донецк', 'Бишкек', 'Ульяновск', 'Орск',
       'Екатеринбург', 'Владимир', 'Городище', 'Владивосток', 'Азов',
       'Иваново', 'Оренбург', 'Бийск'

In [27]:
# Вывод подозрительных значений

# data_users.query('user_city == "еее"')
# data_users.query('user_city == "undefined"')
# data_users.query('user_city == "вар"')
# data_users.query('user_city == "Городище"')
# data_users.query('user_city == "  "')
# data_users.query('user_city == " "')
# data_users.query('user_city == "Выберите"')
# data_users.query('user_city == "1"')

In [28]:
# Словарь для замены значений

dict_2 = {'Чимкент':'Шымкент', 'санкт-петербург':'Санкт-Петербург',
          'Санкт-Петербург ':'Санкт-Петербург', 'Санкт – Петербург':'Санкт-Петербург',
          'Волгоград ':'Волгоград', 'барнаул':'Барнаул', 'Алма-Ата':'Алматы',
          'НижНов':'Нижний Новгород', 'Пенза ':'Пенза', 'Ю.Сахалинск':'Южно-Сахалинск',
          'еее':'-', 'вар':'-', ' ':'-', '  ':'-', 'undefined':'-', '1':'-', 'Выберите':'-'}


In [29]:
# Замена значений

data_users = data_users.replace({"user_city":dict_2})

In [30]:
# Замена пропущенных значений 

data_users['user_city'] = data_users['user_city'].fillna('-')

In [31]:
# Проверка выполнения операций

data_users['user_city'].nunique()

241

In [32]:
# Вывод уникальных записей профессий зрителей

data_users['profession'].unique()

array(['Врач', nan, 'Представитель медицинской компании',
       'Ординатор - Аспирант', 'Руководящее звено клиники',
       'Студент медицинского ВУЗа', 'Фармацевт', 'Другое',
       'Средний медперсонал', 'врач',
       'Представитель общественной организации',
       'обучающийся по медицинской специальности',
       'руководящее звено клиники', 'другое', 'Медперсонал',
       'Исследователь', 'исследователь', 'немедициснкий персонал клиники',
       'представитель медицинской компании', 'средний медпероснал',
       'онколог', 'Онколог', '  ', 'врач-онколог', 'организатор',
       'врач ХТ',
       'Специалист отдела организации доклинических и клинических исследований НМИЦ онкологии им. Н. Н. Петрова',
       'зав отд. ХТ', 'Онколог-гинеколог ',
       'Немедицинский персонал клиники', 'онкогинеколог',
       'врач, химиотерапевт', 'клинический фармаколог',
       'зав. ХТ, химиотерапевт', 'врач-гематолог',
       'онколог, клин.фармаколог', 'врач - онколог, химиотерапевт',
      

In [33]:
# Подсчёт уникальных записей профессий зрителей

data_users['profession'].nunique()

87

In [34]:
# Словарь для замены значений

dict_3 = {'обучающийся по медицинской специальности':'Студент',
          'Студент медицинского ВУЗа':'Студент', 'врач - онколог, химиотерапевт':'Химиотерапевт',
          'врач-онколог,химиотерапевт':'Химиотерапевт',
          'врач онколог, химиотерапиевт':'Химиотерапевт', 'онколог , ХТ':'Химиотерапевт',
          'Онколог, химиотерапевт':'Химиотерапевт', 'врач-онколог':'Онколог',
          'руководитель ХТ службы 1го стационара, врач- онколог, химиотерапевт':'Химиотерапевт',
          'Врач-онколог':'Онколог', 'Врач -онколог':'Онколог',
          'онколог, клин.фармаколог':'Онколог', 'гл. онколог':'Онколог',
          'врач-онколог ( зав.ДС ХТО)':'Онколог', 'Заведующая отделением, онколог':'Онколог',
          'зав. поликлинического отд., онколог':'Онколог', '':'Онколог',
          'онкогинеколог':'Онколог-гинеколог', 'Онколог-гинеколог ':'Онколог-гинеколог',
          'онкоуролог':'Онколог-уролог', 'врач узд':'Врач УЗД', 'УЗИ':'Врач УЗД',
          'нач мед':'Руководящее звено клиники', 'нач. мед':'Руководящее звено клиники',
          'Зам гл врача':'Руководящее звено клиники', 'Зам.гл.врач':'Руководящее звено клиники',
          'врач ХТ':'Химиотерапевт', 'врач, химиотерапевт':'Химиотерапевт',
          'зав отд. ХТ':'Зав. отделением химиотерапии',
          'Руководитель группы проектов &quot;Все не напрасно&quot;':'Руководитель группы проектов',
          'зав.дн.хт стационаром':'Зав. отделением химиотерапии',
          'зав отдел х/т':'Зав. отделением химиотерапии',
          'зав. отд. химиотерапии':'Зав. отделением химиотерапии',
          'зав. ХТ, химиотерапевт':'Зав. отделением химиотерапии',
          'зав.отделением ХТ, врач, химиотерапевт':'Зав. отделением химиотерапии',
          'х/т зав отд':'Зав. отделением химиотерапии', 'зав.отд':'Зав. отделением',
          'Зав. Отделением':'Зав. отделением', 'Заведующая':'Зав. отделением',
          'зав.1 онкологическим отделением':'Зав. отделением',
          'средний медпероснал':'Медперсонал', 'Средний медпероснал':'Медперсонал',
          'Средний медперсонал':'Медперсонал',  'мед сестра':'Медперсонал',
          'старшая операционная сестра':'Медперсонал', 'Перевязочная м/с':'Медперсонал',
          'клинический фармаколог':'Фармаколог',  '  ':'-', ' ':'-', '1':'-',
          'Другое':'-',  'другое':'-'}


In [35]:
# Замена значений

data_users = data_users.replace({"profession":dict_3})

In [36]:
# Унификация записей

data_users['profession'] = data_users['profession'].str.capitalize()

In [37]:
# Замена пропущенных значений 

data_users['profession'] = data_users['profession'].fillna('-')

In [38]:
# Проверка выполнения операций

data_users['profession'].nunique()

38

In [39]:
# Проверка выполнения операций

data_users['profession'].unique()

array(['Врач', '-', 'Представитель медицинской компании',
       'Ординатор - аспирант', 'Руководящее звено клиники', 'Студент',
       'Фармацевт', 'Медперсонал',
       'Представитель общественной организации', 'Исследователь',
       'Немедициснкий персонал клиники', 'Онколог', 'Организатор',
       'Химиотерапевт',
       'Специалист отдела организации доклинических и клинических исследований нмиц онкологии им. н. н. петрова',
       'Зав. отделением химиотерапии', 'Онколог-гинеколог',
       'Немедицинский персонал клиники', 'Фармаколог', 'Врач-гематолог',
       'В.н.с. отделением радионуклидной диагностики',
       'Зав.отделением рнд', 'Зам.гл.вр.по хир.помощи', 'Зав. отделением',
       'Врач-онколог, хирург-маммолог', 'Врач узд', 'Журналист',
       'Онкомаммолог', 'Фельдшер',
       'Представитель пациентской организации',
       'Заведующая отделением молочной железы',
       'Руководитель отделения торакальной онкологии',
       'Зав. отд. радионуклидной диагностики',
    

In [40]:
# Подсчёт числа уникальных записей специализаций зрителей

data_users['specialization'].nunique()

109

In [41]:
# Вывод уникальных записей профессий зрителей

data_users['specialization'].unique()

array(['онкология: хирургия (огш)', nan, 'Другое',
       'патологическая анатомия', 'Хирург онколог',
       'онкология: хирургия (абдоминальная)',
       'лучевая диагностика: рентгенология', 'Химиотерапевт',
       'Онколог-педиатр', 'хирургия', 'лучевая диагностика: УЗД',
       'радиология', 'Радиотерапевт',
       'онкология: хирургия (кости и мягкие ткани)',
       'онкология: лекарственное лечение',
       'Врач ультразвуковой диагностики', 'гематология',
       'онкология: хирургия (рмж)', 'лучевая диагностика: МРТ',
       'Патоморфолог', 'онкология', 'эндоскопия',
       'лучевая терапия (радиотерапия)', 'педиатрия',
       'онкология: хирургия (урология)', 'анестезиология-реаниматология',
       'клиническая лабораторная диагностика', 'генетика',
       'Лучевая диагностика', 'диетология', 'другое',
       'рентгенэндоваскулярная диагностика и лечение',
       'лучевая диагностика: интервенционная радиология',
       'нейроонкология', 'нейрохирургия',
       'онкология: хир

In [42]:
# Словарь для замены значений

dict_4 = {'Радиотерапевт':'Лучевая терапия (радиотерапия)',
          'Врач ультразвуковой диагностики':'Лучевая диагностика: УЗД',
          'торакальная хирургия':'Онкология: хирургия (торакальная)',
          'хирург-онколог':'Хирург онколог', 'онколог':'Онкология', 'онколоигя':'Онкология',
          'врач-онколог':'Онкология', 'Патоморфолог':'Патоморфология',
          'патологоанатом':'Патологическая анатомия',
          'лаборант, патолог':'Патологическая анатомия', 'уролог':'Урология',
          'Гинекология':'Акушерство и гинекология', 'Гинеколог':'Акушерство и гинекология',
          'онкология детская':'Детская онкология', 'врач онколог, химиотерапиевт':'Химиотерапевт',
          'абдоминальная хирургия':'Онкология: хирургия (абдоминальная)',
          'Средний медперсонал':'Медперсонал', ' ':'-', '  ':'-', '1':'-', 'Другое':'-',
          'другое':'-'}


In [43]:
# Замена значений

data_users = data_users.replace({"specialization":dict_4})

In [44]:
# Унификация записей

data_users['specialization'] = data_users['specialization'].str.capitalize()

In [45]:
# Замена пропущенных значений 

data_users['specialization'] = data_users['specialization'].fillna('-')

In [46]:
# Проверка выполнения операций

data_users['specialization'].nunique()

82

In [47]:
# Вывод уникальных типов устройств зрителей

data_users['device'].unique()

array(['Десктоп', nan, 'Мобильное', 'Планшет'], dtype=object)

In [48]:
# Замена пропущенных значений 

data_users['device'] = data_users['device'].fillna('-')

In [49]:
# Вывод номеров потоков трансляций по популярности

data_users['potok'].value_counts()

1.0     42781
2.0     23196
3.0     21393
4.0     16975
5.0     11031
6.0      8391
7.0      4486
10.0     2646
8.0      2352
9.0      1495
11.0      948
Name: potok, dtype: int64

In [50]:
# Вывод видов записи номеров потоков трансляций

data_users['potok'].unique()

array([ 1., nan,  3.,  4.,  2.,  7.,  5.,  6.,  8., 10.,  9., 11.])

In [51]:
# Замена пропущенных значений 

data_users['potok'] = data_users['potok'].fillna('-')

Для последующих операций необходимо ввести поправку в значение временной метки (Timestamp) - прибавить к значению 3 часа, т.е., 10800 секунд. 


In [52]:
# Вывод минимального значения начала просмотра

data_users['user_timestamp_start'].min()

-16070400

In [53]:
# Вывод минимального значения окончания просмотра

data_users['user_timestamp_end'].min()

-16070400

In [54]:
# Введение поправки

data_users['user_timestamp_start'] = data_users['user_timestamp_start'] + 10800
data_users['user_timestamp_end'] = data_users['user_timestamp_end'] + 10800

In [55]:
# Проверка выполнения операции

data_users['user_timestamp_start'].min()

-16059600

In [56]:
# Проверка выполнения операции

data_users['user_timestamp_end'].min()

-16059600

In [57]:
# Функция округления даты до дней

def date_only(cell):
    cell = str(cell).split()[0]
    return cell 

In [58]:
# Применение функции округления даты до дней

data_users['user_date'] = data_users['user_date_start'].apply(date_only)

In [59]:
# Просмотр популярности дат просмотров

data_users['user_date'].value_counts()

01.07.2022                                                                                                                                                                                                                                                         17688
02.07.2022                                                                                                                                                                                                                                                         14842
28.04.2023                                                                                                                                                                                                                                                         13866
29.04.2023                                                                                                                                                                                                   

In [60]:
data_users.head(2)

,user_id,o_id,barcode,country,region,user_city,profession,specialization,potok,device,user_date_start,user_date_end,user_timestamp_start,user_timestamp_end,event,user_date
0,16573.0,626199.0,0809202204421616573,Россия,Амурская обл.,-,Врач,Онкология: хирургия (огш),1.0,Десктоп,13.03.2022 9:42,13.03.2022 9:44,1647164550,1647164657,Мероприятие 1,13.03.2022
1,16573.0,626199.0,0809202204421616573,Россия,Амурская обл.,-,Врач,Онкология: хирургия (огш),1.0,Десктоп,13.03.2022 9:51,13.03.2022 9:51,1647165094,1647165097,Мероприятие 1,13.03.2022


In [61]:
data_users.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 154333 entries, 0 to 154332
Data columns (total 16 columns):
 #   Column                Non-Null Count   Dtype  
---  ------                --------------   -----  
 0   user_id               134779 non-null  float64
 1   o_id                  135694 non-null  float64
 2   barcode               135694 non-null  object 
 3   country               154333 non-null  object 
 4   region                154333 non-null  object 
 5   user_city             154333 non-null  object 
 6   profession            154333 non-null  object 
 7   specialization        154333 non-null  object 
 8   potok                 154333 non-null  object 
 9   device                154333 non-null  object 
 10  user_date_start       154333 non-null  object 
 11  user_date_end         154333 non-null  object 
 12  user_timestamp_start  154333 non-null  int64  
 13  user_timestamp_end    154333 non-null  int64  
 14  event                 154333 non-null  object 
 15  

В датасете есть строки с отсутствующими данными о зрителе. Рассмотрю такие строки подробнее

In [62]:
# Замена пропущенных значений для последующего просмотра соотвествующих строчек датасета

data_users['user_id'] = data_users['user_id'].fillna('no_user_id')
data_users['o_id'] = data_users['o_id'].fillna('no_o_id')
data_users['barcode'] = data_users['barcode'].fillna('no_barcode')

In [63]:
# Просмотр строчек датасета в которых нет id зрителя, но есть другие данные о зрителе

data_users.query('(user_id == "no_user_id") and (o_id != "no_o_id") and (barcode != "no_barcode")')\
                .sample()

print('Строк без user_id, но с другими данными о зрителе:',
      len(data_users\
          .query('(user_id == "no_user_id") and (o_id != "no_o_id") and (barcode != "no_barcode")')))

print()

print('Строк без данных о зрителе:',
      len(data_users\
          .query('(user_id == "no_user_id") and (o_id == "no_o_id") and (barcode == "no_barcode")')))


Строк без user_id, но с другими данными о зрителе: 915

Строк без данных о зрителе: 18639


In [64]:
# Выделение строк без user_id, но с другими данными о зрителе

data_no_user_id = data_users.query('(user_id == "no_user_id")\
                                and (o_id != "no_o_id") and (barcode != "no_barcode")')

In [65]:
# Удаление из датасета строк без данных о зрителе

data_users = data_users.query('(user_id != "no_user_id")\
                            and (o_id != "no_o_id") and (barcode != "no_barcode")')

In [66]:
# Формирование датасета с какими-либо данными о зрителе

data_users = pd.concat([data_users, data_no_user_id], axis = 0)

In [67]:
# Просмотр формата записи дат

data_users['user_date'].unique()

array(['13.03.2022', '14.03.2022', '29.01.2023', '16.03.2023',
       '15.04.2023', '21.07.2022', '22.07.2022', '31.10.2022',
       '05.07.2022', '06.07.2022', '07.07.2022', '01.07.2022',
       '02.07.2022', '03.07.2022', '04.07.2022', '27.04.2023',
       '28.04.2023', '29.04.2023', '30.04.2023', '02.05.2023',
       '01.05.2023'], dtype=object)

Для последующего объединения датасетов необходим столбец с датами в формате "год-месяц-день". Создаю его.

In [68]:
# Функция для переформатирования даты в другой вид

def date_format(cell):
    result = cell.split('.')
    result = result[2]+ '-' + result[1] + '-' + result[0]
    return result

In [69]:
# Добавление столбца для последующих преобразований

data_users['date'] = data_users['user_date'].apply(date_format)

In [70]:
# Контроль выполнения операции

data_users['date'].unique()

array(['2022-03-13', '2022-03-14', '2023-01-29', '2023-03-16',
       '2023-04-15', '2022-07-21', '2022-07-22', '2022-10-31',
       '2022-07-05', '2022-07-06', '2022-07-07', '2022-07-01',
       '2022-07-02', '2022-07-03', '2022-07-04', '2023-04-27',
       '2023-04-28', '2023-04-29', '2023-04-30', '2023-05-02',
       '2023-05-01'], dtype=object)

In [71]:
# Перестановка столбцов для удобства при последующем объединении датасетов и просмотре строк

data_users = data_users[['date', 'potok', 'user_date', 'user_id', 'o_id', 'barcode', 'device',
                         'country', 'region', 'user_city', 'profession', 'specialization',
                         'event', 'user_timestamp_start', 'user_timestamp_end',
                         'user_date_start', 'user_date_end']]

In [72]:
# Проверка наличия дубликатов

data_users.duplicated().sum()

1657

In [73]:
# Удаление дубликатов

data_users = data_users.drop_duplicates() 

In [74]:
# Контроль выполнения операций

data_users.head(1)

,date,potok,user_date,user_id,o_id,barcode,device,country,region,user_city,profession,specialization,event,user_timestamp_start,user_timestamp_end,user_date_start,user_date_end
0,2022-03-13,1.0,13.03.2022,16573.0,626199.0,0809202204421616573,Десктоп,Россия,Амурская обл.,-,Врач,Онкология: хирургия (огш),Мероприятие 1,1647164550,1647164657,13.03.2022 9:42,13.03.2022 9:44


In [75]:
# Контроль выполнения операций

data_users.duplicated().sum()

0

In [76]:
# Число строк датасета

len(data_users)

134037

In [77]:
data_users['date'].info()

<class 'pandas.core.series.Series'>
Int64Index: 134037 entries, 0 to 147507
Series name: date
Non-Null Count   Dtype 
--------------   ----- 
134037 non-null  object
dtypes: object(1)
memory usage: 2.0+ MB


## Обработка датасетов с расписаниями выступлений на крупных форумах

#### Краткое описание операций при обработке датасета с расписаниями выступлений


Объединение датасетов.

Унификация и исправление ошибок в обозначении потоков трансляций.

Замена пропущенных значений в столбцах со временем и датой символом "-".

Замена пропущенных значений в столбцах с наименованием доклада, докладчика, города докладчика, организации докладчика на обозначения: "no_report_name", "no_speaker_name", "no_city_event", "no_organization" соответственно.

Просмотр и отсечение строк без даты/времени начала и окончания мероприятия, без наименования доклада.

Просмотр строк без наименования докладчика, города докладчика, организации докладчика.

Введение вспомогательных столбцов с датой без часов и минут.

Исправление некорректных значений времени начала и окончания трансляций.

Введение вспомогательных столбцов с временем начала и окончания трансляций в формате Timestamp.


In [78]:
# Открытие и просмотр файла Расписание выступлений 11

data_event_11 = pd.read_excel('Расписание выступлений 11.xlsx')

data_event_11.head()

,Время начала\n(UTC+3 МСК),Время окончания \n(UTC+3 МСК),Продолжительность,Дата,Поток,Название доклада,ФИО полностью,Учреждение/Организация,Город
0,10:00:00,10:20:00,00:20:00,2022-07-01 00:00:00,1 поток,Тема 54,Самборский Илья Филиппович,NaN,Санкт-Петербург
1,10:20:00,10:25:00,00:05:00,2022-07-01 00:00:00,1 поток,Дискуссия,NaN,NaN,NaN
2,10:25:00,10:45:00,00:20:00,2022-07-01 00:00:00,1 поток,Тема 56,Шашина Марина Константиновна,NaN,Москва
3,10:45:00,10:50:00,00:05:00,2022-07-01 00:00:00,1 поток,Дискуссия,NaN,NaN,NaN
4,10:50:00,11:10:00,00:20:00,2022-07-01 00:00:00,1 поток,Тема 57,Кабылбекова Ангелина Руслановна,NaN,Санкт-Петербург


In [79]:
# Переименование столбцов

data_event_11 = data_event_11.rename(columns={'Время начала\n(UTC+3 МСК)':'timestart_event',
                           'Время окончания \n(UTC+3 МСК)':'timeend_event',
                           'Продолжительность':'duration_event',
                           'Дата':'date_event',
                           'Поток':'potok',
                           'Название доклада':'report_name',
                           'ФИО полностью':'speaker_name',
                           'Учреждение/Организация':'organization',
                           'Город':'city_event'})

In [80]:
# Контроль выполнения переименования

data_event_11.head(1)

,timestart_event,timeend_event,duration_event,date_event,potok,report_name,speaker_name,organization,city_event
0,10:00:00,10:20:00,00:20:00,2022-07-01 00:00:00,1 поток,Тема 54,Самборский Илья Филиппович,NaN,Санкт-Петербург


In [81]:
# Просмотр информации о датасете

data_event_11.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1350 entries, 0 to 1349
Data columns (total 9 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   timestart_event  1331 non-null   object 
 1   timeend_event    1330 non-null   object 
 2   duration_event   1312 non-null   object 
 3   date_event       1349 non-null   object 
 4   potok            1347 non-null   object 
 5   report_name      1339 non-null   object 
 6   speaker_name     744 non-null    object 
 7   organization     0 non-null      float64
 8   city_event       733 non-null    object 
dtypes: float64(1), object(8)
memory usage: 95.0+ KB


In [82]:
# Открытие и просмотр файла Расписание выступлений 12

data_event_12 = pd.read_excel('Расписание выступлений 12.xlsx')

data_event_12.head()

,Время начала\n(UTC+3 МСК),Время окончания \n(UTC+3 МСК),Продолжительность,Дата,Поток,Название доклада,ФИО полностью,Учреждение/Организация,Город
0,11:00:00,11:05:00,00:05:00,2023-04-27,-,Приветственное слово,Клюйко Эльмира Григорьевна,NaN,Санкт-Петербург
1,11:05:00,11:10:00,00:05:00,2023-04-27,-,Приветственное слово,Дударенко Аркадий Глебович,NaN,Москва
2,11:10:00,11:15:00,00:05:00,2023-04-27,-,Приветственное слово,Бобрищев Марат Ринатович,NaN,NaN
3,11:15:00,11:20:00,00:05:00,2023-04-27,-,Приветственное слово,Толстоброва Виктория Тимофеевна,NaN,Санкт-Петербург
4,11:20:00,11:25:00,00:05:00,2023-04-27,-,Приветственное слово,Карабаева Жанна Георгиевна,NaN,Санкт-Петербург


In [83]:
# Переименование столбцов

data_event_12 = data_event_12.rename(columns={'Время начала\n(UTC+3 МСК)':'timestart_event',
                           'Время окончания \n(UTC+3 МСК)':'timeend_event',
                           'Продолжительность':'duration_event',
                           'Дата':'date_event',
                           'Поток':'potok',
                           'Название доклада':'report_name',
                           'ФИО полностью':'speaker_name',
                           'Учреждение/Организация':'organization',
                           'Город':'city_event'})

In [84]:
# Контроль выполнения переименования

data_event_12.head(1)

,timestart_event,timeend_event,duration_event,date_event,potok,report_name,speaker_name,organization,city_event
0,11:00:00,11:05:00,00:05:00,2023-04-27,-,Приветственное слово,Клюйко Эльмира Григорьевна,NaN,Санкт-Петербург


In [85]:
# Просмотр информации о датасете

data_event_12.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1567 entries, 0 to 1566
Data columns (total 9 columns):
 #   Column           Non-Null Count  Dtype         
---  ------           --------------  -----         
 0   timestart_event  1520 non-null   object        
 1   timeend_event    1520 non-null   object        
 2   duration_event   1511 non-null   object        
 3   date_event       1567 non-null   datetime64[ns]
 4   potok            1567 non-null   object        
 5   report_name      1533 non-null   object        
 6   speaker_name     907 non-null    object        
 7   organization     0 non-null      float64       
 8   city_event       894 non-null    object        
dtypes: datetime64[ns](1), float64(1), object(7)
memory usage: 110.3+ KB


In [86]:
# Объединение датасетов

data_events = pd.concat([data_event_11, data_event_12], axis = 0)

In [87]:
# Проверка выполнения операций

data_events.head(1)

,timestart_event,timeend_event,duration_event,date_event,potok,report_name,speaker_name,organization,city_event
0,10:00:00,10:20:00,00:20:00,2022-07-01 00:00:00,1 поток,Тема 54,Самборский Илья Филиппович,NaN,Санкт-Петербург


In [88]:
# Просмотр значений столбца и их встречаемости

data_events['potok'].value_counts()

3 поток            481
1 поток            457
2 поток            425
4 поток            388
5 поток            322
6 поток            222
7 поток            188
8 поток            126
-                   56
3 поток             41
2 поток             41
10.1 поток          38
4 поток             37
1 поток             35
5 поток             23
6.1 поток           17
9 поток              7
10 поток             7
Поток                1
6 поток              1
Тренинг-комната      1
Name: potok, dtype: int64

In [89]:
# Просмотр уникальных значений столбца

data_events['potok'].unique()

array(['1 поток ', '2 поток ', '3 поток ', '4 поток ', '5 поток ',
       '6 поток ', '1 поток', '2 поток', '3 поток', '4 поток', '5 поток',
       '6 поток', nan, '6.1 поток', 'Поток', '-', '7 поток', '8 поток',
       'Тренинг-комната', '9 поток', '10 поток', '10.1 поток'],
      dtype=object)

In [90]:
# Замена пропущенных значений

data_events['potok'] = data_events['potok'].fillna("-")

In [91]:
# Функция замены записей в столбце

def potok_changing(cell):
    if cell == '6.1 поток':
        result = 6    
    elif cell == '10.1 поток':
        result = 11       
    elif cell == '-':
        result = cell
    elif cell == 'Тренинг-комната':
        result = cell
    elif cell == 'Поток':
        result = cell
    else:
        result = cell.split()[0]
        result = int(result)
    return result
    

In [92]:
# Применение функции замены записей столбца

data_events['potok_new'] = data_events['potok'].apply(potok_changing)

In [93]:
# Контроль выполнения операций

data_events['potok_new'].unique()

array([1, 2, 3, 4, 5, 6, '-', 'Поток', 7, 8, 'Тренинг-комната', 9, 10, 11],
      dtype=object)

In [94]:
# Просмотр подозрительного значения

data_events.query('potok_new == "Поток"')

,timestart_event,timeend_event,duration_event,date_event,potok,report_name,speaker_name,organization,city_event,potok_new
1176,Время начала\n(UTC+3 МСК),Время окончания \n(UTC+3 МСК),Продолжительность,NaN,Поток,Тема 620,NaN,NaN,Город,Поток


In [95]:
# Простотр строк с пропущенным значением потока

data_events.query('potok_new == "-"')

,timestart_event,timeend_event,duration_event,date_event,potok,report_name,speaker_name,organization,city_event,potok_new
382,Поток №1,NaN,NaN,4,-,NaN,NaN,NaN,NaN,-
395,12:00:00,12:30:00,00:30:00,1900-01-04 00:00:00,-,Перерыв в потоке,NaN,NaN,NaN,-
1163,14:45:00,15:00:00,00:15:00,1900-01-04 00:00:00,-,Перерыв в потоке,NaN,NaN,NaN,-
0,11:00:00,11:05:00,00:05:00,2023-04-27 00:00:00,-,Приветственное слово,Клюйко Эльмира Григорьевна,NaN,Санкт-Петербург,-
1,11:05:00,11:10:00,00:05:00,2023-04-27 00:00:00,-,Приветственное слово,Дударенко Аркадий Глебович,NaN,Москва,-
2,11:10:00,11:15:00,00:05:00,2023-04-27 00:00:00,-,Приветственное слово,Бобрищев Марат Ринатович,NaN,NaN,-
3,11:15:00,11:20:00,00:05:00,2023-04-27 00:00:00,-,Приветственное слово,Толстоброва Виктория Тимофеевна,NaN,Санкт-Петербург,-
4,11:20:00,11:25:00,00:05:00,2023-04-27 00:00:00,-,Приветственное слово,Карабаева Жанна Георгиевна,NaN,Санкт-Петербург,-
5,11:25:00,11:30:00,00:05:00,2023-04-27 00:00:00,-,Приветственное слово,Жидович Максим Яковлев,NaN,Санкт-Петербург,-
6,11:30:00,11:35:00,00:05:00,2023-04-27 00:00:00,-,Приветственное слово,Курляева Инна Сергеевна,NaN,Москва,-


In [96]:
# Просмотр строки с событием "Тренинг-комната"

data_events.query('potok_new == "Тренинг-комната"')

,timestart_event,timeend_event,duration_event,date_event,potok,report_name,speaker_name,organization,city_event,potok_new
345,13:30:00,17:30:00,04:00:00,2023-04-28 00:00:00,Тренинг-комната,Тема 871,NaN,NaN,NaN,Тренинг-комната


In [97]:
# Замена пропущенных значений

data_events['timestart_event'] = data_events['timestart_event'].fillna("-")
data_events['timeend_event'] = data_events['timeend_event'].fillna("-")
data_events['duration_event'] = data_events['duration_event'].fillna("-")
data_events['date_event'] = data_events['date_event'].fillna("-")

data_events['report_name'] = data_events['report_name'].fillna("no_report_name")
data_events['speaker_name'] = data_events['speaker_name'].fillna("no_speaker_name")
data_events['organization'] = data_events['organization'].fillna("no_organization")
data_events['city_event'] = data_events['city_event'].fillna("no_city_event")


In [98]:
# Просмотр организаций докладчика

data_events['organization'].unique()

array(['no_organization'], dtype=object)

In [99]:
# Замена отсутствующих значений

data_events['organization'] = data_events['organization'].replace('-', 'no_organization')
data_events['organization'] = data_events['organization'].replace(0, 'no_organization')

In [100]:
# Удаление ненужной строки

data_events = data_events.query('timestart_event != "Поток №1"')

Последовательно вывожу строки с отстутствием каких-либо данных


In [101]:
# Строки без дат начала мероприятия

data_events.query('timestart_event == "-"').sample(3)

,timestart_event,timeend_event,duration_event,date_event,potok,report_name,speaker_name,organization,city_event,potok_new
855,-,-,-,2023-04-30 00:00:00,5 поток,no_report_name,Сеченова Альбина Николаевна,no_organization,Москва,5
890,-,-,-,2022-07-05 00:00:00,3 поток,no_report_name,Уваркин Роберт Федорович,no_organization,Санкт-Петербург,3
791,-,-,-,2023-04-30 00:00:00,4 поток,no_report_name,Гордон Константин Алексеевич,no_organization,Москва,4


In [102]:
# Выделение в отдельный датасет строк с особыми событиями,
# без указанных времени начала и окончания доклада и пр. 

data_other_events = data_events.query('timestart_event == "-"')

In [103]:
# Проверка выполнения операции

data_other_events.head(1)

,timestart_event,timeend_event,duration_event,date_event,potok,report_name,speaker_name,organization,city_event,potok_new
94,-,-,-,2022-07-01 00:00:00,3 поток,no_report_name,Лавриненков Роберт Ефимович,no_organization,Санкт-Петербург,3


In [104]:
# Строки без дат мероприятий

data_events.query('date_event == "-"')

,timestart_event,timeend_event,duration_event,date_event,potok,report_name,speaker_name,organization,city_event,potok_new
1176,Время начала\n(UTC+3 МСК),Время окончания \n(UTC+3 МСК),Продолжительность,-,Поток,Тема 620,no_speaker_name,no_organization,Город,Поток


In [105]:
# Просмотр наиболее часто встречаемых названий доклада

data_events['report_name'].value_counts().head(3)

Дискуссия           1003
Перерыв в потоке     179
no_report_name        44
Name: report_name, dtype: int64

In [106]:
# Удаление ненужных строк

data_events = data_events.query('date_event != "-"')
data_events = data_events.query('timestart_event != "-"')

data_events = data_events.query('report_name != "Перерыв в потоке"')

In [107]:
# Просмотр наиболее часто встречаемых названий доклада

data_events['report_name'].value_counts().head(10)

Дискуссия                            1002
Приветственное слово                   28
Тема 1374                              27
Тема 927                               20
Общая дискуссия участников сессии      18
Тема 127                               13
Тема 619                               12
Тема 388                               12
Тема 106                               11
Тема 665                               11
Name: report_name, dtype: int64

In [108]:
# Строки без названия мероприятий

data_events.query('report_name == "no_report_name"')

,timestart_event,timeend_event,duration_event,date_event,potok,report_name,speaker_name,organization,city_event,potok_new
30,16:30:00,16:45:00,00:15:00,2022-07-01 00:00:00,1 поток,no_report_name,Коловратов Тимур Егорович,no_organization,Москва,1
352,11:15:00,12:15:00,01:00:00,2022-07-02 00:00:00,5 поток,no_report_name,Лутонина Татьяна Вячеславовна,no_organization,Москва,5
353,11:15:00,12:15:00,01:00:00,2022-07-02 00:00:00,5 поток,no_report_name,Пелымцев Геннадий Никитович,no_organization,Москва,5
654,17:10:00,17:20:00,00:10:00,2022-07-04 00:00:00,2 поток,no_report_name,Буданова Раиса Ринатовна,no_organization,Москва,2
685,15:35:00,15:50:00,00:15:00,2022-07-04 00:00:00,3 поток,no_report_name,no_speaker_name,no_organization,no_city_event,3


In [109]:
# Удаление ненужных строк

data_events = data_events.query('report_name != "no_report_name"')

In [110]:
# Строки без имени докладчика

data_events.query('speaker_name == "no_speaker_name"').sample(3)

,timestart_event,timeend_event,duration_event,date_event,potok,report_name,speaker_name,organization,city_event,potok_new
397,12:50:00,12:55:00,00:05:00,2022-07-03 00:00:00,1 поток,Дискуссия,no_speaker_name,no_organization,no_city_event,1
126,11:35:00,11:40:00,00:05:00,2022-07-01 00:00:00,4 поток,Дискуссия,no_speaker_name,no_organization,no_city_event,4
478,11:55:00,12:00:00,00:05:00,2023-04-29 00:00:00,4 поток,Дискуссия,no_speaker_name,no_organization,no_city_event,4


In [111]:
# Строки без наименования организации докладчика

data_events.query('organization == "no_organization"').sample(3)

,timestart_event,timeend_event,duration_event,date_event,potok,report_name,speaker_name,organization,city_event,potok_new
344,10:05:00,10:20:00,00:15:00,2022-07-02 00:00:00,5 поток,Тема 210,Бутогина Ксения Робертовна,no_organization,Санкт-Петербург,5
1056,16:15:00,16:20:00,00:05:00,2022-07-06 00:00:00,2 поток,Дискуссия,no_speaker_name,no_organization,no_city_event,2
1095,16:15:00,16:20:00,00:05:00,2022-07-06 00:00:00,3 поток,Дискуссия,no_speaker_name,no_organization,no_city_event,3


In [112]:
# Строки без наименования города докладчика

data_events.query('city_event == "no_city_event"').sample(3)

,timestart_event,timeend_event,duration_event,date_event,potok,report_name,speaker_name,organization,city_event,potok_new
1253,16:50:00,17:05:00,00:15:00,2023-05-01 00:00:00,7 поток,Тема 1315,no_speaker_name,no_organization,no_city_event,7
1364,15:35:00,15:40:00,00:05:00,2023-05-02 00:00:00,1 поток,Дискуссия,no_speaker_name,no_organization,no_city_event,1
253,11:40:00,11:45:00,00:05:00,2023-04-28 00:00:00,6 поток,Дискуссия,no_speaker_name,no_organization,no_city_event,6


In [113]:
# Удаление ненужного столбца

data_events.drop('potok', axis= 1 , inplace= True )

In [114]:
# Просмотр датасета после преобразований

data_events.head(2)

,timestart_event,timeend_event,duration_event,date_event,report_name,speaker_name,organization,city_event,potok_new
0,10:00:00,10:20:00,00:20:00,2022-07-01 00:00:00,Тема 54,Самборский Илья Филиппович,no_organization,Санкт-Петербург,1
1,10:20:00,10:25:00,00:05:00,2022-07-01 00:00:00,Дискуссия,no_speaker_name,no_organization,no_city_event,1


In [115]:
# Вывод названий городов 

data_events['city_event'].unique()

array(['Санкт-Петербург', 'no_city_event', 'Москва', 'Ставрополь',
       'Томск', 'Вильжюиф, Франция', 'Цюрих, Швейцария',
       'Нижний Новгород', 'Тяньцзинь, Китай', 'Казань', 'Ростов-на-Дону',
       'Волгоград', 'Самара', 'Екатеринбург', 'Алматы, Казахстан',
       'Ярославль', 'Алматы,Казахстан', 'Республика Беларусь',
       'Петрозаводск', 'Новосибирск', 'Нью-Йорк, США', 'Обнинск',
       'Бангалор, Индия', 'Чебоксары', 'Липецк', 'Тель-Авив, Израиль',
       'Минск', 'Краснодар', 'Буэнос-Айрес, Аргентина', 'Уфа', 'Тула',
       'Калининград', 'Архангельск', 'Пул, Великобритания',
       'Лондон, Великобритания', 'Берлин, Германия', 'Балтимор, США',
       'Тель-Авив', 'Воронеж', 'Владикавказ', 'Челябинск',
       'Филадельфия, США', 'Красноярск', 'Леон, Франция', 'Киров',
       'Гамбург, Германия', 'Лос-Анджелес, США', 'Иркутск',
       'Семей, Казахстан', 'Гатчина', 'Гомель', 'Белгород', 'Рязань',
       'Делфт, Нидерланды', 'Ольденбург, Германия', 'Мадрид, Испания',
       

In [116]:
# Вывод количества названий городов 

data_events['city_event'].nunique()

101

In [117]:
# Замена неверных значений

data_events['city_event'] = data_events['city_event'].replace('Алматы,Казахстан', 'Алматы, Казахстан')
data_events['city_event'] = data_events['city_event'].replace('Республика Беларусь', 'Беларусь')
data_events['city_event'] = data_events['city_event'].replace('Минск', 'Минск, Беларусь')
data_events['city_event'] = data_events['city_event'].replace('Тель-Авив', 'Тель-Авив, Израиль')
data_events['city_event'] = data_events['city_event'].replace('Нижний-Новгород', 'Нижний Новгород')


data_events['city_event'] = data_events['city_event'].replace(0, 'no_city_event')
data_events['city_event'] = data_events['city_event'].replace('-', 'no_city_event')
data_events['city_event'] = data_events['city_event'].replace('Город', 'no_city_event')


In [118]:
# Вывод количества названий городов после преобразований

data_events['city_event'].nunique()

95

In [119]:
# Округление дат
# Функция date_only определена в разделе обработки датасета data_users

data_events['date_new'] = data_events['date_event'].apply(date_only)

In [120]:
# Просмотр уникальных значений дат

data_events['date_new'].unique()

array(['2022-07-01', '2022-07-02', '2022-07-03', '2022-07-04',
       '2022-07-05', '2022-07-06', '2022-07-07', '2023-04-27',
       '2023-04-28', '2023-04-29', '2023-04-30', '2023-05-01',
       '2023-05-02'], dtype=object)

In [121]:
# Просмотр строк с подозрительным значением

data_events.query('date_new == "Дата"')

,timestart_event,timeend_event,duration_event,date_event,report_name,speaker_name,organization,city_event,potok_new,date_new


In [122]:
# Удаление ненужной строки

data_events = data_events.query('date_new != "Дата"')

In [123]:
# Контроль выполнения операции

data_events['date_new'].unique()

array(['2022-07-01', '2022-07-02', '2022-07-03', '2022-07-04',
       '2022-07-05', '2022-07-06', '2022-07-07', '2023-04-27',
       '2023-04-28', '2023-04-29', '2023-04-30', '2023-05-01',
       '2023-05-02'], dtype=object)

In [124]:
# Поиск неверных значений

data_events['timestart_event'].value_counts().tail(50)

09:50:00               13
17:00:00               13
17:10:00               11
17:15:00               11
12:40:00               11
09:30:00               11
09:40:00               11
17:05:00               10
12:20:00               10
17:20:00               10
17:35:00                8
14:50:00                8
17:25:00                7
09:15:00                7
17:30:00                6
17:45:00                6
17:40:00                5
09:25:00                5
17:50:00                4
09:05:00                3
17:55:00                3
18:00:00                3
09:10:00                3
08:30:00                2
18:05:00                2
08:55:00                1
1900-01-03 14:25:00     1
13:12:00                1
13:02:00                1
18:55:00                1
18:40:00                1
18:35:00                1
18:20:00                1
18:15:00                1
14:47:30                1
19:30:00                1
08:40:00                1
1900-01-03 14:10:00     1
1900-01-03 1

In [125]:
# Функция замены неверно записанного времени начала доклада

def timestart_clean(cell):
    result = str(cell).split()
    if len(result) == 2:
        result = result[1]
    else:
         result = result[0]
    return result

In [126]:
# Применение функции замены неверно записанного времени начала доклада

data_events['timestart_event_new'] = data_events['timestart_event'].apply(timestart_clean)

In [127]:
# Просмотр уникальных значений времени начала доклада

data_events['timestart_event_new'].unique()

array(['10:00:00', '10:20:00', '10:25:00', '10:45:00', '10:50:00',
       '11:10:00', '11:15:00', '11:55:00', '12:15:00', '13:00:00',
       '13:15:00', '13:20:00', '13:35:00', '13:40:00', '13:55:00',
       '14:00:00', '14:15:00', '14:20:00', '14:35:00', '14:40:00',
       '14:55:00', '15:30:00', '15:45:00', '15:50:00', '16:05:00',
       '16:10:00', '16:25:00', '16:45:00', '16:50:00', '17:05:00',
       '17:10:00', '09:30:00', '09:50:00', '10:30:00', '11:00:00',
       '11:20:00', '11:30:00', '11:50:00', '16:30:00', '17:25:00',
       '17:30:00', '17:45:00', '17:50:00', '10:15:00', '10:35:00',
       '10:40:00', '10:55:00', '11:35:00', '11:40:00', '14:47:30',
       '13:10:00', '13:30:00', '14:25:00', '14:45:00', '14:50:00',
       '16:00:00', '16:40:00', '17:20:00', '13:05:00', '13:50:00',
       '15:35:00', '16:20:00', '17:00:00', '17:40:00', '12:30:00',
       '12:45:00', '13:45:00', '15:00:00', '15:15:00', '16:15:00',
       '09:45:00', '10:10:00', '11:05:00', '11:25:00', '11:45:

In [128]:
# Поиск неверных значений

data_events['timeend_event'].value_counts().tail(50)

17:05:00                 12
17:20:00                 12
09:50:00                 11
12:40:00                 11
09:40:00                 11
09:45:00                 10
18:00:00                  9
17:35:00                  9
17:25:00                  9
17:40:00                  8
09:10:00                  7
09:15:00                  6
17:45:00                  6
17:50:00                  5
09:30:00                  5
09:25:00                  4
17:55:00                  3
18:05:00                  2
до окончания операци      2
09:05:00                  2
18:15:00                  2
09:00:00                  2
до окончания операций     2
08:55:00                  1
08:45:00                  1
13:12:00                  1
13:02:00                  1
14:47:30                  1
19:00:00                  1
18:55:00                  1
18:40:00                  1
18:35:00                  1
18:20:00                  1
20:30:00                  1
19:30:00                  1
1900-01-04 14:30:00 

In [129]:
# Функция замены неверно записанного времени окончания доклада

def timeend_clean(cell):
    if str(cell) == 'до окончания операци':
        result = '18:00:00'     
    elif str(cell) == 'до окончания операций':
        result = '18:00:00'
    elif str(cell) == 'до окончания операции':
        result = '18:00:00' 
    else:
        result = str(cell).split()
        if len(result) == 2:
            result = result[1]
        else:
            result = result[0]
    return result

In [130]:
# Применение функции замены неверно записанного времени начала доклада

data_events['timeend_event_new'] = data_events['timeend_event'].apply(timeend_clean)

In [131]:
# Просмотр уникальных значений времени начала доклада

data_events['timeend_event_new'].unique()

array(['10:20:00', '10:25:00', '10:45:00', '10:50:00', '11:10:00',
       '11:15:00', '11:55:00', '12:00:00', '12:45:00', '13:15:00',
       '13:20:00', '13:35:00', '13:40:00', '13:55:00', '14:00:00',
       '14:15:00', '14:20:00', '14:35:00', '14:40:00', '14:55:00',
       '15:00:00', '15:45:00', '15:50:00', '16:05:00', '16:10:00',
       '16:25:00', '16:30:00', '16:50:00', '17:05:00', '17:10:00',
       '17:25:00', '09:50:00', '10:00:00', '10:30:00', '11:00:00',
       '11:20:00', '11:30:00', '11:50:00', '16:45:00', '17:30:00',
       '17:45:00', '17:50:00', '18:00:00', '10:15:00', '10:35:00',
       '10:40:00', '10:55:00', '11:35:00', '11:40:00', '14:47:30',
       '13:10:00', '13:30:00', '14:25:00', '14:45:00', '14:50:00',
       '16:00:00', '16:40:00', '17:20:00', '13:05:00', '13:50:00',
       '15:15:00', '15:35:00', '16:20:00', '17:00:00', '17:40:00',
       '13:00:00', '13:45:00', '14:30:00', '15:30:00', '16:15:00',
       '10:10:00', '11:05:00', '11:25:00', '11:45:00', '12:05:

In [132]:
# Формирование столбца даты и времени начала доклада

data_events['datestart_event'] = data_events['date_new'] + ' ' + data_events['timestart_event_new']

In [133]:
# Формирование столбца даты и времени начала доклада

data_events['dateend_event'] = data_events['date_new'] + ' ' + data_events['timeend_event_new']

In [134]:
# Удаление ненужных столбцов

data_events.drop(['timestart_event', 'timeend_event', 'date_event'], axis= 1 , inplace= True)

In [135]:
# Контроль выполнения операций

data_events.head(1)

,duration_event,report_name,speaker_name,organization,city_event,potok_new,date_new,timestart_event_new,timeend_event_new,datestart_event,dateend_event
0,00:20:00,Тема 54,Самборский Илья Филиппович,no_organization,Санкт-Петербург,1,2022-07-01,10:00:00,10:20:00,2022-07-01 10:00:00,2022-07-01 10:20:00


In [136]:
# Перевод значения в формат времени

data_events['datestart_event'] = pd.to_datetime(data_events['datestart_event'],
                                               format='%Y-%m-%dT%H:%M:%S', utc=True)

In [137]:
# Перевод значения в формат времени

data_events['dateend_event'] = pd.to_datetime(data_events['dateend_event'],
                                               format='%Y-%m-%dT%H:%M:%S', utc=True)

Для последующих операций понадобится дата в формате Timestamp. Добавляю необходимые столбцы

In [138]:
# Функция перевода дат в формат Timestamp

def date_to_seconds(cell):
    date = cell.timestamp()
    return date

In [139]:
#  Перевод дат в формат Timestamp

data_events['timestamp_start_event'] = data_events['datestart_event'].apply(date_to_seconds)

In [140]:
#  Перевод дат в формат Timestamp

data_events['timestamp_end_event'] = data_events['dateend_event'].apply(date_to_seconds)

In [141]:
# Вычисление длительности доклада в секундах

data_events['duration_sec_event'] = data_events['timestamp_end_event'] -\
                                        data_events['timestamp_start_event']

In [142]:
data_events['timestamp_start_event'].unique()

array([1.6566696e+09, 1.6566708e+09, 1.6566711e+09, ..., 1.6830267e+09,
       1.6830414e+09, 1.6830426e+09])

In [143]:
# Переименование столбцов

data_events = data_events.rename(columns={'potok_new':'potok',
                                                   'date_new':'date'})

In [144]:
# Удаление ненужных столбцов

data_events = data_events[['date', 'potok', 'datestart_event', 'dateend_event', 'report_name',
                           'speaker_name', 'organization', 'city_event',
                    'timestamp_start_event', 'timestamp_end_event', 'duration_sec_event']]

In [145]:
# Проверка наличия дубликатов

data_events.duplicated().sum()

13

In [146]:
# Удаление дубликатов

data_events = data_events.drop_duplicates()

In [147]:
# Контроль выполнения операций

data_events.head(1)

,date,potok,datestart_event,dateend_event,report_name,speaker_name,organization,city_event,timestamp_start_event,timestamp_end_event,duration_sec_event
0,2022-07-01,1,2022-07-01 10:00:00+00:00,2022-07-01 10:20:00+00:00,Тема 54,Самборский Илья Филиппович,no_organization,Санкт-Петербург,1.656670e+09,1.656671e+09,1200.0


In [148]:
# Контроль выполнения операций

data_events.duplicated().sum()

0

## Обработка датасета с расписанием выступлений на небольших мероприятиях

#### Краткое описание операций при обработке датасета с расписаниями небольших мероприятий


Приведение формата записи дат в удобный для последующих преобразований формат и тип.

Переименование столбцов.

Добавление вспомогательных столбцов.


In [149]:
# Открытие и просмотр файла

data_small_events = pd.read_excel('Расписание_небольших_мероприятий 2.xlsx')

data_small_events.head()

,date,datestart_lectures,datefinish_lectures,duration_mins,lecture_name,speaker_name
0,2022-03-13,2022-03-13 10:00:00,2022-03-13 11:15:00,10:00 – 11:15,Тема 1,Ямаева Анна Даниловна
1,2022-03-14,2022-03-14 11:00:00,2022-03-14 11:30:00,11:00-11:20,Тема 2,Михальчишина Эльвира Ярославовна
2,2022-03-14,2022-03-14 11:30:00,2022-03-14 12:00:00,11:30-11:50,Тема 3,Ямаева Анна Даниловна
3,2022-03-14,2022-03-14 12:00:00,2022-03-14 12:30:00,12:00-12:20,Тема 4,Кобранов Валерий Артемович
4,2022-03-14,2022-03-14 12:30:00,2022-03-14 13:00:00,NaN,Тема 5,Рыбачев Владислав Артурович


In [150]:
# Просмотр информации о датасете

data_small_events.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 54 entries, 0 to 53
Data columns (total 6 columns):
 #   Column               Non-Null Count  Dtype         
---  ------               --------------  -----         
 0   date                 54 non-null     datetime64[ns]
 1   datestart_lectures   54 non-null     datetime64[ns]
 2   datefinish_lectures  54 non-null     datetime64[ns]
 3   duration_mins        53 non-null     object        
 4   lecture_name         54 non-null     object        
 5   speaker_name         54 non-null     object        
dtypes: datetime64[ns](3), object(3)
memory usage: 2.7+ KB


In [151]:
# Вывод дат начала докладов

data_small_events['datestart_lectures'].unique()

array(['2022-03-13T10:00:00.000000000', '2022-03-14T11:00:00.000000000',
       '2022-03-14T11:30:00.000000000', '2022-03-14T12:00:00.000000000',
       '2022-03-14T12:30:00.000000000', '2022-03-14T13:30:00.000000000',
       '2022-07-22T09:00:00.000000000', '2022-10-31T11:00:00.000000000',
       '2022-10-31T11:25:00.000000000', '2022-10-31T11:45:00.000000000',
       '2022-10-31T12:05:00.000000000', '2022-10-31T12:25:00.000000000',
       '2022-10-31T12:45:00.000000000', '2022-10-31T13:05:00.000000000',
       '2023-03-16T12:00:00.000000000', '2023-04-15T10:05:00.000000000',
       '2023-04-15T10:30:00.000000000', '2023-04-15T10:55:00.000000000',
       '2023-04-15T11:20:00.000000000', '2023-04-15T11:40:00.000000000',
       '2023-04-15T12:30:00.000000000', '2023-04-15T12:55:00.000000000',
       '2023-04-15T13:20:00.000000000', '2023-04-15T13:45:00.000000000',
       '2023-04-15T14:10:00.000000000', '2023-04-15T15:00:00.000000000',
       '2023-04-15T15:20:00.000000000', '2023-04-15

In [152]:
# Перевод значения в формат времени

data_small_events['datestart_event'] = pd.to_datetime(data_small_events['datestart_lectures'])

In [153]:
# Перевод значения в формат времени

data_small_events['dateend_event'] = pd.to_datetime(data_small_events['datefinish_lectures'])

In [154]:
# Перевод дат в формат Timestamp

data_small_events['timestamp_start_event'] = data_small_events['datestart_event']\
                                                    .apply(date_to_seconds)

In [155]:
# Перевод дат в формат Timestamp

data_small_events['timestamp_end_event'] = data_small_events['dateend_event']\
                                                .apply(date_to_seconds)

In [156]:
# Вычисление длительности доклада в секундах

data_small_events['duration_sec_event'] = data_small_events['timestamp_end_event'] -\
                                        data_small_events['timestamp_start_event']

In [157]:
# Добавление столбцов для последующей конкатенации

data_small_events['city_event'] = '-'
data_small_events['organization'] = '-'

In [158]:
# Добавление столбца для последующих преобразований

data_small_events['date_new'] = data_small_events['date'].astype(str)

In [159]:
# Удаление ненужного столбца

data_small_events.drop('date', axis= 1 , inplace= True )

In [160]:
# Переименование столбцов

data_small_events = data_small_events.rename(columns={'lecture_name':'report_name',
                                                   'date_new':'date'})

In [161]:
# Переименование столбцов

data_small_events = data_small_events.rename(columns={'lecture_name':'report_name'})

In [162]:
# Удаление ненужных столбцов

data_small_events = data_small_events[['date', 'datestart_event', 'dateend_event',
                                     'report_name', 'speaker_name', 'organization', 'city_event',
                    'timestamp_start_event', 'timestamp_end_event', 'duration_sec_event']]

In [163]:
# Проверка наличия дубликатов

data_small_events.duplicated().sum()

0

In [164]:
# Вывод текущего датасета

data_small_events.head(1)

,date,datestart_event,dateend_event,report_name,speaker_name,organization,city_event,timestamp_start_event,timestamp_end_event,duration_sec_event
0,2022-03-13,2022-03-13 10:00:00,2022-03-13 11:15:00,Тема 1,Ямаева Анна Даниловна,-,-,1.647166e+09,1.647170e+09,4500.0


## Объединение датасетов просмотров и расписаний. Обработка объединённого датасета

#### Краткое описание операций при объединении и обработке объединённого датасета (расписания выступлений + просмотры)


Объединение по дате и номеру потока трансляций датасетов с расписаниями data_events и датасета с просмотрами data_users с образованием датасета data_mix.

Фильтрация получившегося датасета по совпадениям времени просмотра зрителем и времени трансляции доклада.

Объединение только по дате датасетов с расписаниями небольших мероприятий data_small_events и датасета с просмотрами data_users с образованием датасета data_mix_small.

Фильтрация получившегося датасета по совпадениям времени просмотра зрителем и времени трансляции доклада.

Объединение конкатенацией отфильтрованных датасетов data_mix и data_mix_small с образованием датасета data.

Определение характерных типов мероприятий для каждого возможного значения длительности мероприятий.

Отсечение ненужных здесь строк с мероприятиями типа: приветствие, напутственная речь и т.п.

Унификация обозначения мероприятий типа дискуссия.

Повторная очистка датасета от строк с мероприятиями типа: антракт, заключительное слово и т.п.

Замена пропусков имени в строках с дискуссиями: отделение от датасета строк, в которых пропуск имени восполнять не нужно (сессии, круглые столы, демонстрация фильма и др.) - датасет data_no_discuss; выделение оставшихся строк в датасет data_rest (в котором нужно восполнить имя); замена в датасете data_rest значений no_speaker_name и no_city_event на NaN, а потом замена NaN на значение предыдущей строки (в предыдущей строке данные предшествующего дискуссии доклада); унификация записи групп докладчиков; замена в датасете data_rest значения "дискуссия" в столбце report_name на NaN, а затем на тему предшествующего доклада; проверка после всех преобразований; объединение конкатенацией датасетов data_no_discuss и data_rest.

Вычисление фактического времени просмотра докладов в течение каждой сессии.

Вычисление доли (процента) длительности просмотра докладов в течение каждой сессии.


In [165]:
# Обновление индексов датасетов перед объединением

data_events = data_events.reset_index(drop=True)
data_users = data_users.reset_index(drop=True)

In [166]:
# Просмотр датасета перед объединением

data_users.head(1)

,date,potok,user_date,user_id,o_id,barcode,device,country,region,user_city,profession,specialization,event,user_timestamp_start,user_timestamp_end,user_date_start,user_date_end
0,2022-03-13,1.0,13.03.2022,16573.0,626199.0,0809202204421616573,Десктоп,Россия,Амурская обл.,-,Врач,Онкология: хирургия (огш),Мероприятие 1,1647164550,1647164657,13.03.2022 9:42,13.03.2022 9:44


In [167]:
# Просмотр датасета перед объединением

data_events.head(1)

,date,potok,datestart_event,dateend_event,report_name,speaker_name,organization,city_event,timestamp_start_event,timestamp_end_event,duration_sec_event
0,2022-07-01,1,2022-07-01 10:00:00+00:00,2022-07-01 10:20:00+00:00,Тема 54,Самборский Илья Филиппович,no_organization,Санкт-Петербург,1.656670e+09,1.656671e+09,1200.0


In [168]:
# Объединение таблиц по дате и номеру потока доклада

data_mix = data_users.merge(data_events, on=['date', 'potok'], how='left')

In [169]:
# Контроль выполнения операции

data_mix.head(1)

,date,potok,user_date,user_id,o_id,barcode,device,country,region,user_city,profession,specialization,event,user_timestamp_start,user_timestamp_end,user_date_start,user_date_end,datestart_event,dateend_event,report_name,speaker_name,organization,city_event,timestamp_start_event,timestamp_end_event,duration_sec_event
0,2022-03-13,1.0,13.03.2022,16573.0,626199.0,0809202204421616573,Десктоп,Россия,Амурская обл.,-,Врач,Онкология: хирургия (огш),Мероприятие 1,1647164550,1647164657,13.03.2022 9:42,13.03.2022 9:44,NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [170]:
len(data_mix)

4245986

Объединив датасеты, я получил большой датасет со множеством строк, которые совпали по дате и номеру потока. Определить какой именно доклад смотрел зритель можно с помощью фильтрации.


Данные о поведении зрителя:


* user_date_start - дата и время начала просмотра доклада
* user_timestamp_start - дата и время начала просмотра доклада в формате Timestamp
* user_date_end - дата и время окончания просмотра доклада
* user_timestamp_end - дата и время окончания просмотра доклада в формате Timestamp


Данные о трансляциях докладов:


* datestart_event - дата и время начала доклада в трансляции
* timestamp_start_event - дата и время начала доклада в трансляции в формате Timestamp
* dateend_event - дата и время окончания просмотра доклада в трансляции
* timestamp_end_event - дата и время окончания просмотра доклада в трансляции в формате Timestamp


Возможны четыре случая соотношения времени просмотра трансляции зрителем и времени самой трансляции:


1. Зритель подключился к трансляции после её начала и отключился от трансляции до её окончания.


2.  Зритель подключился к трансляции до её начала и отключился от трансляции до её окончания.


3.  Зритель подключился к трансляции после её начала и отключился от трансляции после её окончания.


4.  Зритель подключился к трансляции до её начала и отключился от трансляции после её окончания.



In [171]:
# Фильтрация строк по совпадениям времени просмотра и времени трансляции

data_mix = data_mix.\
                query('(user_timestamp_start < timestamp_end_event < user_timestamp_end) |\
                    ((user_timestamp_start < timestamp_start_event < timestamp_end_event) &\
                    (user_timestamp_start < timestamp_end_event < user_timestamp_end)) |\
                    (user_timestamp_start < timestamp_start_event < user_timestamp_end)')

In [172]:
# Вывод отфильтрованных строк

data_mix.head(30)

,date,potok,user_date,user_id,o_id,barcode,device,country,region,user_city,profession,specialization,event,user_timestamp_start,user_timestamp_end,user_date_start,user_date_end,datestart_event,dateend_event,report_name,speaker_name,organization,city_event,timestamp_start_event,timestamp_end_event,duration_sec_event
4825,2022-07-05,1.0,05.07.2022,14825.0,530217.0,0306202210231114825,Десктоп,Россия,Чувашия,Чебоксары,Врач,Лучевая терапия (радиотерапия),Мероприятие 11,1657008218,1657010949,05.07.2022 8:03,05.07.2022 8:49,2022-07-05 08:40:00+00:00,2022-07-05 08:55:00+00:00,Тема 422,Волчанинова Жанна Вадимовна,no_organization,Санкт-Петербург,1.657010e+09,1.657011e+09,900.0
4871,2022-07-05,1.0,05.07.2022,14825.0,530217.0,0306202210231114825,Десктоп,Россия,Чувашия,Чебоксары,Врач,Лучевая терапия (радиотерапия),Мероприятие 11,1657011139,1657016570,05.07.2022 8:52,05.07.2022 10:22,2022-07-05 08:40:00+00:00,2022-07-05 08:55:00+00:00,Тема 422,Волчанинова Жанна Вадимовна,no_organization,Санкт-Петербург,1.657010e+09,1.657011e+09,900.0
4872,2022-07-05,1.0,05.07.2022,14825.0,530217.0,0306202210231114825,Десктоп,Россия,Чувашия,Чебоксары,Врач,Лучевая терапия (радиотерапия),Мероприятие 11,1657011139,1657016570,05.07.2022 8:52,05.07.2022 10:22,2022-07-05 08:55:00+00:00,2022-07-05 09:00:00+00:00,Дискуссия,no_speaker_name,no_organization,no_city_event,1.657011e+09,1.657012e+09,300.0
4873,2022-07-05,1.0,05.07.2022,14825.0,530217.0,0306202210231114825,Десктоп,Россия,Чувашия,Чебоксары,Врач,Лучевая терапия (радиотерапия),Мероприятие 11,1657011139,1657016570,05.07.2022 8:52,05.07.2022 10:22,2022-07-05 09:00:00+00:00,2022-07-05 09:15:00+00:00,Тема 423,Яглин Павел Маратович,no_organization,Санкт-Петербург,1.657012e+09,1.657012e+09,900.0
4874,2022-07-05,1.0,05.07.2022,14825.0,530217.0,0306202210231114825,Десктоп,Россия,Чувашия,Чебоксары,Врач,Лучевая терапия (радиотерапия),Мероприятие 11,1657011139,1657016570,05.07.2022 8:52,05.07.2022 10:22,2022-07-05 09:15:00+00:00,2022-07-05 09:20:00+00:00,Дискуссия,no_speaker_name,no_organization,no_city_event,1.657012e+09,1.657013e+09,300.0
4875,2022-07-05,1.0,05.07.2022,14825.0,530217.0,0306202210231114825,Десктоп,Россия,Чувашия,Чебоксары,Врач,Лучевая терапия (радиотерапия),Мероприятие 11,1657011139,1657016570,05.07.2022 8:52,05.07.2022 10:22,2022-07-05 09:20:00+00:00,2022-07-05 09:35:00+00:00,Тема 424,Лифанов Олег Артемович,no_organization,Санкт-Петербург,1.657013e+09,1.657014e+09,900.0
4876,2022-07-05,1.0,05.07.2022,14825.0,530217.0,0306202210231114825,Десктоп,Россия,Чувашия,Чебоксары,Врач,Лучевая терапия (радиотерапия),Мероприятие 11,1657011139,1657016570,05.07.2022 8:52,05.07.2022 10:22,2022-07-05 09:35:00+00:00,2022-07-05 09:40:00+00:00,Дискуссия,no_speaker_name,no_organization,no_city_event,1.657014e+09,1.657014e+09,300.0
4877,2022-07-05,1.0,05.07.2022,14825.0,530217.0,0306202210231114825,Десктоп,Россия,Чувашия,Чебоксары,Врач,Лучевая терапия (радиотерапия),Мероприятие 11,1657011139,1657016570,05.07.2022 8:52,05.07.2022 10:22,2022-07-05 09:40:00+00:00,2022-07-05 09:55:00+00:00,Тема 425,Безмощук Павел Степанович,no_organization,Санкт-Петербург,1.657014e+09,1.657015e+09,900.0
4878,2022-07-05,1.0,05.07.2022,14825.0,530217.0,0306202210231114825,Десктоп,Россия,Чувашия,Чебоксары,Врач,Лучевая терапия (радиотерапия),Мероприятие 11,1657011139,1657016570,05.07.2022 8:52,05.07.2022 10:22,2022-07-05 09:55:00+00:00,2022-07-05 10:00:00+00:00,Дискуссия,no_speaker_name,no_organization,no_city_event,1.657015e+09,1.657015e+09,300.0
4879,2022-07-05,1.0,05.07.2022,14825.0,530217.0,0306202210231114825,Десктоп,Россия,Чувашия,Чебоксары,Врач,Лучевая терапия (радиотерапия),Мероприятие 11,1657011139,1657016570,05.07.2022 8:52,05.07.2022 10:22,2022-07-05 10:00:00+00:00,2022-07-05 10:15:00+00:00,Тема 426,Баровская Вероника Филипповна,no_organization,Санкт-Петербург,1.657015e+09,1.657016e+09,900.0


Т.к. в датасете с расписаниями небольших мероприятий нет данных о потоке доклада, датасет с небольшими мероприятиями я объединяю отдельно, объединение происходит только по дате. Последующая фильтрация по времени просмотра аналогична.

In [173]:
# Объединение таблиц по дате доклада

data_mix_small = data_users.merge(data_small_events, on='date', how='left')

In [174]:
# Контроль выполнения операции

data_mix_small.head(1)

,date,potok,user_date,user_id,o_id,barcode,device,country,region,user_city,profession,specialization,event,user_timestamp_start,user_timestamp_end,user_date_start,user_date_end,datestart_event,dateend_event,report_name,speaker_name,organization,city_event,timestamp_start_event,timestamp_end_event,duration_sec_event
0,2022-03-13,1.0,13.03.2022,16573.0,626199.0,0809202204421616573,Десктоп,Россия,Амурская обл.,-,Врач,Онкология: хирургия (огш),Мероприятие 1,1647164550,1647164657,13.03.2022 9:42,13.03.2022 9:44,2022-03-13 10:00:00,2022-03-13 11:15:00,Тема 1,Ямаева Анна Даниловна,-,-,1.647166e+09,1.647170e+09,4500.0


In [175]:
# Фильтрация строк по совпадением времени просмотра и времени трансляции

data_mix_small = data_mix_small.\
                query('(user_timestamp_start < timestamp_end_event < user_timestamp_end) |\
                    ((user_timestamp_start < timestamp_start_event < timestamp_end_event) &\
                    (user_timestamp_start < timestamp_end_event < user_timestamp_end)) |\
                    (user_timestamp_start < timestamp_start_event < user_timestamp_end)')

In [176]:
# Вывод отфильтрованных строк

data_mix_small.head(30)

,date,potok,user_date,user_id,o_id,barcode,device,country,region,user_city,profession,specialization,event,user_timestamp_start,user_timestamp_end,user_date_start,user_date_end,datestart_event,dateend_event,report_name,speaker_name,organization,city_event,timestamp_start_event,timestamp_end_event,duration_sec_event
3,2022-03-13,1.0,13.03.2022,16573.0,626199.0,0809202204421616573,Десктоп,Россия,Амурская обл.,-,Врач,Онкология: хирургия (огш),Мероприятие 1,1647165297,1647166185,13.03.2022 9:54,13.03.2022 10:09,2022-03-13 10:00:00,2022-03-13 11:15:00,Тема 1,Ямаева Анна Даниловна,-,-,1.647166e+09,1.647170e+09,4500.0
7,2022-03-13,1.0,13.03.2022,1396.0,619071.0,250820222020141396,Десктоп,Россия,Удмуртия,Ижевск,Врач,-,Мероприятие 1,1647164631,1647168210,13.03.2022 9:43,13.03.2022 10:43,2022-03-13 10:00:00,2022-03-13 11:15:00,Тема 1,Ямаева Анна Даниловна,-,-,1.647166e+09,1.647170e+09,4500.0
10,2022-03-13,1.0,13.03.2022,16584.0,626274.0,0909202209360916584,Десктоп,Россия,Дагестан,Махачкала,Врач,Патологическая анатомия,Мероприятие 1,1647164857,1647168218,13.03.2022 9:47,13.03.2022 10:43,2022-03-13 10:00:00,2022-03-13 11:15:00,Тема 1,Ямаева Анна Даниловна,-,-,1.647166e+09,1.647170e+09,4500.0
11,2022-03-13,1.0,13.03.2022,4387.0,626920.0,140920222135274387,Десктоп,Казахстан,Павлодарская обл.,Павлодар,Врач,Хирург онколог,Мероприятие 1,1647164866,1647168218,13.03.2022 9:47,13.03.2022 10:43,2022-03-13 10:00:00,2022-03-13 11:15:00,Тема 1,Ямаева Анна Даниловна,-,-,1.647166e+09,1.647170e+09,4500.0
12,2022-03-13,1.0,13.03.2022,9023.0,619058.0,250820221834019023,Десктоп,Россия,Ленинградская обл.,Санкт-Петербург,Врач,Онкология: хирургия (абдоминальная),Мероприятие 1,1647165000,1647168213,13.03.2022 9:50,13.03.2022 10:43,2022-03-13 10:00:00,2022-03-13 11:15:00,Тема 1,Ямаева Анна Даниловна,-,-,1.647166e+09,1.647170e+09,4500.0
16,2022-03-13,1.0,13.03.2022,2610.0,619088.0,250820222147302610,Десктоп,Молдова,Молдова,Тирасполь,Врач,Химиотерапевт,Мероприятие 1,1647165096,1647168267,13.03.2022 9:51,13.03.2022 10:44,2022-03-13 10:00:00,2022-03-13 11:15:00,Тема 1,Ямаева Анна Даниловна,-,-,1.647166e+09,1.647170e+09,4500.0
17,2022-03-13,1.0,13.03.2022,2422.0,618609.0,230820221026142422,Десктоп,Россия,Владимирская обл.,Гусь Хрустальный,Представитель медицинской компании,Онколог-педиатр,Мероприятие 1,1647165172,1647167143,13.03.2022 9:52,13.03.2022 10:25,2022-03-13 10:00:00,2022-03-13 11:15:00,Тема 1,Ямаева Анна Даниловна,-,-,1.647166e+09,1.647170e+09,4500.0
19,2022-03-13,1.0,13.03.2022,10014.0,626878.0,1409202212552710014,Десктоп,Россия,Белгородская обл.,Белгород,Врач,Патологическая анатомия,Мероприятие 1,1647165177,1647168215,13.03.2022 9:52,13.03.2022 10:43,2022-03-13 10:00:00,2022-03-13 11:15:00,Тема 1,Ямаева Анна Даниловна,-,-,1.647166e+09,1.647170e+09,4500.0
20,2022-03-13,1.0,13.03.2022,10560.0,626389.0,1009202200024010560,Мобильное,Россия,Ленинградская обл.,Санкт-Петербург,Врач,Хирургия,Мероприятие 1,1647165202,1647168218,13.03.2022 9:53,13.03.2022 10:43,2022-03-13 10:00:00,2022-03-13 11:15:00,Тема 1,Ямаева Анна Даниловна,-,-,1.647166e+09,1.647170e+09,4500.0
21,2022-03-13,1.0,13.03.2022,14613.0,619295.0,2808202209151114613,Мобильное,Россия,Ленинградская обл.,Санкт-Петербург,Врач,Лучевая диагностика: рентгенология,Мероприятие 1,1647165262,1647168218,13.03.2022 9:54,13.03.2022 10:43,2022-03-13 10:00:00,2022-03-13 11:15:00,Тема 1,Ямаева Анна Даниловна,-,-,1.647166e+09,1.647170e+09,4500.0


После фильтрации датасеты можно объединить конкатенацией.


In [177]:
# Число строк в датасете data_mix

len(data_mix)

404585

In [178]:
# Число строк в датасете data_mix_small

len(data_mix_small)

2266

In [179]:
# Объединение датасетов

data = pd.concat([data_mix, data_mix_small], axis = 0)

In [180]:
# Контроль выполнения операции

data.head()

,date,potok,user_date,user_id,o_id,barcode,device,country,region,user_city,profession,specialization,event,user_timestamp_start,user_timestamp_end,user_date_start,user_date_end,datestart_event,dateend_event,report_name,speaker_name,organization,city_event,timestamp_start_event,timestamp_end_event,duration_sec_event
4825,2022-07-05,1.0,05.07.2022,14825.0,530217.0,0306202210231114825,Десктоп,Россия,Чувашия,Чебоксары,Врач,Лучевая терапия (радиотерапия),Мероприятие 11,1657008218,1657010949,05.07.2022 8:03,05.07.2022 8:49,2022-07-05 08:40:00+00:00,2022-07-05 08:55:00+00:00,Тема 422,Волчанинова Жанна Вадимовна,no_organization,Санкт-Петербург,1.657010e+09,1.657011e+09,900.0
4871,2022-07-05,1.0,05.07.2022,14825.0,530217.0,0306202210231114825,Десктоп,Россия,Чувашия,Чебоксары,Врач,Лучевая терапия (радиотерапия),Мероприятие 11,1657011139,1657016570,05.07.2022 8:52,05.07.2022 10:22,2022-07-05 08:40:00+00:00,2022-07-05 08:55:00+00:00,Тема 422,Волчанинова Жанна Вадимовна,no_organization,Санкт-Петербург,1.657010e+09,1.657011e+09,900.0
4872,2022-07-05,1.0,05.07.2022,14825.0,530217.0,0306202210231114825,Десктоп,Россия,Чувашия,Чебоксары,Врач,Лучевая терапия (радиотерапия),Мероприятие 11,1657011139,1657016570,05.07.2022 8:52,05.07.2022 10:22,2022-07-05 08:55:00+00:00,2022-07-05 09:00:00+00:00,Дискуссия,no_speaker_name,no_organization,no_city_event,1.657011e+09,1.657012e+09,300.0
4873,2022-07-05,1.0,05.07.2022,14825.0,530217.0,0306202210231114825,Десктоп,Россия,Чувашия,Чебоксары,Врач,Лучевая терапия (радиотерапия),Мероприятие 11,1657011139,1657016570,05.07.2022 8:52,05.07.2022 10:22,2022-07-05 09:00:00+00:00,2022-07-05 09:15:00+00:00,Тема 423,Яглин Павел Маратович,no_organization,Санкт-Петербург,1.657012e+09,1.657012e+09,900.0
4874,2022-07-05,1.0,05.07.2022,14825.0,530217.0,0306202210231114825,Десктоп,Россия,Чувашия,Чебоксары,Врач,Лучевая терапия (радиотерапия),Мероприятие 11,1657011139,1657016570,05.07.2022 8:52,05.07.2022 10:22,2022-07-05 09:15:00+00:00,2022-07-05 09:20:00+00:00,Дискуссия,no_speaker_name,no_organization,no_city_event,1.657012e+09,1.657013e+09,300.0


In [181]:
# Число строк в датасете data

len(data)

406851

Ранее отсутствующие данные об имени докладчика, организации докладчика и городе докладчика были заменены значениеми "no_speaker_name", "no_organization", "no_city_event" . После обработки, преобразований и объединении данных в один датасет стал возможным просмотр отдельных сессий (просмотров) с данными о теме трансляции. При просмотре строк датасета видно, что после доклада в большинстве случаев следует дискуссия (обозначенная как тема доклада), длительностью 5 минут, что даёт основания предположить, что дискуссия посвящена теме предшествующего доклада. В текущем исследовании имеет смысл присвоить такой дискуссии имя, организацию и город докладчика предшествующего, предваряющего доклада. Необходимо проверить также мероприятия, следующие за докладом длительностью отличной от пяти минут, проверить способы обозначения таких дискуссий.

Для более прицельной замены пропусков в данных об имени, организации и городе докладчика, проверяю строки по теме доклада и по длительности доклада.

In [182]:
# Просмотр уникальных значений длительности доклада/мероприятия

data['duration_sec_event'].unique()

array([  900.,   300.,  1200.,  1800.,  5400.,  1500.,   600.,  3600.,
       32400.,  2400.,  7200., 28800.,   450., 12600., 10800.,  4200.,
        6000.,   180.,   120.,  6300.,  5100.,  4500., 14400., 18000.])

In [183]:
# Вывожу последовательно строки с каждым из возможных значений длительности мероприятия
# и присущие им типы (темы)

# data.query('duration_sec_event == 120')['report_name'].unique()
# data.query('duration_sec_event == 120')
# data.query('duration_sec_event == 180')['report_name'].unique()
# data.query('duration_sec_event == 180')
# data.query('duration_sec_event == 300')['report_name'].unique()
# data.query('duration_sec_event == 300')
# data.query('duration_sec_event == 450')['report_name'].unique()
# data.query('duration_sec_event == 450')
# data.query('duration_sec_event == 600')['report_name'].unique()
# data.query('duration_sec_event == 600')
# data.query('duration_sec_event == 900')['report_name'].unique()
# data.query('duration_sec_event == 900')
# data.query('duration_sec_event == 1200')['report_name'].unique()
# data.query('duration_sec_event == 1200')
# data.query('duration_sec_event == 1500')['report_name'].unique()
# data.query('duration_sec_event == 1500')
# data.query('duration_sec_event == 1800')['report_name'].unique()
# data.query('duration_sec_event == 1800')
# data.query('duration_sec_event == 2400')['report_name'].unique()
# data.query('duration_sec_event == 2400')
# data.query('duration_sec_event == 3600')['report_name'].unique()
# data.query('duration_sec_event == 3600')
# data.query('duration_sec_event == 4200')['report_name'].unique()
# data.query('duration_sec_event == 4200')
# data.query('duration_sec_event == 4500')['report_name'].unique()
# data.query('duration_sec_event == 4500')
# data.query('duration_sec_event == 5100')['report_name'].unique()
# data.query('duration_sec_event == 5100')
# data.query('duration_sec_event == 5400')['report_name'].unique()
# data.query('duration_sec_event == 5400')
# data.query('duration_sec_event == 6000')['report_name'].unique()
# data.query('duration_sec_event == 6000')
# data.query('duration_sec_event == 6300')['report_name'].unique()
# data.query('duration_sec_event == 6300')
# data.query('duration_sec_event == 7200')['report_name'].unique()
# data.query('duration_sec_event == 7200')
# data.query('duration_sec_event == 10800')['report_name'].unique()
# data.query('duration_sec_event == 10800')
# data.query('duration_sec_event == 12600')['report_name'].unique()
# data.query('duration_sec_event == 12600')
# data.query('duration_sec_event == 14400')['report_name'].unique()
# data.query('duration_sec_event == 14400')
# data.query('duration_sec_event == 18000')['report_name'].unique()
# data.query('duration_sec_event == 18000')
# data.query('duration_sec_event == 28800')['report_name'].unique()
# data.query('duration_sec_event == 28800')
data.query('duration_sec_event == 32400')['report_name'].unique()
# data.query('duration_sec_event == 32400')



array(['Тема 619'], dtype=object)

Выяснено, что мероприятия длительностью:


* 2 минуты (120 секунд) - это "Открытие" и "Приветственное слово" - не интересны для этого исследования;


* 3 минуты (180 секунд) - это "Обсуждение" - судя по всему, аналогично Дискуссии после доклада. Имеет смысл присвоить данные предшествующего доклада такому мероприятию;


* 5 минут (300 секунд) - мероприятия имеют широкий разброс типов: вступительные и заключительные речи, награждения, дискуссии по теме предшествующего доклада, непосредственно доклады. В дальнейшем мероприятия, не являющиеся непосредственно докладами и дискуссиями по теме доклада, будут исключены из датасета;


* 7.5 минут (450 секунд) - это доклады, представляющие интерес для этого исследования;


* 10 минут (600 секунд) - это за редким исключением доклады, представляющие интерес для этого исследования;


* 15 минут (900 секунд) - это за редким исключением доклады, представляющие интерес для этого исследования, основная часть докладов;


* 20 минут (1200 секунд) - это за редким исключением доклады, представляющие интерес для этого исследования;


* 25 минут (1500 секунд) - это доклады, представляющие интерес для этого исследования;


* 30 минут (1800 секунд) - это доклады, представляющие интерес для этого исследования;


* 40 минут (2400 секунд) - это доклады, представляющие интерес для этого исследования;


* 1 час (3600 секунд) - это, за одним исключением, доклады, круглые столы, видеопрезентации, интервью представляющие интерес для этого исследования;


* 1 час 10 минут (4200 секунд) - это мероприятия типа "Круглый стол" - представляющие интерес для этого исследования;


* 1 час 15 минут (4500 секунд) - это трансляция фильма - представляет интерес для этого исследования;


* 1 час 25 минут (5100 секунд) - это доклады, круглые столы, обсуждения - представляющие интерес для этого исследования;


* 1 час 30 минут (5400 секунд) - это доклады - представляющие интерес для этого исследования;


* 1 час 40 минут (6000 секунд) - это мероприятия типа "Круглый стол" - представляющие интерес для этого исследования;


* 1 час 45 минут (6300 секунд) - это доклад - представляет интерес для этого исследования;


* 2 часа (7200 секунд) - это доклады, обзоры, баттлы - представляющие интерес для этого исследования;


* 3 часа (10800 секунд) - это мероприятие типа "Круглый стол" - представляет интерес для этого исследования;


* 3 часа 30 минут (12600 секунд) - это трансляция операции - представляет интерес для этого исследования;


* 4 часа (14400 секунд) - это трансляция операции - представляет интерес для этого исследования;


* 5 часов (18000 секунд) - это трансляция операции - представляет интерес для этого исследования;


* 8 часов (28800 секунд) - это трансляции операций - представляющие интерес для этого исследования;


* 9 часов (32400 секунд) - это трансляция операции - представляет интерес для этого исследования.


Мероприятия, не представляющие интереса для этого исследования следует исключить из датасета. Также, необходимо присвоить недостающие данные дискуссиям по темам предшествующих докладов, привести обозначения таких дискуссий к единой форме записи.


In [184]:
# Подсчёт количества мероприятий типа Дискуссия до преобразований

len(data.query('report_name == "Дискуссия"'))

160316

In [185]:
# Составление словаря для замены

dict_5 = {'Ответы на вопросы':'Дискуссия', 'Дискуссия ':'Дискуссия',
          'Обсуждение, ответы на вопросы':'Дискуссия', 'Заключение. Выводы':'Дискуссия',
          'Дискуссия   ':'Дискуссия', 'Обсуждение':'Дискуссия',
          'Сессия вопросов и ответов ':'Дискуссия',
          'Дискуссия, заключительное слово':'Дискуссия',
          'Обсуждение, ответы на вопросы,подведение итогов ':'Дискуссия',
          'Круглый стол:':'Круглый стол'}


In [186]:
# Приведение к единой форме записи

data = data.replace({"report_name":dict_5})

In [187]:
# Список мероприятий, не представляющих интереса для этого исследования

wrong_reports = ['Заключительное слово', 'Вступительное слово',
                 'Общая дискуссия участников сессии', 'Приветственное слово',
                 'Напутствие руководителя Филатовой Ларисы Валентиновны, вручение диплома',
                 'Напутствие руководителя Берлева Игоря Викторовича, вручение диплома',
                 'Напутствие руководителя Малек Анастасия Валерьевна, вручение диплома',
                 'Напутствие руководителя Куликова Евгения Петровича, вручение диплома',
                 'Напутствие руководителя Проценко Светланы Анатольевны, вручение диплома',
                 'Напутствие руководителя Кайдаровой Диляры Радиковны, вручение диплома',
                 'Напутствие руководителя Колядина Ирина Владимировна, вручение диплома',
                 'Вступительное слово ведущих и лечащего врача',
                 'Диалог с аудиторией, вопросы и ответы', 'Вступление', 'Приветствие участников',
                 'Вопросы для обсуждения:', 'Вступительные слова ', 'Подведение итогов',
                 'Приветственное слово организатора SPOT Radiology ', 'Напутствие наставника',
                 'Приветсвенное слово', 'Приветственное слово председателей',
                 'Прикоснуться к «Совершенству»: набор открыт', 'Обсуждение, подведение итогов',
                 'Общая дискуссия участников сессии', 'Представление проектов',
                 'Приветственное слово председателей', 'Видеоприветствие',
                 'Обсуждение, Подведение итогов', 'Вступительное слово ',
                 'Заключительное слово ', 'Общая дискусия участников сессии',
                 'Вступительное слово, открытие сессии', 'Перерыв']


In [188]:
# Исключение ненужных строк

data = data.query('report_name != @wrong_reports')

In [189]:
# Контроль выполнения операций

len(data)

402627

In [190]:
# Контроль выполнения операций

len(data.query('report_name == "Дискуссия"'))

160426

Далее необходимо проверить, во всех ли строках с Дискуссиями отсутствует имя докладчика и во всех ли таких строках временное обозначение: "no_speaker_name".


In [191]:
# Вывод обозначений имени докладчика в дискуссиях

data.query('report_name == "Дискуссия"')['speaker_name'].value_counts().head()

no_speaker_name                 152155
Васючкова Оксана Руслановна       4070
Кормилицин Павел Ярославович      2725
Айриева Юлия Леонидовна            156
Вереитинов Алексей Ефимович         66
Name: speaker_name, dtype: int64

In [192]:
# Просмотр количества строк  без имени докладчика

len(data.query('speaker_name == "no_speaker_name"'))

157566

In [193]:
# Поиск отсутствия имени докладчика вне мероприятий Дискуссия

data.query('report_name != "Дискуссия" & speaker_name == "no_speaker_name"')\
            ['report_name'].unique()

array(['Тема 188', 'Тема 479', 'Тема 508', 'Тема 208', 'Тема 512',
       'Тема 127', 'Тема 601', 'Тема 603', 'Тема 606', 'Тема 608',
       'Тема 610', 'Тема 612', 'Тема 614', 'Тема 655', 'Тема 656',
       'Тема 658', 'Тема 674', 'Тема 664', 'Тема 60', 'Тема 112',
       'Тема 169', 'Тема 137', 'Тема 140', 'Тема 209', 'Тема 211',
       'Тема 214', 'Тема 223', 'Тема 231', 'Тема 227', 'Тема 312',
       'Тема 391', 'Тема 415', 'Тема 394', 'Тема 381', 'Тема 732',
       'Тема 518', 'Тема 1000', 'Тема 996', 'Тема 1016', 'Тема 1018',
       'Тема 1141', 'Тема 1161', 'Тема 1162', 'Тема 1421', 'Тема 1366',
       'Тема 1379', 'Тема 1481', 'Тема 1402', 'Тема 1315', 'Тема 1301',
       'Тема 1279'], dtype=object)

Обнаружилось, что в датасете всё ещё присутствуют ненужные для исследования строки (Антракт, церемония награжденя и др.). Также, замечены иные обозначения дискуссий. Повторяю шаги по удалению и унификации записей с Дискуссиями.

In [194]:
# Приведение к единой форме записи

data['report_name'] = data['report_name'].replace('Вопросы и ответы', 'Дискуссия')
data['report_name'] = data['report_name'].replace('Дискуссия экспертов ', 'Дискуссия')
data['report_name'] = data['report_name'].replace(' «Consensus omnium», дискуссия и закрытие',
                                                  'Дискуссия')

data['report_name'] = data['report_name'].replace('Дискуссия, подведение итогов', 'Дискуссия')

In [195]:
# Контроль выполнения операций

len(data.query('report_name == "Дискуссия"'))

160426

In [196]:
# Список мероприятий, не представляющих интереса для этого исследования

wrong_reports_2 = ['Антракт',
                    'Спасибо за помощь: церемония награждения лучших пациент-ориентированных проектов/пациентских организаций',
       'Подведение итогов сессии', 'Приветствие участников мероприятия в честь открытия',
                   'Заключительное слово председателей и дискуссия ', 'Общая дискуссия ',
                   'Открытие, приветственное слово', 'Общая дискуссия  участников сессии',
                   'Закличительное слово по сессиям ']

In [197]:
# Исключение ненужных строк

data = data.query('report_name != @wrong_reports_2')

In [198]:
# Контроль выполнения операций

len(data)

402627

In [199]:
# Вывод обозначений организации докладчика в дискусиях

data.query('report_name == "Дискуссия"')['organization'].value_counts().head()

no_organization    160426
Name: organization, dtype: int64

In [200]:
# Вывод обозначений города докладчика в дискусиях

data.query('report_name == "Дискуссия"')['city_event'].value_counts().head()

no_city_event        159106
Санкт-Петербург         726
Москва                  396
Беларусь                132
Алматы, Казахстан        66
Name: city_event, dtype: int64

Как оказывается, организация и город докладчика в Дискуссиях отсутствует чаще, чем его имя. 


Также, выяснено, что имя докладчика отстутствует не только в Дискуссиях, но и в некоторых докладах, симпозиумах, демонстрациях фильмов.


Для замены пропусков имени и других данных о докладчике только в строках с Дискуссиями разделяю датасет на датасет со строками с отсутствием имени no_speaker_name и мероприятиями не Дискуссия - data_no_discuss, и датасет с остальными строками - data_rest.

In [201]:
# Выделение в отдельный датасет строк без Дискуссий и без имени докладчика

data_no_discuss = data.query('report_name != "Дискуссия" & speaker_name == "no_speaker_name"')

In [202]:
# Контроль выполнения операции

data_no_discuss.head(1)

,date,potok,user_date,user_id,o_id,barcode,device,country,region,user_city,profession,specialization,event,user_timestamp_start,user_timestamp_end,user_date_start,user_date_end,datestart_event,dateend_event,report_name,speaker_name,organization,city_event,timestamp_start_event,timestamp_end_event,duration_sec_event
4988,2022-07-05,1.0,05.07.2022,14825.0,530217.0,0306202210231114825,Десктоп,Россия,Чувашия,Чебоксары,Врач,Лучевая терапия (радиотерапия),Мероприятие 11,1657023529,1657036988,05.07.2022 12:18,05.07.2022 16:03,2022-07-05 13:25:00+00:00,2022-07-05 13:30:00+00:00,Тема 188,no_speaker_name,no_organization,no_city_event,1.657028e+09,1.657028e+09,300.0


In [203]:
# Контроль выполнения операции

data_no_discuss['report_name'].unique()

array(['Тема 188', 'Тема 479', 'Тема 508', 'Тема 208', 'Тема 512',
       'Тема 127', 'Тема 601', 'Тема 603', 'Тема 606', 'Тема 608',
       'Тема 610', 'Тема 612', 'Тема 614', 'Тема 655', 'Тема 656',
       'Тема 658', 'Тема 674', 'Тема 664', 'Тема 60', 'Тема 112',
       'Тема 169', 'Тема 137', 'Тема 140', 'Тема 209', 'Тема 211',
       'Тема 214', 'Тема 223', 'Тема 231', 'Тема 227', 'Тема 312',
       'Тема 391', 'Тема 415', 'Тема 394', 'Тема 381', 'Тема 732',
       'Тема 518', 'Тема 1000', 'Тема 996', 'Тема 1016', 'Тема 1018',
       'Тема 1141', 'Тема 1161', 'Тема 1162', 'Тема 1421', 'Тема 1366',
       'Тема 1379', 'Тема 1481', 'Тема 1402', 'Тема 1315', 'Тема 1301',
       'Тема 1279'], dtype=object)

In [204]:
# Список мероприятий не Дискуссий, в которых нет имени докладчика

reports = ['Список мероприятий не Дискуссий, в которых нет имени докладчика']

In [205]:
# Выделение в отдельный датасет остальных строк датасета

data_rest = data.query('report_name != @reports')

In [206]:
# Контроль выполнения операций
# В датасете не должно быть мероприятий Круглый стол

len(data_rest.query('report_name == "Круглый стол"'))

0

In [207]:
# Замена пропусков для дальнейших преобразований

data_rest['speaker_name'] = data_rest['speaker_name'].replace('no_speaker_name', np.nan)
data_rest['city_event'] = data_rest['city_event'].replace('no_city_event', np.nan)

In [208]:
# Замена пропусков на данные предыдущей строки

data_rest = data_rest.fillna(method='ffill')

In [209]:
# Контроль выполнения операций

data_rest.query('report_name == "Дискуссия"')['speaker_name'].value_counts().head(10)

Васючкова Оксана Руслановна         4130
Полянина Наталья Ринатовна          3508
Милешкина Жанна Артемовна           2951
Кормилицин Павел Ярославович        2725
Дзевалтовская Дарья Владимировна    2538
Коляскина Виктория Алексеевна       2444
Элькина Римма Геннадьевна           2436
Самборский Илья Филиппович          2405
Тезиков Радик Ярославович           2282
Бакшаев Радик Алексеевич            2271
Name: speaker_name, dtype: int64

Запись "Все участники симпозиума" тождественна записи "Все спикеры". Для унификации объеденю эти записи в одну: "Все участники мероприятия".


In [210]:
# Приведение к единой форме записи

data_rest['speaker_name'] = data_rest['speaker_name'].replace('Все участники симпозиума',
                                                              'Все участники мероприятия')
data_rest['speaker_name'] = data_rest['speaker_name'].replace('Все спикеры',
                                                              'Все участники мероприятия')

In [211]:
# Контроль выполнения операций

data_rest.query('report_name == "Дискуссия"')['speaker_name'].value_counts().head(2)

Васючкова Оксана Руслановна    4130
Полянина Наталья Ринатовна     3508
Name: speaker_name, dtype: int64

In [212]:
# Контроль выполнения операций

data_rest.query('report_name == "Дискуссия"')['organization'].value_counts().head()

no_organization    160426
Name: organization, dtype: int64

In [213]:
# Контроль выполнения операций

data_rest.query('organization == "no_organization"').head(2)

,date,potok,user_date,user_id,o_id,barcode,device,country,region,user_city,profession,specialization,event,user_timestamp_start,user_timestamp_end,user_date_start,user_date_end,datestart_event,dateend_event,report_name,speaker_name,organization,city_event,timestamp_start_event,timestamp_end_event,duration_sec_event
4825,2022-07-05,1.0,05.07.2022,14825.0,530217.0,0306202210231114825,Десктоп,Россия,Чувашия,Чебоксары,Врач,Лучевая терапия (радиотерапия),Мероприятие 11,1657008218,1657010949,05.07.2022 8:03,05.07.2022 8:49,2022-07-05 08:40:00+00:00,2022-07-05 08:55:00+00:00,Тема 422,Волчанинова Жанна Вадимовна,no_organization,Санкт-Петербург,1.657010e+09,1.657011e+09,900.0
4871,2022-07-05,1.0,05.07.2022,14825.0,530217.0,0306202210231114825,Десктоп,Россия,Чувашия,Чебоксары,Врач,Лучевая терапия (радиотерапия),Мероприятие 11,1657011139,1657016570,05.07.2022 8:52,05.07.2022 10:22,2022-07-05 08:40:00+00:00,2022-07-05 08:55:00+00:00,Тема 422,Волчанинова Жанна Вадимовна,no_organization,Санкт-Петербург,1.657010e+09,1.657011e+09,900.0


Для уверенности обоснованности предположения того, что следующая за докладом дискуссия посвящена теме доклада я вывожу случайные строки датасета с нетипичной длительностью дискуссии, и сравниваю строки с расписанием мероприятий.


In [214]:
# Вывод случайных строк для сравнения с расписанием мероприятий

data_rest.query('report_name == "Дискуссия" & duration_sec_event >= 1500').sample()

,date,potok,user_date,user_id,o_id,barcode,device,country,region,user_city,profession,specialization,event,user_timestamp_start,user_timestamp_end,user_date_start,user_date_end,datestart_event,dateend_event,report_name,speaker_name,organization,city_event,timestamp_start_event,timestamp_end_event,duration_sec_event
1187101,2022-07-02,3.0,02.07.2022,14439.0,528010.0,2704202213471314432.0,Десктоп,Россия,Московская обл.,Москва,Исследователь,Онкология: лекарственное лечение,Мероприятие 11,1656772587,1656776815,02.07.2022 14:36,02.07.2022 15:46,2022-07-02 15:00:00+00:00,2022-07-02 17:00:00+00:00,Дискуссия,Чечко Егор Антонович,no_organization,Беларусь,1.656774e+09,1.656781e+09,7200.0


Убедившись, что моё предположение о теме дискуссий было верным, заменяю report_name этих дискуссий на название предшествующего доклада.


In [215]:
# Подсчёт количества строк с дискуссиями

len(data_rest.query('report_name == "Дискуссия"'))

160426

In [216]:
# Замена значений на NaN

data_rest['report_name'] = data_rest['report_name'].replace('Дискуссия', np.nan)

In [217]:
# Замена пропусков на данные предыдущей строки

data_rest = data_rest.fillna(method='ffill')

In [218]:
# Контроль выполнения операции

len(data_rest.query('report_name == "Дискуссия"'))

0

После всех преобразований строк с дискуссиями нужно объединить датасеты data_no_discuss и data_rest.


In [219]:
# Подсчёт количества строк в датасете

len(data_rest)

402627

In [220]:
# Подсчёт количества строк в датасете

len(data_no_discuss)

5411

In [221]:
# Объединение датасетов

data = pd.concat([data_rest, data_no_discuss], axis = 0)

In [222]:
# Контроль выполнения операции

len(data)

408038

In [223]:
# Контроль выполнения операции

data.head(1)

,date,potok,user_date,user_id,o_id,barcode,device,country,region,user_city,profession,specialization,event,user_timestamp_start,user_timestamp_end,user_date_start,user_date_end,datestart_event,dateend_event,report_name,speaker_name,organization,city_event,timestamp_start_event,timestamp_end_event,duration_sec_event
4825,2022-07-05,1.0,05.07.2022,14825.0,530217.0,0306202210231114825,Десктоп,Россия,Чувашия,Чебоксары,Врач,Лучевая терапия (радиотерапия),Мероприятие 11,1657008218,1657010949,05.07.2022 8:03,05.07.2022 8:49,2022-07-05 08:40:00+00:00,2022-07-05 08:55:00+00:00,Тема 422,Волчанинова Жанна Вадимовна,no_organization,Санкт-Петербург,1.657010e+09,1.657011e+09,900.0


Для дальнейшего анализа необходимо вычислить фактическое время просмотра докладов в течении каждой сессии. 


Выше обозначалось, что возможны четыре случая соотношения времени просмотра трансляции зрителем и времени самой трансляции. Из этого следуют четыре варианта вычисления фактического времени просмотра:

1. Зритель подключился к трансляции после её начала и отключился от трансляции до её окончания. В таком случае, фактическое время просмотра лежит между подключением и отключением зрителя к трансляции. Т.е.:
Фактическое время просмотра = [Временная точка отключения от трансляции] - [Временная точка подключения к трансляции]


2.  Зритель подключился к трансляции до её начала и отключился от трансляции до её окончания. В таком случае, фактическое время просмотра лежит между началом трансляции и отключением зрителя от трансляции. Т.е.:
Фактическое время просмотра = [Временная точка отключения от трансляции] - [Временная точка начала трансляции]


3.  Зритель подключился к трансляции после её начала и отключился от трансляции после её окончания. В таком случае, фактическое время просмотра лежит между подключением зрителя к трансляции и окончанием трансляции. Т.е.:
Фактическое время просмотра = [Временная точка окончания трансляции] - [Временная точка подключения к трансляции]


4.  Зритель подключился к трансляции до её начала и отключился от трансляции после её окончания. В таком случае, фактическое время просмотра лежит между началом и окончанием трансляции. Т.е.:
Фактическое время просмотра = [Временная точка окончания трансляции] - [Временная точка начала трансляции]


* Временначя точка подключения к трансляции - user_timestamp_start

* Временначя точка отключения от трансляции - user_timestamp_end

* Временначя точка начала трансляции - timestamp_start_event

* Временначя точка окончания трансляции - timestamp_end_event


In [224]:
# Функция вычисления фактического времени просмотра трансляции зрителем в течении одной сессии

def duration_fact_func(cell):
    if cell[13] >= cell[23] and cell[14] <= cell[24]:
        result = cell[14] - cell[13]
    elif cell[13] <= cell[23] and cell[14] <= cell[24]:
        result = cell[14] - cell[23]
    elif cell[13] >= cell[23] and cell[14] >= cell[24]:
        result = cell[24] - cell[13]
    elif cell[13] <= cell[23] and cell[14] >= cell[24]:
        result = cell[24] - cell[23]
    else:
        result = np.nan
    return result

In [225]:
# Вычисление фактического времени просмотра трансляции зрителем в течении одной сессии

data['duration_fact'] = data.apply(duration_fact_func, axis = 1)

In [226]:
# Формирование столбца с долей просмотренного доклада в процентах

data['duration_part'] = round(data['duration_fact']\
                                       / data['duration_sec_event'], 2) * 100

In [227]:
# Контроль выполнения операций

data.head(3)

,date,potok,user_date,user_id,o_id,barcode,device,country,region,user_city,profession,specialization,event,user_timestamp_start,user_timestamp_end,user_date_start,user_date_end,datestart_event,dateend_event,report_name,speaker_name,organization,city_event,timestamp_start_event,timestamp_end_event,duration_sec_event,duration_fact,duration_part
4825,2022-07-05,1.0,05.07.2022,14825.0,530217.0,0306202210231114825,Десктоп,Россия,Чувашия,Чебоксары,Врач,Лучевая терапия (радиотерапия),Мероприятие 11,1657008218,1657010949,05.07.2022 8:03,05.07.2022 8:49,2022-07-05 08:40:00+00:00,2022-07-05 08:55:00+00:00,Тема 422,Волчанинова Жанна Вадимовна,no_organization,Санкт-Петербург,1.657010e+09,1.657011e+09,900.0,549.0,61.0
4871,2022-07-05,1.0,05.07.2022,14825.0,530217.0,0306202210231114825,Десктоп,Россия,Чувашия,Чебоксары,Врач,Лучевая терапия (радиотерапия),Мероприятие 11,1657011139,1657016570,05.07.2022 8:52,05.07.2022 10:22,2022-07-05 08:40:00+00:00,2022-07-05 08:55:00+00:00,Тема 422,Волчанинова Жанна Вадимовна,no_organization,Санкт-Петербург,1.657010e+09,1.657011e+09,900.0,161.0,18.0
4872,2022-07-05,1.0,05.07.2022,14825.0,530217.0,0306202210231114825,Десктоп,Россия,Чувашия,Чебоксары,Врач,Лучевая терапия (радиотерапия),Мероприятие 11,1657011139,1657016570,05.07.2022 8:52,05.07.2022 10:22,2022-07-05 08:55:00+00:00,2022-07-05 09:00:00+00:00,Тема 422,Волчанинова Жанна Вадимовна,no_organization,Санкт-Петербург,1.657011e+09,1.657012e+09,300.0,300.0,100.0


Полные данные этого проекта содержат датасет-рубрикатор: доклады мероприятий соответствуют направлениям, разбиты на сессии и классифицированы. На этом этапе необходимо присоединить датасет-рубрикатор. Перед присоединением датасет-рубрикатор нужно обработать.


Вывод первых трёх строк текущего датасета показывает следующую картину: один и тот же зритель подключился к трансляции в 8:03 - задолго до начала показа докладов. В 8:40 началась трансляция доклада. По какой-то причине зритель отключился от трансляции в 8:49, успев просмотреть 61% длительности 15-ти минутного доклада. В 8:52 этот же зритель вновь подключился к трансляции этого же доклада и просмотрел его до конца, успев просмотреть ещё 18% длительности этого 15-ти минутного доклада. После доклада последовала дискуссия (как я предполагаю, по теме доклада), начало дискуссии в 8:55, окончание дискусии в 9:00, для всех подобных дискуссий в таблице в колонке Организации докладчика значение: no_organization. Описываемый здесь зритель посмотрел эту дискуссию от начала до конца, т.е., 100% её длительности.


Т.о., очевидно, что при подсчёте длительности просмотра докладов нужно учитывать все подключения к конкретной трансляции каждым конкретным зрителем. Операция подсчёта всех частей просмотра с помощью группировки будет проведена после объединения текущего датасета и датасета-рубрикатора.

## Обработка датасета словаря рубрик форума

#### Краткое описание операций при  обработке  датасета словаря рубрик форума


Приведение к единой для этого исследования форме записи номера потока трансляций.

Восполнение пропущенных значений времени окончания рубрик.

Замена типа данных дат/времени.

Формирование вспомогательных столбцов с датой/временем с удобным для дальнейших преобразований типом.


In [228]:
# Загрузка датасета

data_rubric = pd.read_excel('Словарь рубрик 11-12.xlsx')

data_rubric.head()

,Unnamed: 0,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Год,Столбец2,Поток,Начало,Конец,Столбец1,Код классификатора,Столбец12
2,2022,2022-07-01 00:00:00,1 поток,10:00:00,12:00:00,Сессия 1,Основное,Направление 1
3,2022,2022-07-01 00:00:00,1 поток,10:00:00,12:00:00,Сессия 1,Нозологии,Направление 2
4,2022,2022-07-01 00:00:00,1 поток,13:00:00,15:00:00,Сессия 2,Основное,Направление 3


In [229]:
# Удаление лишнего столбца, переименование столбцов, удаление первых двух строк

data_rubric.drop('Unnamed: 0', axis= 1 , inplace= True )

data_rubric = data_rubric.rename(columns={
                           'Unnamed: 1':'date_rubric',
                           'Unnamed: 2':'potok_rubric',
                           'Unnamed: 3':'time_start',
                           'Unnamed: 4':'time_end',
                           'Unnamed: 5':'session',
                           'Unnamed: 6':'class',
                           'Unnamed: 7':'direction'})

data_rubric = data_rubric.drop(index=[0, 1])


In [230]:
# Контроль выполнения операций

data_rubric.head(2)

,date_rubric,potok_rubric,time_start,time_end,session,class,direction
2,2022-07-01 00:00:00,1 поток,10:00:00,12:00:00,Сессия 1,Основное,Направление 1
3,2022-07-01 00:00:00,1 поток,10:00:00,12:00:00,Сессия 1,Нозологии,Направление 2


In [231]:
# Просмотр информации о датасете

data_rubric.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 343 entries, 2 to 344
Data columns (total 7 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   date_rubric   343 non-null    object
 1   potok_rubric  343 non-null    object
 2   time_start    343 non-null    object
 3   time_end      327 non-null    object
 4   session       342 non-null    object
 5   class         343 non-null    object
 6   direction     343 non-null    object
dtypes: object(7)
memory usage: 18.9+ KB


Датасет data_rubric нужно обработать аналогично обработанным другим датасетам этого исследования.

In [232]:
# Просмотр обозначения номера потока

data_rubric['potok_rubric'].unique()

array(['1 поток ', '2 поток', '3 поток', '4 поток', '5 поток', '6 поток',
       '10 поток', '7 поток', '8 поток', '9 поток'], dtype=object)

In [233]:
# Просмотр информации о столбце

data_rubric['potok_rubric'].info()

<class 'pandas.core.series.Series'>
RangeIndex: 343 entries, 2 to 344
Series name: potok_rubric
Non-Null Count  Dtype 
--------------  ----- 
343 non-null    object
dtypes: object(1)
memory usage: 2.8+ KB


In [234]:
# Замена записи номера потока трансляций
# Функция potok_changing определена в разделе обработки датасета data_events

data_rubric['potok'] = data_rubric['potok_rubric'].apply(potok_changing)

In [235]:
# Контроль выполнения операции

data_rubric['potok'].unique()

array([ 1,  2,  3,  4,  5,  6, 10,  7,  8,  9], dtype=int64)

In [236]:
# Замена записи номера потока трансляций
# Функция potok_changing определена в разделе обработки датасета data_users

data_rubric['date'] = data_rubric['date_rubric'].apply(date_only)

In [237]:
# Контроль выполнения операции

data_rubric['date'].unique()

array(['2022-07-01', '2022-07-02', '2022-07-03', '2022-07-04',
       '2022-07-05', '2022-07-06', '2022-07-07', '2023-04-28',
       '2023-04-29', '2023-04-30', '2023-05-01', '2023-05-02'],
      dtype=object)

In [238]:
# Функция замены типа данных

def time_func(cell):
    result = str(cell)
    return result

In [239]:
# В части строк исходного датасета отсутствует время окончания рубрики
# Заменю пропущенное значение на 18:00

data_rubric = data_rubric.fillna('18:00:00')

In [240]:
# Замена типа данных

data_rubric['time_end_str'] = data_rubric['time_end'].apply(time_func)

data_rubric['time_start_str'] = data_rubric['time_start'].apply(time_func)

In [241]:
# Контроль выполнения операции

data_rubric['time_start_str'].unique()

array(['10:00:00', '13:00:00', '15:30:00', '09:30:00', '14:15:00',
       '16:45:00', '12:30:00', '15:00:00', '09:45:00', '09:20:00',
       '11:15:00', '13:45:00', '16:15:00', '09:00:00', '15:15:00',
       '14:00:00', '12:00:00', '08:20:00', '11:30:00', '17:00:00',
       '10:15:00', '12:45:00', '16:00:00', '15:20:00', '10:35:00',
       '17:45:00', '12:15:00', '10:30:00', '14:30:00', '14:45:00',
       '11:50:00', '11:25:00', '14:20:00', '13:30:00', '12:05:00',
       '14:25:00', '08:30:00', '09:05:00', '11:00:00', '12:10:00',
       '14:40:00', '10:20:00', '16:30:00', '09:15:00', '11:45:00',
       '13:50:00'], dtype=object)

In [242]:
# Контроль выполнения операции

data_rubric['time_end_str'].unique()

array(['12:00:00', '15:00:00', '17:25:00', '18:00:00', '17:30:00',
       '14:00:00', '15:15:00', '16:30:00', '17:45:00', '14:30:00',
       '17:00:00', '12:20:00', '12:15:00', '11:00:00', '13:30:00',
       '14:45:00', '16:00:00', '17:15:00', '14:35:00', '14:40:00',
       '11:40:00', '17:05:00', '16:40:00', '10:00:00', '11:15:00',
       '13:45:00', '18:15:00', '13:10:00', '10:35:00', '14:10:00',
       '17:40:00', '11:30:00', '11:45:00', '14:15:00', '16:45:00',
       '12:30:00', '15:30:00', '11:35:00', '16:50:00', '14:50:00',
       '17:35:00', '11:50:00', '12:25:00', '19:00:00', '13:00:00',
       '13:55:00', '16:25:00', '09:05:00', '10:40:00', '14:20:00',
       '10:20:00', '13:35:00', '15:55:00', '18:30:00', '11:20:00'],
      dtype=object)

In [243]:
# Формирование столбцов даты и времени начала и окончания сессий

data_rubric['date_rubric_start'] = data_rubric['date']\
                                            + ' ' + data_rubric['time_start_str']

data_rubric['date_rubric_end'] = data_rubric['date']\
                                            + ' ' + data_rubric['time_end_str']

In [244]:
# Перевод значений в формат времени

data_rubric['date_rubric_start'] = pd.to_datetime(data_rubric['date_rubric_start'])                                            

data_rubric['date_rubric_end'] = pd.to_datetime(data_rubric['date_rubric_end'])

In [245]:
# Формирование столбцов с временными точками начала и окончания сессий
# Функция date_to_seconds определена в разделе обработки датасета data_events

data_rubric['timestamp_rubric_start'] = data_rubric['date_rubric_start'].apply(date_to_seconds)

data_rubric['timestamp_rubric_end'] = data_rubric['date_rubric_end'].apply(date_to_seconds)

In [246]:
# Отсечение лишних столбцов

data_rubric = data_rubric[['date', 'potok', 'session', 'class', 'direction',
                                        'date_rubric_start', 'date_rubric_end',
                           'timestamp_rubric_start', 'timestamp_rubric_end']]

In [247]:
# Контроль выполнения операций

data_rubric.head(1)

,date,potok,session,class,direction,date_rubric_start,date_rubric_end,timestamp_rubric_start,timestamp_rubric_end
2,2022-07-01,1,Сессия 1,Основное,Направление 1,2022-07-01 10:00:00,2022-07-01 12:00:00,1.656670e+09,1.656677e+09


## Объединение датасета с датасетом словарём рубрик

#### Краткое описание операций при  объединении с датасетом словарём рубрик форума 


Формирование списка дат малых мероприятий (необходим для включения в фильтрацию, т.к. для этих мероприятий нет данных в словаре рубрик).

Объединение датасетов data и data_rubric по дате и нормеру потока трансляций с образованием датасета data_full.

Фильтрация строк по совпадениям времени доклада и времени сессии (рубрики). Добавлены строки с малыми мероприятиями (по дате) и мероприятия потока 11, для которых также нет данных в словаре рубрик. Получение датасета data_full_filtered.

Замена пропусков в строках без данных о рубрике символом "-".


In [248]:
# В словаре рубрик нет данных о малых мероприятиях, поэтому нужно их включить по датам

date_small_events = list(data_small_events['date'].unique())

In [249]:
# Объединение датасетов

data_full = data.merge(data_rubric, on=['date', 'potok'], how='left')

In [250]:
# Контроль выполнения операции

data_full.head(1)

,date,potok,user_date,user_id,o_id,barcode,device,country,region,user_city,profession,specialization,event,user_timestamp_start,user_timestamp_end,user_date_start,user_date_end,datestart_event,dateend_event,report_name,speaker_name,organization,city_event,timestamp_start_event,timestamp_end_event,duration_sec_event,duration_fact,duration_part,session,class,direction,date_rubric_start,date_rubric_end,timestamp_rubric_start,timestamp_rubric_end
0,2022-07-05,1.0,05.07.2022,14825.0,530217.0,0306202210231114825,Десктоп,Россия,Чувашия,Чебоксары,Врач,Лучевая терапия (радиотерапия),Мероприятие 11,1657008218,1657010949,05.07.2022 8:03,05.07.2022 8:49,2022-07-05 08:40:00+00:00,2022-07-05 08:55:00+00:00,Тема 422,Волчанинова Жанна Вадимовна,no_organization,Санкт-Петербург,1.657010e+09,1.657011e+09,900.0,549.0,61.0,Сессия 65,Основное,Направление 3,2022-07-05 08:20:00,2022-07-05 11:00:00,1.657009e+09,1.657019e+09


In [251]:
# Подсчёт количества строк перед фильтрацией

len(data_full)

1920572

In [252]:
# Фильтрация строк по совпадениям времени доклада и времени сессии
# Добавлены строки с малыми мероприятиями (по дате)
# Добавлены мероприятия потока 11, для которых также нет данных в рубрикаторе

data_full_filtered = data_full.\
                query('(timestamp_start_event <= timestamp_rubric_end <= timestamp_end_event) |\
(timestamp_start_event <= timestamp_rubric_start <= timestamp_rubric_end <= timestamp_end_event) |\
(timestamp_start_event <= timestamp_rubric_start <= timestamp_end_event) |\
(timestamp_rubric_start <= timestamp_start_event <= timestamp_end_event <= timestamp_rubric_end) |\
date in @date_small_events | potok == 11')


In [253]:
# Контроль выполнения операции

data_full_filtered['potok'].unique()

array([1.0, 3.0, 4.0, 6.0, 2.0, 5.0, 8.0, 7.0, 10.0, 9.0, 11.0],
      dtype=object)

In [254]:
# Вывод дат малых мероприятий

date_small_events

['2022-03-13',
 '2022-03-14',
 '2022-07-22',
 '2022-10-31',
 '2023-03-16',
 '2023-04-15',
 '2023-01-29',
 '2022-07-21']

In [255]:
# Контроль выполнения операции

data_full_filtered.query('date == "2022-03-13"').head(1)

,date,potok,user_date,user_id,o_id,barcode,device,country,region,user_city,profession,specialization,event,user_timestamp_start,user_timestamp_end,user_date_start,user_date_end,datestart_event,dateend_event,report_name,speaker_name,organization,city_event,timestamp_start_event,timestamp_end_event,duration_sec_event,duration_fact,duration_part,session,class,direction,date_rubric_start,date_rubric_end,timestamp_rubric_start,timestamp_rubric_end
1891180,2022-03-13,1.0,13.03.2022,16573.0,626199.0,0809202204421616573,Десктоп,Россия,Амурская обл.,-,Врач,Онкология: хирургия (огш),Мероприятие 1,1647165297,1647166185,13.03.2022 9:54,13.03.2022 10:09,2022-03-13 10:00:00,2022-03-13 11:15:00,Тема 1,Ямаева Анна Даниловна,-,-,1.647166e+09,1.647170e+09,4500.0,585.0,13.0,NaN,NaN,NaN,NaT,NaT,NaN,NaN


In [256]:
# Замена пропусков в строках без классификации

data_full_filtered = data_full_filtered.fillna('-')

In [257]:
# Контроль выполнения операции

data_full_filtered.query('date == "2022-03-13"').head(1)

,date,potok,user_date,user_id,o_id,barcode,device,country,region,user_city,profession,specialization,event,user_timestamp_start,user_timestamp_end,user_date_start,user_date_end,datestart_event,dateend_event,report_name,speaker_name,organization,city_event,timestamp_start_event,timestamp_end_event,duration_sec_event,duration_fact,duration_part,session,class,direction,date_rubric_start,date_rubric_end,timestamp_rubric_start,timestamp_rubric_end
1891180,2022-03-13,1.0,13.03.2022,16573.0,626199.0,0809202204421616573,Десктоп,Россия,Амурская обл.,-,Врач,Онкология: хирургия (огш),Мероприятие 1,1647165297,1647166185,13.03.2022 9:54,13.03.2022 10:09,2022-03-13 10:00:00,2022-03-13 11:15:00,Тема 1,Ямаева Анна Даниловна,-,-,1.647166e+09,1.647170e+09,4500.0,585.0,13.0,-,-,-,-,-,-,-


In [258]:
# Контроль выполнения операции

data_full_filtered.sample(2)

,date,potok,user_date,user_id,o_id,barcode,device,country,region,user_city,profession,specialization,event,user_timestamp_start,user_timestamp_end,user_date_start,user_date_end,datestart_event,dateend_event,report_name,speaker_name,organization,city_event,timestamp_start_event,timestamp_end_event,duration_sec_event,duration_fact,duration_part,session,class,direction,date_rubric_start,date_rubric_end,timestamp_rubric_start,timestamp_rubric_end
219064,2022-07-02,2.0,02.07.2022,8877.0,506452.0,20320220838388876.0,Десктоп,Россия,Республика Карелия,Петрозаводск,Врач,-,Мероприятие 11,1656774923,1656775914,02.07.2022 15:15,02.07.2022 15:31,2022-07-02 15:30:00+00:00,2022-07-02 15:45:00+00:00,Тема 156,Дежнев Владислав Дамирович,no_organization,Санкт-Петербург,1.656776e+09,1.656777e+09,900.0,114.0,13.0,Сессия 22,Нозологии,Направление 11,2022-07-02 15:00:00,2022-07-02 17:00:00,1656774000.0,1656781200.0
263587,2022-07-03,1.0,03.07.2022,426.0,534604.0,27062022101825424.0,Десктоп,Россия,Чувашия,Чебоксары,Врач,Химиотерапевт,Мероприятие 11,1656861086,1656863673,03.07.2022 15:11,03.07.2022 15:54,2022-07-03 15:00:00+00:00,2022-07-03 15:15:00+00:00,Тема 245,Березутский Яков Марселевич,no_organization,Москва,1.656860e+09,1.656861e+09,900.0,214.0,24.0,Сессия 35,Основное,Направление 14,2022-07-03 15:00:00,2022-07-03 17:00:00,1656860400.0,1656867600.0


In [259]:
# Подсчёт количества строк после фильтрации

len(data_full_filtered)

599968

In [260]:
# Подсчёт количества уникальных user_id до объединения датасетов

data['user_id'].nunique()

2730

In [261]:
# Подсчёт количества уникальных user_id после объединения датасетов

data_full_filtered['user_id'].nunique()

2726

In [262]:
# Подсчёт дубликатов

data_full_filtered.duplicated().sum()

2367

In [263]:
# Удаление дубликатов

data_full_filtered = data_full_filtered.dropna()

## Группировка итогового датасета

#### Краткое описание операций при группировке датасета и обработке итогового датасета


Группировка датасета data_full_filtered по данным зрителя (user_id, country, profession и др.) и данным доклада (report_name, organization, direction и др.) с образованием датасета data_full_group. В группировку включён расчёт вспомогательной метрики - числа подключений к трансляции одного и того же мероприятия/доклада.

Просмотр строк со значением процента длительности просмотра докладов duration_part более 100, т.е., строк, в которых значение duration_part указывают на то, что доклад был просмотрен одновременно более одного раза в одно и то же время. 

Оценка и подсчёт количественных и качественных показателей параллельных просмотров трансляций докладов.

Выводы.

Замена значений duration_part в строках, где duration_part более 100% на 100%.



In [264]:
# Группировка по данным зрителя и данным доклада

data_full_group = data_full_filtered.groupby(['date', 'potok', 'user_id', 'o_id', 'barcode',
                        'device', 'country', 'region', 'user_city', 'profession',
                        'specialization', 'session', 'class', 'direction',
                        'date_rubric_start', 'date_rubric_end', 'event', 'city_event',
                        'organization', 'report_name', 'speaker_name', 
                        'duration_sec_event'],
                      as_index=False).agg\
                        (datestart_event=('datestart_event', 'first'),
                         dateend_event=('dateend_event', 'first'),
                        duration_part_total=('duration_part', 'sum'),
                         connections_cnt=('duration_part', 'count')
                        ).sort_values(by='duration_part_total', ascending = False).reset_index()

data_full_group.drop('index', axis= 1 , inplace= True )

In [265]:
# Вывод итогового датасета
# Контроль выполнения операции

data_full_group.head(3)

,date,potok,user_id,o_id,barcode,device,country,region,user_city,profession,specialization,session,class,direction,date_rubric_start,date_rubric_end,event,city_event,organization,report_name,speaker_name,duration_sec_event,datestart_event,dateend_event,duration_part_total,connections_cnt
0,2023-04-28,5.0,10108.0,907010.0,1703202316454810108,Мобильное,Россия,Ленинградская обл.,Санкт-Петербург,Студент,Эндоскопия,Сессия 128,Основное,Направление 7,2023-04-28 15:00:00,2023-04-28 17:00:00,Мероприятие 12,"Минск, Беларусь",no_organization,Тема 819,Бакшаев Радик Алексеевич,300.0,2023-04-28 16:55:00+00:00,2023-04-28 17:00:00+00:00,19695.0,204
1,2023-04-28,5.0,10108.0,907010.0,1703202316454810108,Мобильное,Россия,Ленинградская обл.,Санкт-Петербург,Студент,Эндоскопия,Сессия 128,Основное,Направление 7,2023-04-28 15:00:00,2023-04-28 17:00:00,Мероприятие 12,"Минск, Беларусь",no_organization,Тема 819,Бакшаев Радик Алексеевич,900.0,2023-04-28 16:40:00+00:00,2023-04-28 16:55:00+00:00,18989.0,190
2,2023-04-28,2.0,3710.0,923617.0,080620232045573710,Десктоп,Россия,Республика Карелия,Петрозаводск,Врач,-,Сессия 119,Основное,Направление 7,2023-04-28 14:45:00,2023-04-28 16:45:00,Мероприятие 12,Санкт-Петербург,no_organization,Тема 769,Полянина Наталья Ринатовна,300.0,2023-04-28 16:40:00+00:00,2023-04-28 16:45:00+00:00,18492.0,202


In [266]:
# Контроль выполнения операции

data_full_group['potok'].unique()

array([ 5.,  2.,  6.,  3.,  7.,  1.,  8.,  4., 10.,  9., 11.])

In [267]:
# Контроль выполнения операции

data_full_group.query('date =="2022-03-13"').head(1)

,date,potok,user_id,o_id,barcode,device,country,region,user_city,profession,specialization,session,class,direction,date_rubric_start,date_rubric_end,event,city_event,organization,report_name,speaker_name,duration_sec_event,datestart_event,dateend_event,duration_part_total,connections_cnt
281947,2022-03-13,1.0,2610.0,619088.0,250820222147302610,Десктоп,Молдова,Молдова,Тирасполь,Врач,Химиотерапевт,-,-,-,-,-,Мероприятие 1,-,-,Тема 1,Ямаева Анна Даниловна,4500.0,2022-03-13 10:00:00,2022-03-13 11:15:00,59.0,1


In [268]:
# Просмотр числа подключений

data_full_group['connections_cnt'].describe()

count    321353.000000
mean          1.867006
std           3.288336
min           1.000000
25%           1.000000
50%           1.000000
75%           2.000000
max         204.000000
Name: connections_cnt, dtype: float64

В сгруппированной таблице оказались строки со значением процента просмотренной трансляции многократно превышающие 100%. В сгруппированную таблицу добавлен столбец с количеством подключений к трансляции. Выше рассматривался случай, когда один и тот же зритель дважды с небольшим перерывом подключался к трансляции. В сгруппированной таблице максимальное число подключений одного зрителя (один id, o_id, barcode) - 230. Рассмотрю подобные случаи подробнее.

In [269]:
# Вывод строк просмотров дискуссии (здесь организация имеет значение no_organization)
# после доклада по теме: "Тема 819" зрителя с user_id равным 10108

data_10108 = data_full_filtered.query('user_id == 10108.0 &\
    report_name == "Тема 819" &\
    organization == "no_organization"')

data_10108.sample(3)

,date,potok,user_date,user_id,o_id,barcode,device,country,region,user_city,profession,specialization,event,user_timestamp_start,user_timestamp_end,user_date_start,user_date_end,datestart_event,dateend_event,report_name,speaker_name,organization,city_event,timestamp_start_event,timestamp_end_event,duration_sec_event,duration_fact,duration_part,session,class,direction,date_rubric_start,date_rubric_end,timestamp_rubric_start,timestamp_rubric_end
1143767,2023-04-28,5.0,28.04.2023,10108.0,907010.0,1703202316454810108,Мобильное,Россия,Ленинградская обл.,Санкт-Петербург,Студент,Эндоскопия,Мероприятие 12,1682700017,1682706628,28.04.2023 16:40,28.04.2023 18:30,2023-04-28 16:55:00+00:00,2023-04-28 17:00:00+00:00,Тема 819,Бакшаев Радик Алексеевич,no_organization,"Минск, Беларусь",1.682701e+09,1.682701e+09,300.0,300.0,100.0,Сессия 128,Основное,Направление 7,2023-04-28 15:00:00,2023-04-28 17:00:00,1682694000.0,1682701200.0
1138117,2023-04-28,5.0,28.04.2023,10108.0,907010.0,1703202316454810108,Мобильное,Россия,Ленинградская обл.,Санкт-Петербург,Студент,Эндоскопия,Мероприятие 12,1682689464,1682706628,28.04.2023 13:44,28.04.2023 18:30,2023-04-28 16:40:00+00:00,2023-04-28 16:55:00+00:00,Тема 819,Бакшаев Радик Алексеевич,no_organization,"Минск, Беларусь",1.682700e+09,1.682701e+09,900.0,900.0,100.0,Сессия 128,Основное,Направление 7,2023-04-28 15:00:00,2023-04-28 17:00:00,1682694000.0,1682701200.0
1141572,2023-04-28,5.0,28.04.2023,10108.0,907010.0,1703202316454810108,Мобильное,Россия,Ленинградская обл.,Санкт-Петербург,Студент,Эндоскопия,Мероприятие 12,1682694873,1682706628,28.04.2023 15:14,28.04.2023 18:30,2023-04-28 16:55:00+00:00,2023-04-28 17:00:00+00:00,Тема 819,Бакшаев Радик Алексеевич,no_organization,"Минск, Беларусь",1.682701e+09,1.682701e+09,300.0,300.0,100.0,Сессия 128,Основное,Направление 7,2023-04-28 15:00:00,2023-04-28 17:00:00,1682694000.0,1682701200.0


In [270]:
# Уникальные значения доли просмотра трансляции зрителем с id 10108

data_10108['duration_part'].unique()

array([100.,  83.,  67.,  46.,  15.,  95.,  64.,  61.,  57.,  50.,  35.,
        23.,  11.,  94.,  86.,  38.,  37.,  24.,  20.,  17.,  65.,  87.,
        19.,  93.,  48.,  44.,  26.,  18.,  16.,  98.,  92.,  77.,  63.,
        91.,  89.,   0.,  22.,   2.,  97.,  96.])

In [271]:
# Выделение в отдельный датасет строк с долей просмотра трансляций более 100%

data_donors = data_full_group.query('duration_part_total > 100')

data_donors.head(2)

,date,potok,user_id,o_id,barcode,device,country,region,user_city,profession,specialization,session,class,direction,date_rubric_start,date_rubric_end,event,city_event,organization,report_name,speaker_name,duration_sec_event,datestart_event,dateend_event,duration_part_total,connections_cnt
0,2023-04-28,5.0,10108.0,907010.0,1703202316454810108,Мобильное,Россия,Ленинградская обл.,Санкт-Петербург,Студент,Эндоскопия,Сессия 128,Основное,Направление 7,2023-04-28 15:00:00,2023-04-28 17:00:00,Мероприятие 12,"Минск, Беларусь",no_organization,Тема 819,Бакшаев Радик Алексеевич,300.0,2023-04-28 16:55:00+00:00,2023-04-28 17:00:00+00:00,19695.0,204
1,2023-04-28,5.0,10108.0,907010.0,1703202316454810108,Мобильное,Россия,Ленинградская обл.,Санкт-Петербург,Студент,Эндоскопия,Сессия 128,Основное,Направление 7,2023-04-28 15:00:00,2023-04-28 17:00:00,Мероприятие 12,"Минск, Беларусь",no_organization,Тема 819,Бакшаев Радик Алексеевич,900.0,2023-04-28 16:40:00+00:00,2023-04-28 16:55:00+00:00,18989.0,190


In [272]:
# Выяснение самых востребованных потоков трансляций у пользователей, раздающих свои данные

data_donors_potok = data_donors.groupby('potok', as_index=False)['potok'].value_counts()

potok = data.groupby('potok', as_index=False)['potok'].value_counts()

data_donors_potok = data_donors_potok.merge(potok, on='potok', how='left')

data_donors_potok['demand_potok'] = round(data_donors_potok['count_x'] /\
                                          data_donors_potok['count_y']*100)

data_donors_potok.sort_values(by='demand_potok', ascending=False)

,potok,count_x,count_y,demand_potok
7,8.0,3180,10530,30.0
1,2.0,21562,85383,25.0
6,7.0,7035,28653,25.0
3,4.0,11020,47217,23.0
9,10.0,175,750,23.0
2,3.0,10711,60009,18.0
4,5.0,8887,49751,18.0
5,6.0,5413,35079,15.0
0,1.0,9551,88716,11.0
8,9.0,29,272,11.0


In [273]:
# Подсчёт массовости явления раздачи данных для просмотра трансляций

print('Количество зрителей (user_id), организующих параллельные просмотры трансляций:',
      data_donors['user_id'].nunique())

print()

print('Процент зрителей (user_id), организующих параллельные просмотры трансляций:',
      round((data_donors['user_id'].nunique()/data['user_id'].nunique()*100), 1), '%'
     )

print()

print('Количество зрителей (user_id), раздающих свои данные для просмотра трансляций более,\
        чем 100 зрителям:',
      data_full_group.query('duration_part_total > 10000')['user_id'].nunique())

print()

print('Количество зрителей (user_id), раздающих свои данные для просмотра трансляций более,\
        чем 10 зрителям:',
      data_full_group.query('duration_part_total > 1000')['user_id'].nunique())

print()

print('Количество зрителей (user_id), раздающих свои данные для просмотра трансляций более,\
        чем двум зрителям:',
      data_full_group.query('duration_part_total > 200')['user_id'].nunique())

print()

print('Количество докладов (без учёта разделения на основную часть и дискуссию),\
    транслируемых параллельно:', data_donors['report_name'].nunique())

print()

print('Процент докладов (без учёта разделения на основную часть и дискуссию),\
    транслируемых параллельно:',
      round((data_donors['report_name'].nunique()/data['report_name'].nunique()*100), 1), '%'
     )


Количество зрителей (user_id), организующих параллельные просмотры трансляций: 1332

Процент зрителей (user_id), организующих параллельные просмотры трансляций: 48.8 %

Количество зрителей (user_id), раздающих свои данные для просмотра трансляций более,        чем 100 зрителям: 3

Количество зрителей (user_id), раздающих свои данные для просмотра трансляций более,        чем 10 зрителям: 176

Количество зрителей (user_id), раздающих свои данные для просмотра трансляций более,        чем двум зрителям: 750

Количество докладов (без учёта разделения на основную часть и дискуссию),    транслируемых параллельно: 697

Процент докладов (без учёта разделения на основную часть и дискуссию),    транслируемых параллельно: 49.0 %


In [274]:
# Страны зрителей, организующиих параллельные трансляции

data_donors['country'].unique()

array(['Россия', 'Узбекистан', 'Беларусь', 'Украина', '-', 'Казахстан',
       'Молдова', 'Кыргызстан', 'Армения', 'Азербайджан', 'Латвия',
       'Китай'], dtype=object)

In [275]:
# Все страны зрителей трансляций

data['country'].unique()

array(['Россия', '-', 'Кыргызстан', 'Казахстан', 'Беларусь', 'Украина',
       'Узбекистан', 'Молдова', 'Германия', 'Таджикистан', 'Китай',
       'Азербайджан', 'Армения', 'Латвия'], dtype=object)

#### Выводы


Выясняется, что почти 49% зрителей или делятся своими данными для просмотра трансляций, или включают одновременно множество вкладок с разными трансляциями на одном устройстве, или и то и другое одновременно. Параллельно транслируется 49% докладов (без учёта того, это основная часть доклада или это дискуссия после доклада).


Число зрителей каким-либо образом организующие более, чем 100 параллельных трансляций - 3 пользователя; более, чем десять параллельных трансляций 176 пользователей; две трансляции - 750 пользователей. Можно сделать вывод о том, что параллельные трансляции - явление массовое, но, при этом, злостных нарушителей не так много.


Параллельные просмотры трансляций организуют зрители из всех стран, указанных в данных зрителями  о себе, кроме зрителей из Германии и Таджикистана. При этом, в изначальном датасете присутствуют зрители, не указавшие свою страну, такие зрители есть и среди организаторов параллельных просмотров.


С большим отрывом по частоте чаще всего делятся данными для просмотра трансляций 8-ого потока. Самый непопулярный поток в этом случае - поток № 11.

In [276]:
# Вывод датасета в текущем виде

data_full_group.head(2)

,date,potok,user_id,o_id,barcode,device,country,region,user_city,profession,specialization,session,class,direction,date_rubric_start,date_rubric_end,event,city_event,organization,report_name,speaker_name,duration_sec_event,datestart_event,dateend_event,duration_part_total,connections_cnt
0,2023-04-28,5.0,10108.0,907010.0,1703202316454810108,Мобильное,Россия,Ленинградская обл.,Санкт-Петербург,Студент,Эндоскопия,Сессия 128,Основное,Направление 7,2023-04-28 15:00:00,2023-04-28 17:00:00,Мероприятие 12,"Минск, Беларусь",no_organization,Тема 819,Бакшаев Радик Алексеевич,300.0,2023-04-28 16:55:00+00:00,2023-04-28 17:00:00+00:00,19695.0,204
1,2023-04-28,5.0,10108.0,907010.0,1703202316454810108,Мобильное,Россия,Ленинградская обл.,Санкт-Петербург,Студент,Эндоскопия,Сессия 128,Основное,Направление 7,2023-04-28 15:00:00,2023-04-28 17:00:00,Мероприятие 12,"Минск, Беларусь",no_organization,Тема 819,Бакшаев Радик Алексеевич,900.0,2023-04-28 16:40:00+00:00,2023-04-28 16:55:00+00:00,18989.0,190


При наличии в датасете значения процента просмотра трансляций более 100 проводить анализ неудобно. Выше было выяснено, что на практике это означает параллельные просмотры. Количество реальных зрителей при таких парллельных просмотрах выяснить невозможно. К тому же, без параллельных просмотров и раздачи данных для просмотра смотреть трансляцию со одного устройства  может более одного человека. Таким образом, т.к., в рамках текущего исследования невозможно выяснить фактическое число зрителей каждой трансляции, для решения целей текущего исследования существенных различий между зрителем с параллельными трансляциями и без - нет (кроме отмеченных случаев). Принимаю решение заменить все значения просмотра трансляций более 100% на значение 100%; каждый уникальный user_id считать одним уникальным зрителем.


In [277]:
# Функция замены значений процента просмотра трансляций

def total_duration_func(cell):
    if cell > 100:
        cell = 100
    else:
        cell = cell
    return cell

In [278]:
# Замена значений процента длительности просмотра трансляций

data_full_group['duration_part_total_normal'] = data_full_group['duration_part_total']\
                                                    .apply(total_duration_func)

In [279]:
# Контроль выполнения операции

data_full_group.head(2)

,date,potok,user_id,o_id,barcode,device,country,region,user_city,profession,specialization,session,class,direction,date_rubric_start,date_rubric_end,event,city_event,organization,report_name,speaker_name,duration_sec_event,datestart_event,dateend_event,duration_part_total,connections_cnt,duration_part_total_normal
0,2023-04-28,5.0,10108.0,907010.0,1703202316454810108,Мобильное,Россия,Ленинградская обл.,Санкт-Петербург,Студент,Эндоскопия,Сессия 128,Основное,Направление 7,2023-04-28 15:00:00,2023-04-28 17:00:00,Мероприятие 12,"Минск, Беларусь",no_organization,Тема 819,Бакшаев Радик Алексеевич,300.0,2023-04-28 16:55:00+00:00,2023-04-28 17:00:00+00:00,19695.0,204,100.0
1,2023-04-28,5.0,10108.0,907010.0,1703202316454810108,Мобильное,Россия,Ленинградская обл.,Санкт-Петербург,Студент,Эндоскопия,Сессия 128,Основное,Направление 7,2023-04-28 15:00:00,2023-04-28 17:00:00,Мероприятие 12,"Минск, Беларусь",no_organization,Тема 819,Бакшаев Радик Алексеевич,900.0,2023-04-28 16:40:00+00:00,2023-04-28 16:55:00+00:00,18989.0,190,100.0


## Исследование датасета

#### Краткое описание операций при  исследовании итогового датасета


Формирование вспомогательной группировки датасета - data_group группировки датасета data (полученного ранее, до присоединения датасета словаря рубрик). 

Выделение года из дат в датасете data_group.

Выделение в отдельный датасет строк без дискуссий после докладов - data_group_no_discuss.

Группировка data_group_no_discuss по дате доклада (с днём и месям) - data_days.

Группировка data_group_no_discuss по году доклада - data_year.

Выводы по результатам группировок по датам докладов.

Группировка data_group по устройству зрителя - data_device.

Выводы по результату группировки по устройству зрителя.

Группировка data_group по стране зрителя - data_country.

Расчёт и сравнение характеристик зрителей по странам.

Выводы по результату группировки по стране зрителя.

Выделение строк со зрителями из России в датасет data_rus. Группировка data_rus по регионам. 

Расчёт и сравнение характеристик зрителей по регионам России.

Выводы по результату группировки по региону зрителя (для России).

Выделение строк со зрителями из России в датасет data_city. Группировка data_city по городам. 

Расчёт и сравнение характеристик зрителей по городам России.

Выводы по результату группировки по городу зрителя (для России).

Группировка data_group по профессии зрителя - data_prof.

Группировка data_group по специализации зрителя - data_spec.

Просмотр строк с обозначением профессии и специализации зрителя общими понятиями.

Выводы по результатам группировок по профессиям и специализациям зрителей.

Группировка датасета data_group по названиям докладов (включая дискуссии) - data_report.

Группировка датасета data_group_no_discuss по названиям докладов (исключая дискуссии) - data_report_no_discuss.

Расчёты и сравнения просматриваемости докладов и последующих за ними дискуссий.

Сравнение просматриваемости докладов и дискуссий длительностью 5 минут, 10 минут, 15 минут.

Выводы по результатам расчётов и группировок по названию доклада. Приведение списка из 10-ти наиболее популярных докладов.

Группировка датасета data_group_no_discuss по имени докладчика - data_name_no_discuss.

Выводы по результату группировки по имени докладчика. Приведение списка из 10-ти наиболее популярных докладчиков.

Группировка датасета data_full_group по названию конференции/семинара - data_event_views.

Выводы по результату группировки по названию конференции/семинара.

Группировка датасета data_full_group по названию сессии - data_sessions_views.

Выводы по результату группировки по названию сессии.

Группировка датасета data_full_group по коду классификации - data_class_views.

Выводы по результату группировки по коду классификации.

Поиск популярных направлений докладов. Группировка датасета data_full_group по направлению доклада - data_directions_views.

Выводы по результату группировки по направлению доклада. Приведение списка из 10-ти наиболее популярных направлений докладов.

Поиск востребованных направлений докладов. Группировка датасета data_full_group по направлению доклада - demand_directions.

Введение вспомогательных метрик для оценки заинтересованности зрителей. Формирование списка востребованных направлений докладов.


❗ Присоединение рубрикатора привело к тому, что часть строк продублировалось. Это произошло потому, что один и тот же доклад может соответствовать двум направлениям/классам. Для дальнейшего анализа докладов с точки зрения классификации по предложенному рубрикатору такой датасет подойдёт. Однако, для анализа числа/популярности просмотров, тем, докладов и докладчиков такой датасет труден для изучения. Сформирую отдельный датасет с просмотрами, но без данных рубрикатора.

In [280]:
# Группировка по данным зрителя, теме доклада и части доклада (основной и дискуссия)

data_group = data.groupby(['date', 'potok', 'user_id', 'o_id', 'barcode', 'device', 'country',
                       'region', 'user_city', 'profession', 'specialization','report_name',
                       'speaker_name', 'organization', 'event', 'city_event', 'duration_sec_event'],
                      as_index=False).agg\
                         (duration_part_total=('duration_part', 'sum'),
                         connections_cnt=('duration_part', 'count')
                        ).sort_values(by='duration_part_total', ascending = False).reset_index()

data_group.drop('index', axis= 1 , inplace= True )

In [281]:
# Замена значений процента просмотра трансляций

data_group['duration_part_total_normal'] = data_group['duration_part_total']\
                                                    .apply(total_duration_func)

In [282]:
# Просмотр группировки

data_group.head(3)

,date,potok,user_id,o_id,barcode,device,country,region,user_city,profession,specialization,report_name,speaker_name,organization,event,city_event,duration_sec_event,duration_part_total,connections_cnt,duration_part_total_normal
0,2023-04-28,5.0,10108.0,907010.0,1703202316454810108,Мобильное,Россия,Ленинградская обл.,Санкт-Петербург,Студент,Эндоскопия,Тема 819,Бакшаев Радик Алексеевич,no_organization,Мероприятие 12,"Минск, Беларусь",300.0,20993.0,230,100.0
1,2023-04-28,2.0,3710.0,923617.0,080620232045573710,Десктоп,Россия,Республика Карелия,Петрозаводск,Врач,-,Тема 769,Полянина Наталья Ринатовна,no_organization,Мероприятие 12,Санкт-Петербург,300.0,19790.0,229,100.0
2,2023-04-28,5.0,10108.0,907010.0,1703202316454810108,Мобильное,Россия,Ленинградская обл.,Санкт-Петербург,Студент,Эндоскопия,Тема 819,Бакшаев Радик Алексеевич,no_organization,Мероприятие 12,"Минск, Беларусь",900.0,18989.0,190,100.0


In [283]:
# Перевод данных в тип дат

data_group['date'] = pd.to_datetime(data_group['date'])

In [284]:
# Функция выделения года из дат

def year_only(cell):
    result = str(cell).split('-')[0]
    return result

In [285]:
# Формирование столбца с годом трансляции

data_group['year'] = data_group['date'].apply(year_only)

In [286]:
# Выделение в отдельный датасет строк без дискуссий после докладов

data_group_no_discuss = data_group.query('organization != "no_organization"')

В датасете представлены просмотры зрителей без user_id, но с другими данными: o_id, и номером билета - barcode. Для более точного анализа здесь веду подсчёт также уникальных o_id и barcode. При этом, один зритель может иметь более одного o_id и более одного билета, т.е., более одного уникального номера barcode.


In [287]:
# Вывод строки просмотра зрителя с user_id равным "no_user_id"

data_group.query('user_id == "no_user_id"').sample()

,date,potok,user_id,o_id,barcode,device,country,region,user_city,profession,specialization,report_name,speaker_name,organization,event,city_event,duration_sec_event,duration_part_total,connections_cnt,duration_part_total_normal,year
99638,2023-04-29,7.0,no_user_id,926419.0,hx9rQaioM,Десктоп,-,-,-,-,-,Тема 927,Шашина Марина Константиновна,no_organization,Мероприятие 12,Москва,4200.0,100.0,1,100.0,2023


In [288]:
# Вывод самых популярных дат трансляций,
# число их уникальных зрителей, число уникальных докладов в них

data_days = data_group_no_discuss.groupby('date', as_index=False).agg(
                            views_cnt=('user_id', 'count'),
                            users_cnt=('user_id', 'nunique'),
                            o_id_cnt=('o_id', 'nunique'),
                            barcode_cnt=('barcode', 'nunique'),
                            reports_cnt=('report_name', 'nunique')).sort_values(by='views_cnt',
                                                                              ascending=False)
data_days

,date,views_cnt,users_cnt,o_id_cnt,barcode_cnt,reports_cnt
2,2022-07-21,898,152,152,152,8
7,2023-04-15,482,56,56,56,16
5,2023-01-29,257,26,26,26,15
1,2022-03-14,180,51,51,51,5
4,2022-10-31,71,17,17,17,6
3,2022-07-22,27,27,27,27,1
6,2023-03-16,25,25,25,25,1
0,2022-03-13,18,18,18,18,1


In [289]:
# Сравнение 2022 и 2023 годов по числу просмотров, числу зрителей, числу докладов

data_year = data_group_no_discuss.groupby('year', as_index=False).agg(
                            views_cnt=('user_id', 'count'),
                            users_cnt=('user_id', 'nunique'),
                            o_id_cnt=('o_id', 'nunique'),
                            barcode_cnt=('barcode', 'nunique'),
                            reports_cnt=('report_name', 'nunique')).sort_values(by='views_cnt',
                                                                              ascending=False)


data_year

,year,views_cnt,users_cnt,o_id_cnt,barcode_cnt,reports_cnt
0,2022,1194,223,232,232,20
1,2023,764,100,107,107,32


#### Выводы


Здесь и далее отдельным просмотром обозначаю отдельную строку в датасете, соответствующую  уникальному зрителю, уникальному докладу в временных рамках только этого доклада или дискуссии.


Трансляции 2022 года набрали почти в 1,6 раза больше просмотров, чем в 2023 году. При этом, количество уникальных зрителей уменьшилось в 2,2 раза, а число докладов увеличилось в полтора раза.


С большим отрывом самая популярная дата по просмотрам - это 21 июля 2022 года. В 2023 году - это 15 апреля 2023 года.


Большое число уникальных зрителей не означает большого охвата (числа просмотров). Также, большое число докладов не обязательно даёт большое число просмотров. Например: 21 июля 2022 года 152 уникальных зрителей посмотрело 8 докладов, число просмотров 898. В другой день, 31 октября 2022 года было представлено 6 докладов, а 29 января 2023 года - 15 докладов, однако, число просмотров в эти дни значительно меньше, чем число просмотров 21 июля 2022 года. Зависимость между количеством транслируемых докладов и количеством просмотров существует, но есть немало исключений.


In [290]:
# Сравнение типов устройств по числу просмотров (включая дискуссии) и числу зрителей

data_device = data_group.groupby('device', as_index=False).agg(
                            device_cnt=('device', 'count'),
                            users_cnt=('o_id', 'nunique'),
                            o_id_cnt=('user_id', 'nunique'),
                            barcode_cnt=('barcode', 'nunique')).sort_values(by='users_cnt',
                                                                              ascending=False)


data_device

,device,device_cnt,users_cnt,o_id_cnt,barcode_cnt
0,Десктоп,142217,2150,1806,2688
1,Мобильное,76552,1772,1514,2054
2,Планшет,4658,87,81,98


#### Выводы


Десктопными устройствами пользуется немного большее число зрителей, чем зрителей с мобильными устройствами. Однако, число просмотров с десктопных устройств почти в два раза больше, чем просмотров с мобильных устройств. Число просмотров с планшетов невелико, как и число их обладателей.


In [291]:
# Сравнение стран зрителей по числу просмотров (включая дискуссии) и числу зрителей

data_country = data_group.groupby('country', as_index=False).agg(
                            views_cnt=('user_id', 'count'),
                            users_cnt=('user_id', 'nunique'),
                            o_id_cnt=('o_id', 'nunique'),
                            barcode_cnt=('barcode', 'nunique')).sort_values(by='users_cnt',
                                                                              ascending=False)


# Вычисление процентного соотношения зрителей из конкретной страны по user_id и по номеру билета

data_country['users_part'] = round(data_country['users_cnt'] / data_country['users_cnt']\
                                   .sum()*100, 2)

data_country['barcode_part'] = round(data_country['barcode_cnt'] / data_country['barcode_cnt']\
                                   .sum()*100, 2)

data_country

,country,views_cnt,users_cnt,o_id_cnt,barcode_cnt,users_part,barcode_part
10,Россия,205890,2518,3080,3806,92.10,92.11
0,-,6293,94,117,137,3.44,3.32
3,Беларусь,5801,44,58,70,1.61,1.69
5,Казахстан,1123,19,23,27,0.69,0.65
12,Узбекистан,886,18,21,25,0.66,0.61
7,Кыргызстан,1421,16,21,30,0.59,0.73
13,Украина,823,9,14,16,0.33,0.39
1,Азербайджан,372,3,3,3,0.11,0.07
2,Армения,354,3,3,3,0.11,0.07
9,Молдова,253,3,7,8,0.11,0.19


#### Выводы


Около 92% зрителей трансляций из России, менее 2% зрителей из Беларуси. Число зрителей из других стран составляет менее одного процента в каждой стране, или менее 20-ти человек в абсолютных значениях. Самое маленькое число зрителей из Латвии, Таджикистана, Германии, Китая. При этом, зрители из Латвии по числу просмотров значительно опережают зрителей из Таджикистана, Германии, Китая.


Почти 3,5% зрителей не указали свою страну.

In [292]:
# Формирование датасета с просмотрами только из России
# Сравнение регионов зрителей по числу просмотров (включая дискуссии) и числу зрителей

data_rus = data_group.query('country == "Россия"')

data_rus = data_rus.groupby('region', as_index=False).agg(
                            views_cnt=('user_id', 'count'),
                            users_cnt=('user_id', 'nunique'),
                            o_id_cnt=('o_id', 'nunique'),
                            barcode_cnt=('barcode', 'nunique')).sort_values(by='users_cnt',
                                                                              ascending=False)

# Вычисление процентного соотношения зрителей из конкретной страны по user_id и по номеру билета

data_rus['users_part'] = round(data_rus['users_cnt'] / data_rus['users_cnt']\
                                   .sum()*100, 2)

data_rus['barcode_part'] = round(data_rus['barcode_cnt'] / data_rus['barcode_cnt']\
                                   .sum()*100, 2)

data_rus.head(6)

,region,views_cnt,users_cnt,o_id_cnt,barcode_cnt,users_part,barcode_part
36,Ленинградская обл.,82812,965,1246,1517,38.32,39.86
41,Московская обл.,31961,487,554,664,19.34,17.45
54,Ростовская обл.,6274,74,92,112,2.94,2.94
31,Краснодарский край,4919,53,66,83,2.10,2.18
0,-,4647,49,59,71,1.95,1.87
43,Нижегородская обл.,3279,38,42,54,1.51,1.42


In [293]:
# Вывод регионов, где трансляции докладов оказались непопулярны

data_rus.tail(10)

,region,views_cnt,users_cnt,o_id_cnt,barcode_cnt,users_part,barcode_part
26,Карелия,147,2,2,4,0.08,0.11
16,Забайкальский край,395,2,2,2,0.08,0.05
34,Курганская обл.,345,2,3,4,0.08,0.11
78,Ямало-Ненецкий АО,214,2,2,3,0.08,0.08
1,Адыгея,48,2,2,2,0.08,0.05
24,Камчатский край,23,1,1,1,0.04,0.03
38,Липецкая область,228,1,1,1,0.04,0.03
15,Еврейская АО,314,1,2,2,0.04,0.05
61,Северная Осетия-Алания,2,1,1,1,0.04,0.03
77,Чукотский АО,67,1,1,2,0.04,0.05


In [294]:
# Число регионов, представленных в датасете

data_rus['region'].nunique()

80

#### Выводы


Более 38% зрителей трансляций из Ленинградской области. В первую пятёрку регионов зрителей также входят: Московская, Ростовская, Нижегородская области и Краснодарский край.


Менее всего зрителей из Карелии, Забайкальского края,  Курганской области, Ямало-Ненецкого АО,  Адыгеи, Камчатского края, Липецкой области, Еврейской АО, Северной Осетии - Алании, Чукотского АО.


Всего в датасете данные о просмотрах зрителей из 80-ти регионов России.


Около 2% зрителей из России не указали свой регион.

In [295]:
# Формирование датасета с просмотрами только из России
# Сравнение городов зрителей по числу просмотров (включая дискуссии) и числу зрителей

data_city = data_group.query('country == "Россия"')

data_city = data_city.groupby('user_city', as_index=False).agg(
                            views_cnt=('user_id', 'count'),
                            users_cnt=('user_id', 'nunique'),
                            o_id_cnt=('o_id', 'nunique'),
                            barcode_cnt=('barcode', 'nunique')).sort_values(by='users_cnt',
                                                                              ascending=False)

# Вычисление процентного соотношения зрителей из конкретной страны по user_id и по номеру билета

data_city['users_part'] = round(data_city['users_cnt'] / data_city['users_cnt']\
                                   .sum()*100, 2)

data_city['barcode_part'] = round(data_city['barcode_cnt'] / data_city['barcode_cnt']\
                                   .sum()*100, 2)


data_city.head(10)

,user_city,views_cnt,users_cnt,o_id_cnt,barcode_cnt,users_part,barcode_part
135,Санкт-Петербург,81637,952,1228,1493,37.79,39.23
97,Москва,29287,458,516,617,18.18,16.21
132,Ростов-на-Дону,5565,69,86,104,2.74,2.73
80,Краснодар,3842,42,55,68,1.67,1.79
104,Нижний Новгород,3070,36,41,53,1.43,1.39
54,Екатеринбург,2956,35,43,53,1.39,1.39
39,Воронеж,3071,31,38,52,1.23,1.37
166,Уфа,961,30,36,43,1.19,1.13
134,Самара,2688,30,37,45,1.19,1.18
12,Архангельск,3343,28,36,48,1.11,1.26


In [296]:
# Число городов, представленных в датасете

data_city['user_city'].nunique()

182

In [297]:
# Страны, в которых зрители не указали свой город

data_group.query('user_city == "-"')['country'].unique()

array(['-', 'Россия', 'Германия'], dtype=object)

#### Выводы


Более 37% зрителей трансляций из Санкт-Петербурга. В первую десятку городов зрителей также входят: Москва, Ростов-на-Дону, Краснодар, Нижний Новогород, Екатеринбург, Воронеж, Уфа, Самара, Архангельск. 


Стоит отметить, что большое число зрителей не всегда означает большое число зрительских сессий (просмотров). Так, в Уфе сравнительно мало просмотров.


Всего в датасете данные о просмотрах зрителей из 186-ти городов России.


Часть зрителей не указали свой город.

In [298]:
# Формирование датасета с группировкой по профессии зрителя
# Сравнение зрителей по числу просмотров (включая дискуссии)

data_prof = data_group.query('profession != "-"')

data_prof = data_prof.groupby('profession', as_index=False).agg(
                            views_cnt=('user_id', 'count'),
                            users_cnt=('user_id', 'nunique'),
                            o_id_cnt=('o_id', 'nunique'),
                            barcode_cnt=('barcode', 'nunique'),
                            reports_cnt=('report_name', 'nunique')).sort_values(by='users_cnt',
                                                                              ascending=False)

data_prof.head(10)

,profession,views_cnt,users_cnt,o_id_cnt,barcode_cnt,reports_cnt
1,Врач,147424,1605,2002,2547,1417
18,Ординатор - аспирант,17248,263,312,375,1220
11,Медперсонал,7283,143,182,208,995
19,Представитель медицинской компании,5080,115,121,133,747
10,Исследователь,7734,113,133,156,982
24,Руководящее звено клиники,8260,86,104,128,1069
26,Студент,3254,70,76,86,780
14,Онколог,6753,50,67,83,964
30,Химиотерапевт,2293,19,24,29,588
28,Фармацевт,425,9,10,13,205


In [299]:
# Формирование датасета с группировкой по специализации зрителя
# Сравнение зрителей по числу просмотров (включая дискуссии)

data_spec = data_group.query('specialization != "-"')

data_spec = data_spec.groupby('specialization', as_index=False).agg(
                            views_cnt=('user_id', 'count'),
                            users_cnt=('user_id', 'nunique'),
                            o_id_cnt=('o_id', 'nunique'),
                            barcode_cnt=('barcode', 'nunique'),
                            reports_cnt=('report_name', 'nunique')).sort_values(by='users_cnt',
                                                                              ascending=False)
data_spec.head(10)


,specialization,views_cnt,users_cnt,o_id_cnt,barcode_cnt,reports_cnt
45,Онкология: лекарственное лечение,27295,355,401,501,1293
69,Хирург онколог,27673,267,359,468,1304
68,Химиотерапевт,22190,159,209,280,1247
46,Онкология: хирургия (абдоминальная),7162,112,134,161,990
32,Лучевая терапия (радиотерапия),10994,105,133,172,1075
49,Онкология: хирургия (рмж),7257,99,108,131,883
42,Онкология,8877,85,108,137,1055
0,Акушерство и гинекология,4866,81,101,119,740
55,Патологическая анатомия,7068,77,101,129,812
30,Лучевая диагностика: рентгенология,4584,69,78,104,809


In [300]:
# Вывод специализаций зрителей, обозначивших себя как "Врач"

data_group.query('profession == "Врач"')['specialization'].unique()

array(['-', 'Онкология: хирургия (торакальная)', 'Радиология',
       'Онкология: хирургия (рмж)', 'Лучевая диагностика: узд',
       'Хирург онколог', 'Онкология: лекарственное лечение', 'Врач',
       'Лучевая терапия (радиотерапия)', 'Химиотерапевт',
       'Лучевая диагностика: рентгенология',
       'Клиническая лабораторная диагностика', 'Хирургия',
       'Онкология: хирургия (абдоминальная)', 'Лучевая диагностика: мрт',
       'Онкология: хирургия (огш)', 'Патологическая анатомия',
       'Патоморфология', 'Акушерство и гинекология', 'Онкология',
       'Онкология: хирургия (урология)', 'Гематология',
       'Анестезиология и реанимация', 'Дерматовенерология',
       'Психотерапия', 'Онкогинеколог',
       'Лучевая диагностика: радионуклидная диагностика', 'Педиатрия',
       'Онкология: хирургия (кости и мягкие ткани)',
       'Клиническая фармакология', 'Детская хирургия',
       'Лучевая диагностика: интервенционная радиология',
       'Лучевая диагностика', 'Колопроктология

In [301]:
# Вывод специализаций зрителей, обозначивших себя как "Исследователь"

data_group.query('profession == "Исследователь"')['specialization'].unique()

array(['-', 'Онкология: хирургия (огш)',
       'Онкология: лекарственное лечение', 'Генетика',
       'Детская онкология', 'Химиотерапевт', 'Патологическая анатомия',
       'Лучевая диагностика: радионуклидная диагностика', 'Вирусология',
       'Лабораторная генетика', 'Клиническая фармакология',
       'Клиническая лабораторная диагностика', 'Хирургия',
       'Акушерство и гинекология', 'Онкология: хирургия (рмж)',
       'Гематология', 'Онкология: хирургия (кости и мягкие ткани)'],
      dtype=object)

In [302]:
# Вывод специализаций зрителей, обозначивших себя как "Онколог"

data_group.query('profession == "Онколог"')['specialization'].unique()

array(['-', 'Онкология', 'Химиотерапевт', 'Гинекология, узд'],
      dtype=object)

In [303]:
# Вывод профессий зрителей, обозначивших свою специализацию как "Онкология"

data_group.query('specialization == "Онкология"')['profession'].unique()

array(['Химиотерапевт', 'Зам.гл.вр.по хир.помощи', 'Онколог', 'Врач',
       'Руководящее звено клиники',
       'Зав. отделением противоопухолевой терапии', 'Студент',
       'Онколог-гинеколог', 'Фармаколог', 'Зав. отделением химиотерапии',
       'Зав. отделением'], dtype=object)

#### Выводы


Анализ профессий и специальностей зрителей трансляций в текущем исследовании затруднён. Главные трудности заключаются в следующем: 


* Излишняя вариабельность профессий и специализаций. Вероятно, данные вносились зрителем путём самостоятельного набора, а не выбора из предложенных вариатов;


* Неоднозначность внесённых данных. Существует путаница с тем, что является профессией человека, а что его специализацией;


* Наличие таких общих понятий/профессий/специализаций, как "Врач", "Исследователь", "Представитель медицинской компании" и др.; 


* Одновременное наличие в датасете детального описания и обобщённого описания, например: 'лучевая диагностика: МРТ', 'лучевая диагностика: УЗД', 'лучевая диагностика: радионуклидная диагностика' и 'лучевая диагностика';


* Возможность не указать свою профессию и/или специализацию;


* Неоднозначность идентификации отдельного зрителя. Один зритель может иметь (и указать) более одного o_id, barcode, или не указать никаких данных о себе, или иметь o_id, barcode, но не иметь user_id.Ъ


#### Результаты исследования данных в текущем их виде.


Профессии зрителей докладов в порядке убывания их количества (первые пять строчек): "Врач", "Ординатор - Аспирант", "Медперсонал", "Представитель медицинской компании", "Исследователь".


Специализации зрителей докладов в порядке убывания их количества (первые пять строчек): "Онкология: лекарственное лечение", "Хирург онколог", "Химиотерапевт", "Онкология: хирургия (абдоминальная)", "Лучевая терапия (радиотерапия)".



In [304]:
# Формирование датасета с группировкой по названию доклада
# Включены только доклады, просмотренные полностью
# Сравнение докладов по числу просмотров
# (доклады объединены с последующими дискуссиями по теме доклада)

data_report = data_group.query('duration_part_total_normal == 100')

data_report = data_report.groupby('report_name', as_index=False).agg(
                            speaker_name=('speaker_name', 'first'),
                            organization=('organization', 'first'),
                            event=('event', 'first'),
                            city_event=('city_event', 'first'),
                            date=('date', 'first'),
                            potok=('potok', 'first'),
                            profession_cnt=('profession', 'nunique'),
                            specialization_cnt=('specialization', 'nunique'),
                            duration_sec_event=('duration_sec_event', 'first'),
                            country_cnt=('country', 'nunique'),
                            city_user_cnt=('user_city', 'nunique'),
                            views_cnt=('user_id', 'count'),
                            users_cnt=('user_id', 'nunique'),
                            o_id_cnt=('o_id', 'nunique'),
                            barcode_cnt=('barcode', 'nunique')).sort_values(by='views_cnt',
                                                                              ascending=False)
data_report.head(10)


,report_name,speaker_name,organization,event,city_event,date,potok,profession_cnt,specialization_cnt,duration_sec_event,country_cnt,city_user_cnt,views_cnt,users_cnt,o_id_cnt,barcode_cnt
1176,Тема 800,Аскалепова Эльвира Кирилловна,no_organization,Мероприятие 12,Санкт-Петербург,2023-04-28,4.0,18,42,3600.0,10,79,2100,328,328,328
1177,Тема 801,Недобоева Жанна Маратовна,no_organization,Мероприятие 12,Санкт-Петербург,2023-04-28,4.0,18,43,3600.0,10,82,1935,359,359,359
770,Тема 388,Инжакова Юлия Артуровна,no_organization,Мероприятие 11,Санкт-Петербург,2022-07-04,4.0,10,25,300.0,6,50,1370,127,127,127
1124,Тема 753,Васючкова Оксана Руслановна,no_organization,Мероприятие 12,Москва,2023-04-28,1.0,19,47,600.0,10,100,1214,568,572,572
1192,Тема 815,Бальестерос Иван Ефимович,no_organization,Мероприятие 12,Москва,2023-04-28,5.0,17,38,900.0,8,83,1168,365,366,366
1121,Тема 750,Васючкова Оксана Руслановна,no_organization,Мероприятие 12,Санкт-Петербург,2023-04-28,1.0,19,47,900.0,10,100,1164,552,555,555
1141,Тема 769,Полянина Наталья Ринатовна,no_organization,Мероприятие 12,Санкт-Петербург,2023-04-28,2.0,20,49,300.0,11,111,1159,545,548,548
1140,Тема 768,Милешкина Жанна Артемовна,no_organization,Мероприятие 12,Москва,2023-04-28,2.0,20,44,300.0,11,110,1133,528,531,531
1139,Тема 767,Дзевалтовская Дарья Владимировна,no_organization,Мероприятие 12,Екатеринбург,2023-04-28,2.0,20,44,300.0,11,110,1122,523,526,526
1138,Тема 766,Элькина Римма Геннадьевна,no_organization,Мероприятие 12,Москва,2023-04-28,2.0,20,44,300.0,11,110,1114,519,522,522


In [305]:
# Формирование датасета с группировкой по названию доклада
# Включены только доклады, просмотренные полностью
# Сравнение докладов по числу просмотров
# (только доклады без последующих дискуссий по теме доклада)
# Датасет data_group_no_discuss сформирован в начале текущего раздела

data_report_no_discuss = data_group_no_discuss.query('duration_part_total_normal == 100')

data_report_no_discuss = data_report_no_discuss.groupby('report_name', as_index=False).agg(
                            speaker_name=('speaker_name', 'first'),
                            organization=('organization', 'first'),
                            event=('event', 'first'),
                            city_event=('city_event', 'first'),
                            date=('date', 'first'),
                            potok=('potok', 'first'),
                            profession_cnt=('profession', 'nunique'),
                            specialization_cnt=('specialization', 'nunique'),
                            duration_sec_event=('duration_sec_event', 'first'),
                            country_cnt=('country', 'nunique'),
                            city_user_cnt=('user_city', 'nunique'),
                            views_cnt=('user_id', 'count'),
                            users_cnt=('user_id', 'nunique'),
                            o_id_cnt=('o_id', 'nunique'),
                            barcode_cnt=('barcode', 'nunique')).sort_values(by='views_cnt',
                                                                              ascending=False)

data_report_no_discuss.head(10)

,report_name,speaker_name,organization,event,city_event,date,potok,profession_cnt,specialization_cnt,duration_sec_event,country_cnt,city_user_cnt,views_cnt,users_cnt,o_id_cnt,barcode_cnt
41,Тема 50,Джин Геннадий Степанович,-,Мероприятие 5,-,2022-07-21,1.0,9,18,600.0,3,41,86,86,86,86
37,Тема 47,Боглаева Татьяна Глебовна,-,Мероприятие 5,-,2022-07-21,1.0,10,17,1200.0,4,39,85,85,85,85
42,Тема 51,Самедов Валерий Дамирович,-,Мероприятие 5,-,2022-07-21,1.0,10,20,600.0,4,41,83,82,82,82
39,Тема 49,Бабочкина Елизавета Сергеевна,-,Мероприятие 5,-,2022-07-21,1.0,8,18,900.0,4,36,78,78,78,78
38,Тема 48,Влодзимирская Наталья Тимуровна,-,Мероприятие 5,-,2022-07-21,1.0,8,17,900.0,3,37,71,71,71,71
43,Тема 52,Растегаев Роман Валентинович,-,Мероприятие 5,-,2022-07-21,1.0,9,18,600.0,5,36,71,71,71,71
44,Тема 53,Монина Галина Андреевна,-,Мероприятие 5,-,2022-07-21,1.0,7,19,1200.0,3,31,60,60,60,60
18,Тема 3,Ямаева Анна Даниловна,-,Мероприятие 1,-,2022-03-14,1.0,6,12,1800.0,2,13,26,26,26,26
15,Тема 27,Дубровкин Александр Олегович,-,Мероприятие 4,-,2023-04-15,1.0,4,10,1200.0,3,13,24,24,24,24
40,Тема 5,Рыбачев Владислав Артурович,-,Мероприятие 1,-,2022-03-14,1.0,6,13,1800.0,3,14,23,23,23,23


In [306]:
# Рассчитаю, какая часть дискуссий по теме доклада относительно их общего количества 
# просматривается полностью
# Сравню с таким же расчётом по непосредственно докладам

# Выделение в отдельный датасет строк только с дискуссиями по теме докладов

data_only_discuss = data_group.query('organization == "no_organization"')

# Расчёт долей

print('Полностью просматривается ',
      round(len(data_group_no_discuss.query('duration_part_total_normal == 100'))\
      /len(data_group_no_discuss)*100, 1), '% докладов.')
     
print()
    
print('Полностью просматривается ',
      round(len(data_only_discuss.query('duration_part_total_normal == 100'))\
      /len(data_only_discuss)*100, 1), '% дискуссий по теме доклада.')

print()
    
print('Полностью просматривается ',
      round(len(data_group.query('duration_part_total_normal == 100'))\
      /len(data_group)*100, 1), '% мероприятий.')


Полностью просматривается  55.0 % докладов.

Полностью просматривается  77.7 % дискуссий по теме доклада.

Полностью просматривается  77.5 % мероприятий.


In [307]:
# Данные о длительности докладов

data_group_no_discuss['duration_sec_event'].describe()

count     1958.000000
mean      2157.763023
std       3569.546790
min        600.000000
25%        900.000000
50%       1200.000000
75%       1500.000000
max      18000.000000
Name: duration_sec_event, dtype: float64

In [308]:
# Данные о длительности дискуссий

data_only_discuss['duration_sec_event'].describe()

count    221469.000000
mean        939.064926
std        1684.554677
min         120.000000
25%         300.000000
50%         900.000000
75%         900.000000
max       32400.000000
Name: duration_sec_event, dtype: float64

#### Выводы


Ранее в текущем исследовании было принято, что дискуссия, следующая за докладом посвящена теме доклада, а значит тема дискуссии совпадает с темой доклада, имя докладчика то же. В текущих данных дискуссии выделены в отдельные мероприятия с отдельным временем начала и окончания. Таким образом, зритель мог посмотреть дискуссию, но не доклад, доклад, но не дискуссию посмотреть оба этих мероприятия, или не смотреть ни одно из них. С учётом этого, существует возможность анализировать просмотры докладов с последующими дискуссиями и без них. Это же разделение затрудняет анализ данных.


Первая десятка самых популярных докладов выглядит так:

1. 21.07.2022. Поток 1. Тема 50. Джин Геннадий Степанович. 
 
 
2. 21.07.2022. Поток 1. Тема 47. Боглаева Татьяна Глебовна. 


3. 21.07.2022. Поток 1. Тема 51. Самедов Валерий Дамирович.


4. 21.07.2022. Поток 1. Тема 49. Бабочкина Елизавета Сергеевна.


5. 21.07.2022. Поток 1. Тема 48. Влодзимирская Наталья Тимуровна. 


6. 21.07.2022. Поток 1. Тема 52. Растегаев Роман Валентинович.


7. 21.07.2022. Поток 1. Тема 53. Монина Галина Андреевна.


8. 14.03.2022. Поток 1. Тема 3. Ямаева Анна Даниловна.


9. 15.04.2023. Поток 1. Тема 27. Дубровкин Александр Олегович.


10. 14.03.2022. Поток 1. Тема 5. Рыбачев Владислав Артурович.

In [309]:
# Формирование датасета с группировкой по имени докладчика
# Включены только доклады, просмотренные полностью
# Сравнение докладов по числу просмотров
# (только доклады без последующих дискуссий по теме доклада)

data_report_no_discuss = data_group_no_discuss.query('duration_part_total_normal == 100')

data_name_no_discuss = data_report_no_discuss.groupby('speaker_name', as_index=False).agg(
                            report_cnt=('report_name', 'nunique'),
                            organization=('organization', 'first'),
                            event_cnt=('event', 'nunique'),
                            city_event=('city_event', 'first'),
                            potok_cnt=('potok', 'nunique'),
                            profession_cnt=('profession', 'nunique'),
                            specialization_cnt=('specialization', 'nunique'),
                            country_cnt=('country', 'nunique'),
                            city_user_cnt=('user_city', 'nunique'),
                            views_cnt=('user_id', 'count'),
                            users_cnt=('user_id', 'nunique'),
                            o_id_cnt=('o_id', 'nunique'),
                            barcode_cnt=('barcode', 'nunique')).sort_values(by='views_cnt',
                                                                              ascending=False)
data_name_no_discuss.head(10)


,speaker_name,report_cnt,organization,event_cnt,city_event,potok_cnt,profession_cnt,specialization_cnt,country_cnt,city_user_cnt,views_cnt,users_cnt,o_id_cnt,barcode_cnt
14,Джин Геннадий Степанович,2,-,1,-,1,9,18,3,41,87,86,86,86
8,Боглаева Татьяна Глебовна,1,-,1,-,1,10,17,4,39,85,85,85,85
33,Самедов Валерий Дамирович,1,-,1,-,1,10,20,4,41,83,82,82,82
2,Бабочкина Елизавета Сергеевна,1,-,1,-,1,8,18,4,36,78,78,78,78
13,Влодзимирская Наталья Тимуровна,1,-,1,-,1,8,17,3,37,71,71,71,71
30,Растегаев Роман Валентинович,1,-,1,-,1,9,18,5,36,71,71,71,71
28,Монина Галина Андреевна,1,-,1,-,1,7,19,3,31,60,60,60,60
16,Закржевский Ильдар Дмитриевич,3,-,1,-,1,4,12,4,18,60,33,33,33
19,Клепова Маргарита Яковлевна,2,-,1,-,1,5,11,4,15,32,26,26,26
43,Ямаева Анна Даниловна,1,-,1,-,1,6,12,2,13,26,26,26,26


#### Выводы


Ниже представлены десять имён докладчиков, чьи доклады посмотрели наибольшее количество раз. Не включены в отдельные мероприятия дискусси по теме доклада.


1. Джин Геннадий Степанович.


2. Боглаева Татьяна Глебовна


3. Самедов Валерий Дамирович.


4. Бабочкина Елизавета Сергеевна.


5. Влодзимирская Наталья Тимуровна.


6. Растегаев Роман Валентинович.


7. Монина Галина Андреевна.


8. Закржевский Ильдар Дмитриевич.


9. Клепова Маргарита Яковлевна.


10. Ямаева Анна Даниловна.


По-видимому, существует следующая зависимость: чем популярнее докладчик, тем большему числу  разных профессий и специализаций зрителей интересен он. Т.е., наибольшее число уникальных профессий и специализаций зрителей у более популярных докладчиков.


In [310]:
# Формирование датасета с группировкой по наименованию сессий
# Включены все доклады, в том числе просмотренные не полностью
# Сравнение сессий по числу просмотров


data_sessions_views = data_full_group.groupby('session', as_index=False).agg(
                            report_cnt=('report_name', 'nunique'),
                            speaker_cnt=('speaker_name', 'nunique'),
                            profession_cnt=('profession', 'nunique'),
                            specialization_cnt=('specialization', 'nunique'),
                            country_cnt=('country', 'nunique'),
                            city_user_cnt=('user_city', 'nunique'),
                            views_cnt=('user_id', 'count'),
                            users_cnt=('user_id', 'nunique'),
                            o_id_cnt=('o_id', 'nunique'),
                            barcode_cnt=('barcode', 'nunique')).sort_values(by='views_cnt',
                                                                              ascending=False)
data_sessions_views.head(10)


,session,report_cnt,speaker_cnt,profession_cnt,specialization_cnt,country_cnt,city_user_cnt,views_cnt,users_cnt,o_id_cnt,barcode_cnt
23,Сессия 119,12,11,20,49,11,111,13524,547,550,550
22,Сессия 118,13,13,20,44,11,104,11154,474,476,476
27,Сессия 122,11,11,18,43,8,89,10558,443,445,445
30,Сессия 125,2,11,18,45,10,92,8730,394,395,395
39,Сессия 133,15,17,17,39,7,70,8030,273,273,273
32,Сессия 127,12,12,17,36,8,80,7680,332,332,332
178,Сессия 51,25,27,13,32,7,78,6772,297,297,297
29,Сессия 124,14,14,17,41,10,75,6554,298,298,298
177,Сессия 50,43,43,14,27,8,71,6484,299,299,299
21,Сессия 117,14,14,19,41,11,85,5578,333,334,334


In [311]:
# Подсчёт числа уникальных наименований сессий

data_sessions_views['session'].nunique()

230

#### Выводы


В текущем исследовании рассматриваются данные о просмотрах трансляций 230-ти сессий. В десятку наименований сессий с наибольшим числом просмотров входят следующие сессии (перечислены в порядке убывания числа просмотров):


1. Сессия 119.

На эту тему представлено 12 докладов. Около 550 зрителей.


2. Сессия 118.

На эту тему представлено 13 докладов. Около 480 зрителей.


3. Сессия 122.

На эту тему представлено 11 докладов. Около 440 зрителей.


4. Сессия 125.

На эту тему представлено 2 доклада. Около 400 зрителей.


5. Сессия 133.

На эту тему представлено 15 докладов. Около 270 зрителей.


6. Сессия 127.

На эту тему представлено 12 докладов. Около 330 зрителей.


7. Сессия 51.

На эту тему представлено 25 докладов. Около 300 зрителей.


8. Сессия 124.

На эту тему представлено 14 доклада. Около 300 зрителей.


9. Сессия 50.

На эту тему представлено 43 доклада. Около 300 зрителей.


10. Сессия 117.

На эту тему представлено 14 докладов. Около 330 зрителей.



In [312]:
# Формирование датасета с группировкой по коду классификатора
# Включены все доклады, в том числе просмотренные не полностью
# Сравнение по числу просмотров


data_class_views = data_full_group.groupby('class', as_index=False).agg(
                            report_cnt=('report_name', 'nunique'),
                            speaker_cnt=('speaker_name', 'nunique'),
                            profession_cnt=('profession', 'nunique'),
                            specialization_cnt=('specialization', 'nunique'),
                            country_cnt=('country', 'nunique'),
                            city_user_cnt=('user_city', 'nunique'),
                            views_cnt=('user_id', 'count'),
                            users_cnt=('user_id', 'nunique'),
                            o_id_cnt=('o_id', 'nunique'),
                            barcode_cnt=('barcode', 'nunique')).sort_values(by='views_cnt',
                                                                              ascending=False)
data_class_views


,class,report_cnt,speaker_cnt,profession_cnt,specialization_cnt,country_cnt,city_user_cnt,views_cnt,users_cnt,o_id_cnt,barcode_cnt
2,Основное,1340,936,32,77,14,202,219672,2513,3003,3781
1,Нозологии,637,488,29,72,13,189,98319,2015,2391,2961
0,-,87,78,19,45,12,99,3362,391,432,432


#### Выводы


В текущем исследовании рассматриваются данные о просмотрах трансляций докладов, ссответствующих двум кодам классификатора. Большее число докладов, докладчиков, просмотров и зрителей у докладов, соответстующих коду классификатора "Основное": 1340 докладов, 936 докладчиков, более 2500 зрителей. Менее популярны доклады, соответствующие коду классификатора "Нозологии": 637 докладов, 488 докладчиков, около 2000 зрителей. 87 докладов не имеют данных о соответствии коду классификатора.


In [313]:
# Формирование датасета с группировкой по направлениям докладов
# Включены все доклады, в том числе просмотренные не полностью
# Сравнение по числу просмотров


data_direction_views = data_full_group.groupby('direction', as_index=False).agg(
                            report_cnt=('report_name', 'nunique'),
                            speaker_cnt=('speaker_name', 'nunique'),
                            profession_cnt=('profession', 'nunique'),
                            specialization_cnt=('specialization', 'nunique'),
                            country_cnt=('country', 'nunique'),
                            city_user_cnt=('user_city', 'nunique'),
                            views_cnt=('user_id', 'count'),
                            users_cnt=('user_id', 'nunique'),
                            o_id_cnt=('o_id', 'nunique'),
                            barcode_cnt=('barcode', 'nunique')).sort_values(by='views_cnt',
                                                                              ascending=False)
data_direction_views.head(10)


,direction,report_cnt,speaker_cnt,profession_cnt,specialization_cnt,country_cnt,city_user_cnt,views_cnt,users_cnt,o_id_cnt,barcode_cnt
1,Направление 1,506,434,28,69,12,176,61423,1717,2026,2376
34,Направление 7,363,346,26,65,13,175,43604,1585,1835,2093
23,Направление 3,435,314,25,62,11,174,38333,1616,1871,2176
12,Направление 2,266,222,24,65,13,169,37021,1403,1621,1877
8,Направление 16,104,84,18,54,10,126,16711,827,910,935
31,Направление 4,241,225,23,55,12,140,14788,959,1085,1147
5,Направление 13,77,77,17,48,12,116,10386,630,680,680
15,Направление 22,83,75,16,45,9,108,7003,537,600,600
33,Направление 6,99,87,18,47,8,101,6995,520,549,553
6,Направление 14,78,75,18,46,10,116,6901,656,712,746


In [314]:
# Подсчёт числа уникальных направлений докладов

data_full_group['direction'].nunique()

37

#### Выводы


В текущем исследовании рассматриваются данные о просмотрах трансляций 37-ми направлений. В десятку направлений с наибольшим числом просмотров входят следующие направления докладов (перечислены в порядке убывания числа просмотров):


1. Направление 1.

По этому направлению представлено 506 докладов. Около 1700 зрителей.


2. Направление 7.

По этому направлению представлено 363 доклада. Около 1600 зрителей.


3. Направление 3.

По этому направлению представлено 435 докладов. Около 1600 зрителей.


4. Направление 2.

По этому направлению представлено 266 докладов. Около 1400 зрителей.


5. Направление 16.

По этому направлению представлено 104 доклада. Около 830 зрителей.


6. Направление 4.

По этому направлению представлено 241 доклад. Около 960 зрителей.


7. Направление 13.

По этому направлению представлено 77 докладов. Около 630 зрителей.


8. Направление 22.

По этому направлению представлено 83 доклада. Около 540 зрителей.


9. Направление 6.

По этому направлению представлено 99 докладов. Около 520 зрителей.


10. Направление 14.

По этому направлению представлено 78 докладов. Около 660 зрителей.



Большое число просмотров и зрителей докладов определённого направления сильно зависит от количества докладов по этому направлению. Таким образом, направление доклада входит в верхние строчки по популярности, в том числе, за счёт большого количества докладов этого направления.


Для определения наиболее востребованных (интересных зрителю) направлений докладов необходимо выяснить: какие направления докладов набирают много просмотров и зрителей при небольшом количестве таких докладов. Для этого нужно ввести относительную величину, отражающую востребованность направления у зрителей. 


Т.к., в имеющемся датасете существует несколько метрик, отражающих зрительскую активность: число просмотров, число уникальных user_id, число уникальных o_id, число уникальных barcode, в дальнейших вычислениях я буду параллельно использовать две из этих метрик: число просмотров, число уникальных user_id. Получив два списка направлений и объединив получившиеся списки, я получу один список наиболее востребованных направлений докладов.


In [315]:
# Ограничение рассматриваемых направлений

demand_directions = data_full_group\
                    .query('duration_part_total_normal == 100')

demand_directions = demand_directions.query('direction != "-"')

In [316]:
# Формирование датасета с группировкой по направлениям докладов

demand_directions = demand_directions.groupby('direction', as_index=False).agg(
                            report_cnt=('report_name', 'nunique'),
                            speaker_cnt=('speaker_name', 'nunique'),
                            profession_cnt=('profession', 'nunique'),
                            specialization_cnt=('specialization', 'nunique'),
                            country_cnt=('country', 'nunique'),
                            city_user_cnt=('user_city', 'nunique'),
                            views_cnt=('user_id', 'count'),
                            users_cnt=('user_id', 'nunique'),
                            o_id_cnt=('o_id', 'nunique'),
                            barcode_cnt=('barcode', 'nunique'))

demand_directions.sample(1)

,direction,report_cnt,speaker_cnt,profession_cnt,specialization_cnt,country_cnt,city_user_cnt,views_cnt,users_cnt,o_id_cnt,barcode_cnt
3,Направление 12,17,14,14,36,6,78,1905,278,295,295


In [317]:
# Введение столбцов с относительными величинами, отражающими зрительский интерес

demand_directions['demand_views'] = demand_directions['report_cnt']\
                                    /demand_directions['views_cnt']

demand_directions['demand_users'] = demand_directions['report_cnt']\
                                    /demand_directions['users_cnt']

demand_directions.sample(1)

,direction,report_cnt,speaker_cnt,profession_cnt,specialization_cnt,country_cnt,city_user_cnt,views_cnt,users_cnt,o_id_cnt,barcode_cnt,demand_views,demand_users
4,Направление 13,35,36,17,47,12,111,8201,583,621,621,0.004268,0.060034


In [318]:
# Формирование и вывод списка направлений, наиболее интересных с т.з. просмотров

demand_directions_views = demand_directions.sort_values(by='demand_views',
                                                        ascending=False)

demand_directions_views.head(10)

,direction,report_cnt,speaker_cnt,profession_cnt,specialization_cnt,country_cnt,city_user_cnt,views_cnt,users_cnt,o_id_cnt,barcode_cnt,demand_views,demand_users
25,Направление 32,35,38,10,19,4,22,502,74,76,76,0.069721,0.472973
21,Направление 29,19,16,10,19,4,25,274,52,53,53,0.069343,0.365385
29,Направление 36,6,6,8,18,5,22,160,44,44,44,0.037500,0.136364
34,Направление 8,64,85,17,37,6,73,1961,294,304,315,0.032636,0.217687
24,Направление 31,7,7,9,19,5,21,340,43,43,43,0.020588,0.162791
26,Направление 33,18,16,11,29,4,27,890,140,164,164,0.020225,0.128571
20,Направление 28,68,55,12,33,9,71,3535,237,261,261,0.019236,0.286920
10,Направление 19,32,32,13,33,9,69,1824,228,232,232,0.017544,0.140351
31,Направление 5,14,14,10,31,4,49,891,158,159,160,0.015713,0.088608
1,Направление 10,45,47,20,41,7,105,2937,422,454,454,0.015322,0.106635


In [319]:
# Формирование и вывод списка направлений, наиболее интересных с т.з. числа зрителей

demand_directions_users = demand_directions.sort_values(by='demand_users',
                                                        ascending=False)

demand_directions_users.head(10)

,direction,report_cnt,speaker_cnt,profession_cnt,specialization_cnt,country_cnt,city_user_cnt,views_cnt,users_cnt,o_id_cnt,barcode_cnt,demand_views,demand_users
25,Направление 32,35,38,10,19,4,22,502,74,76,76,0.069721,0.472973
21,Направление 29,19,16,10,19,4,25,274,52,53,53,0.069343,0.365385
20,Направление 28,68,55,12,33,9,71,3535,237,261,261,0.019236,0.286920
34,Направление 8,64,85,17,37,6,73,1961,294,304,315,0.032636,0.217687
22,Направление 3,268,183,25,59,11,166,30384,1488,1712,1954,0.008820,0.180108
0,Направление 1,285,265,26,67,12,169,48438,1598,1883,2175,0.005884,0.178348
24,Направление 31,7,7,9,19,5,21,340,43,43,43,0.020588,0.162791
10,Направление 19,32,32,13,33,9,69,1824,228,232,232,0.017544,0.140351
28,Направление 35,59,48,20,44,8,91,4527,424,427,427,0.013033,0.139151
19,Направление 27,23,17,11,29,7,61,1562,168,176,176,0.014725,0.136905


#### Выводы


По результатам исследования, формирую список десяти наиболее востребованных направлений докладов.


1. Направление 32.


2. Направление 29.


3. Направление 28.


4. Направление 8.


5. Направление 3.


6. Направление 1.


7. Направление 31.


8. Направление 19.


9. Направление 35.


10. Направление 10.	


Стоит отметить, что в списке востребованных направлений докладов присутсвуют два направления из популярных (с большим числом докладов). Эти направления: Направление 1, Направление 3.


## Выделение групп зрителей

#### Краткое описание операций выделении групп зрителей


Выделение группы активных зрителей.

Выделение групп зрителей наиболее популярных направлений докладов.

Выделение групп зрителей наиболее востребованных направлений докладов.

Выводы с перечнем списков выделенных групп зрителей.


### Выделение активных зрителей

Активность зрителей здесь: большое число просмотров, доклады просматриваются полностью. 

In [320]:
# Группировка по user_id
# Выделение групп зрителей с максимальной активностью:
#     большим количеством просмотренных докладов, большим количеством просмотров 

data_top_users = data_group\
                    .query('duration_part_total_normal == 100 & user_id != "no_user_id"')

data_top_users = data_top_users.groupby('user_id', as_index=False).agg(
                            report_cnt=('report_name', 'nunique'),
                            views_cnt=('report_name', 'count'),
                            o_id_cnt=('o_id', 'nunique'),
                            barcode_cnt=('barcode', 'nunique')).sort_values(by='report_cnt',
                                                                              ascending=False)
data_top_users.head(10)

,user_id,report_cnt,views_cnt,o_id_cnt,barcode_cnt
110,948.0,375,637,2,3
257,1643.0,342,592,1,1
1056,9739.0,336,609,3,4
1404,13859.0,327,564,2,3
300,1837.0,309,539,3,4
915,7322.0,303,538,2,3
599,3949.0,286,503,2,3
399,2664.0,277,491,2,3
320,1920.0,269,637,2,3
98,896.0,267,473,2,3


In [321]:
# Процент зрителей, просмотревших хотя бы один доклад полностью
# В расчёте все зрители из исходных данных

round(data_top_users['user_id'].nunique()/data_users['user_id'].nunique()*100, 1)

79.0

In [322]:
# Процент зрителей, просмотревших хотя бы один доклад полностью
# В расчёте зрители после очистки данных

round(data_top_users['user_id'].nunique()/data_group['user_id'].nunique()*100, 1)

92.3

In [323]:
# Вывод количеств просмотренных зрителями докладов

data_top_users['report_cnt'].describe()

count    2521.000000
mean       38.883776
std        49.865633
min         1.000000
25%         6.000000
50%        17.000000
75%        53.000000
max       375.000000
Name: report_cnt, dtype: float64

In [324]:
# Вывод количеств просмотров зрителей

data_top_users['views_cnt'].describe()

count    2521.000000
mean       68.343911
std        91.358879
min         1.000000
25%         8.000000
50%        29.000000
75%        94.000000
max       637.000000
Name: views_cnt, dtype: float64

In [325]:
# Выделение только самых активных зрителей

data_top_users = data_top_users.query('report_cnt >= 53 & views_cnt >= 94')

In [326]:
# Контроль выполнения операций

data_top_users.head(2)

,user_id,report_cnt,views_cnt,o_id_cnt,barcode_cnt
110,948.0,375,637,2,3
257,1643.0,342,592,1,1


In [327]:
# Выделение зрителей, активно раздающих свои данные для параллельных просмотров

donors = data_full_group.query('duration_part_total > 1000')['user_id'].unique()

In [328]:
# Удаление из датасета зрителей, активно раздающих свои данные для параллельных просмотров

data_top_users = data_top_users.query('user_id not in @donors')

In [329]:
# Объединение с датасетом с прочими данными зрителей

data_top_users = data_top_users.merge(data_users, on='user_id', how='left')

In [330]:
# Группировка по user_id

data_top_users = data_top_users.groupby('user_id', as_index=False).agg(
                            report_cnt=('report_cnt', 'first'),
                            views_cnt=('views_cnt', 'first'),
                            o_id_cnt=('o_id_cnt', 'first'),
                            barcode_cnt=('barcode_cnt', 'first'),
                            date_cnt=('date', 'nunique'),
                            event_cnt=('event', 'nunique'),
                            device=('device', 'first'),
                            country=('country', 'first'),
                            region=('region', 'first'),
                            user_city=('user_city', 'first'),
                            profession=('profession', 'first'),
                            specialization=('specialization', 'first'))\
                        .sort_values(by='report_cnt', ascending=False)


data_top_users.head(10)


,user_id,report_cnt,views_cnt,o_id_cnt,barcode_cnt,date_cnt,event_cnt,device,country,region,user_city,profession,specialization
42,948.0,375,637,2,3,13,2,Десктоп,Россия,Ленинградская обл.,Санкт-Петербург,Исследователь,Химиотерапевт
260,9739.0,336,609,3,4,12,3,Десктоп,Россия,Ленинградская обл.,Санкт-Петербург,Врач,Акушерство и гинекология
105,2664.0,277,491,2,3,10,2,Десктоп,Россия,Ленинградская обл.,Санкт-Петербург,Врач,-
38,896.0,267,473,2,3,9,2,Мобильное,Россия,Ленинградская обл.,Санкт-Петербург,Врач,-
221,6700.0,260,449,2,3,10,2,Десктоп,Россия,Ленинградская обл.,Санкт-Петербург,Врач,-
261,9750.0,256,451,3,4,12,3,Десктоп,-,-,-,Руководящее звено клиники,Акушерство и гинекология
25,787.0,251,429,5,6,14,5,Десктоп,Россия,Ленинградская обл.,Санкт-Петербург,Врач,-
195,5456.0,249,422,2,3,10,2,Десктоп,Россия,Краснодарский край,Краснодар,Онколог,Химиотерапевт
366,15184.0,247,409,2,3,13,2,Десктоп,Россия,Коми,Сыктывкар,Врач,Радиология
435,20013.0,246,436,1,1,5,1,Десктоп,Россия,Ленинградская обл.,Санкт-Петербург,Медперсонал,Сестринское дело


In [331]:
# Вывод городов наиболее активных зрителей

data_top_users['user_city'].value_counts().head(16)

Санкт-Петербург    170
Москва              57
Ростов-на-Дону      15
-                   13
Архангельск          9
Воронеж              9
Краснодар            9
Ижевск               9
Минск                8
Псков                8
Белгород             7
Чебоксары            6
Петрозаводск         6
Нижний Новгород      6
Екатеринбург         5
Хабаровск            5
Name: user_city, dtype: int64

In [332]:
# Вывод регионов наиболее активных зрителей

data_top_users['region'].value_counts().head(16)

Ленинградская обл.    172
Московская обл.        63
-                      25
Ростовская обл.        18
Краснодарский край     12
Псковская обл.          9
Воронежская обл.        9
Удмуртия                9
Архангельская обл.      9
Минская обл.            8
Белгородская обл.       7
Хабаровский край        6
Республика Карелия      6
Самарская обл.          6
Нижегородская обл.      6
Чувашия                 6
Name: region, dtype: int64

In [333]:
# Вывод стран наиболее активных зрителей

data_top_users['country'].value_counts().head()

Россия        450
Беларусь       14
-              11
Кыргызстан      4
Украина         2
Name: country, dtype: int64

In [334]:
# Вывод профессий наиболее активных зрителей

data_top_users['profession'].value_counts().head(11)

Врач                                  332
Ординатор - аспирант                   36
-                                      25
Руководящее звено клиники              19
Исследователь                          18
Онколог                                18
Медперсонал                            13
Представитель медицинской компании     10
Студент                                 4
Зав. отделением химиотерапии            4
Химиотерапевт                           4
Name: profession, dtype: int64

In [335]:
# Вывод специализаций наиболее активных зрителей

data_top_users['specialization'].value_counts().head(11)

-                                      88
Хирург онколог                         59
Химиотерапевт                          58
Онкология: лекарственное лечение       57
Лучевая терапия (радиотерапия)         27
Онкология: хирургия (абдоминальная)    20
Онкология                              20
Патологическая анатомия                19
Онкология: хирургия (рмж)              18
Радиология                             15
Патоморфология                         12
Name: specialization, dtype: int64

In [336]:
# Число уникальных o_id у активных зрителей

data_top_users['o_id_cnt'].describe()

count    489.000000
mean       1.519427
std        0.610807
min        1.000000
25%        1.000000
50%        1.000000
75%        2.000000
max        5.000000
Name: o_id_cnt, dtype: float64

In [337]:
# Число уникальных билетов (barcode) у активных зрителей

data_top_users['barcode_cnt'].describe()

count    489.000000
mean       2.038855
std        0.936795
min        1.000000
25%        1.000000
50%        2.000000
75%        3.000000
max        6.000000
Name: barcode_cnt, dtype: float64

In [338]:
# Число дат просмотров трансляций активными зрителями

data_top_users['date_cnt'].describe()

count    489.000000
mean       6.229039
std        2.927522
min        1.000000
25%        4.000000
50%        6.000000
75%        8.000000
max       18.000000
Name: date_cnt, dtype: float64

In [339]:
# Число семинаров/конференций, посещённых активными зрителями

data_top_users['event_cnt'].describe()

count    489.000000
mean       1.580777
std        0.688100
min        1.000000
25%        1.000000
50%        1.000000
75%        2.000000
max        5.000000
Name: event_cnt, dtype: float64

In [340]:
# Процент выявленных активных зрителей
# В расчёте все зрители из исходных данных

round(data_top_users['user_id'].nunique()/data_users['user_id'].nunique()*100, 1)

15.3

In [341]:
# Список user_id, принадлежащих самым активным зрителям

top_users = data_top_users['user_id'].unique()

# Количество зрителей

len(top_users)

489

### Выделение зрителей наиболее популярных направлений докладов

Используя ранее выделенные наиболее популярные направления докладов, сформирую списки user_id зрителей, которые посмотрели хотя бы один такой доклад полностью. Такие списки user_id зрителей можно использовать для адресной рассылки анонсов, рекламы, приглашений на мероприятия и пр.  

In [342]:
# Формирование датасета из строк с полностью просмотренными докладами (включая дискуссии)

data_top_users_directions = data_full_group\
                    .query('duration_part_total_normal == 100 & user_id != "no_user_id"')

In [343]:
# Удаление из датасета зрителей, активно раздающих свои данные для параллельных просмотров

data_top_users_directions = data_top_users_directions.query('user_id not in @donors')

In [344]:
# Список зрителей докладов направления "Направление 1" 

direct_1_users = data_top_users_directions\
                    .query('direction == "Направление 1"')['user_id'].unique()

# Количество зрителей

len(direct_1_users)

1430

### Выделение зрителей наиболее востребованных направлений докладов

Используя ранее выделенные наиболее востребованные направления докладов, сформирую списки user_id зрителей, которые посмотрели хотя бы один такой доклад полностью. Такие списки user_id зрителей можно использовать для адресной рассылки анонсов, рекламы, приглашений на мероприятия и пр.  

In [345]:
# Формирование датасета из строк с полностью просмотренными докладами (включая дискуссии)

data_demand_users_directions = data_full_group\
                    .query('duration_part_total_normal == 100 & user_id != "no_user_id"')

In [346]:
# Удаление из датасета зрителей, активно раздающих свои данные для параллельных просмотров

data_demand_users_directions = data_demand_users_directions.query('user_id not in @donors')

In [347]:
# Список зрителей докладов направления "Направление 32" 

direction_32_users = data_demand_users_directions\
                            .query('direction == "Направление 32"')['user_id'].unique()

# Количество зрителей докладов

len(direction_32_users)

67

#### Выводы


При выделении группы активных зрителей докладов были приняты следующие критерии отбора: число полностью просмотренных уникальных докладов: не менее 53-ух, число просмотров: не менее 94-ёх, что соотвествует верхним 25-ти процентам строк списка зрителей. Критерии отбора могут быть пересмотрены и/или дополнены. 

Активные зрители характеризуются также тем, что половиной из них было приобретено 2 и более билета; половина таких зрителей смотрели трансляции в течение шести и более дней; посмотрели трансляции до 5-ти разных конференций.

* Список активных зрителей (размер: 489 user_id) - top_users


###### Списки зрителей популярных направлений докладов


* Список зрителей докладов направления "Направление 1" (размер: 1430 user_id) - direction_1_users


###### Списки зрителей востребованных направлений докладов


* Список зрителей докладов направления "Направление 32" (размер: 67 user_id) - direction_32_users


К числу востребованных направлений докладов относятся два направления из популярных: Направление 1 и Направление 3.


Из всех выделенных списков зрителей исключены зрители, чьи данные для просмотра трансляций были продублированы более 10-ти раз, т.е. организовывали параллельные просмотры. Такое исключение также может быть пересмотрено и/или дополнено.

## Выводы по проекту


В исходных данных трансляций докладов НМИЦ представлены данные о просмотрах около трёх тысяч двухсот зрителей. 


* Устройство промотра трансляций в большинстве случаев - это десктоп. Число просмотров с мобильных устройств также велико. Необходимо поддерживать возможность просмотра трансляций с обоих типов устройств.


##### География зрителей (в скобках указана доля зрителей):

* Россия (92,1%), Беларусь (1,6%), Казахстан (0,7%), Узбекистан  (0,7%), Кыргызстан  (0,6%), Украина  (0,3%), Азербайджан  (0,1%), Армения  (0,1%), Молдова  (0,1%), Германия  (менее 0,1%), Латвия (менее 0,1%), Таджикистан (менее 0,1%), Китай (менее 0,1%).


* Зрители из России: более 38% зрителей трансляций из Ленинградской области. В первую пятёрку регионов зрителей также входят: Московская, Ростовская, Нижегородская области и Краснодарский край. Менее всего зрителей из Карелии, Забайкальского края, Курганской области, Ямало-Ненецкого АО, Адыгеи, Камчатского края, Липецкой области, Еврейской АО, Северной Осетии - Алании, Чукотского АО.


* Зрители из России из следующих городов (топ-10): Санкт-Петербург (37,8%), Москва (18,2%), Ростов-на-Дону (2,7%), Краснодар (1,7%), Нижний Новгород (1,4%), Екатеринбург (1,4%), Воронеж (1,2%), Уфа (1,2%), Самара (1,2%), Архангельск (1,1%).	


* Около 3,5% зрителей не указали свою страну. Около 2% зрителей из России не указали свой регион.


##### Сфера деятельности зрителей


* Профессии зрителей докладов в порядке убывания их количества (топ-10): "Врач", "Ординатор - Аспирант", "Медперсонал", "Представитель медицинской компании", "Исследователь", "Руководящее звено клиники", "Студент", "Онколог", "Химиотерапевт", "Немедицинский персонал клиники".


* Специализации зрителей докладов в порядке убывания их количества (топ-10): "Онкология: лекарственное лечение", "Хирург онколог", "Химиотерапевт", "Онкология: хирургия (абдоминальная)", "Лучевая терапия (радиотерапия)", "Онкология: хирургия (рмж)", "Онкология", "Акушерство и гинекология", "Патологическая анатомия", "Лучевая диагностика: рентгенология".


* Порядка 42% зрителей делятся своими данными для просмотра трансляций. Параллельно транслируется почти 50% докладов (без учёта того, это основная часть доклада или это дискуссия после доклада). Раздача своих данных для просмотра трансляций - явление массовое, но злостных нарушителей не так много. Параллельные просмотры трансляций организуют зрители из всех стран, кроме зрителей из Германии и Таджикистана.



##### Первая десятка самых популярных докладов


1. 21.07.2022. Поток 1. Тема 50. Джин Геннадий Степанович. 
 
 
2. 21.07.2022. Поток 1. Тема 47. Боглаева Татьяна Глебовна. 


3. 21.07.2022. Поток 1. Тема 51. Самедов Валерий Дамирович.


4. 21.07.2022. Поток 1. Тема 49. Бабочкина Елизавета Сергеевна.


5. 21.07.2022. Поток 1. Тема 48. Влодзимирская Наталья Тимуровна. 


6. 21.07.2022. Поток 1. Тема 52. Растегаев Роман Валентинович.


7. 21.07.2022. Поток 1. Тема 53. Монина Галина Андреевна.


8. 14.03.2022. Поток 1. Тема 3. Ямаева Анна Даниловна.


9. 15.04.2023. Поток 1. Тема 27. Дубровкин Александр Олегович.


10. 14.03.2022. Поток 1. Тема 5. Рыбачев Владислав Артурович.


##### Первая десятка самых популярных докладчиков


1. Джин Геннадий Степанович.


2. Боглаева Татьяна Глебовна


3. Самедов Валерий Дамирович.


4. Бабочкина Елизавета Сергеевна.


5. Влодзимирская Наталья Тимуровна.


6. Растегаев Роман Валентинович.


7. Монина Галина Андреевна.


8. Закржевский Ильдар Дмитриевич.


9. Клепова Маргарита Яковлевна.


10. Ямаева Анна Даниловна.


Наибольшее число уникальных профессий и специализаций зрителей у более популярных докладчиков.


##### Тематические сессии

В текущем исследовании рассматриваются данные о просмотрах трансляций 230-ти сессий. В десятку наименований сессий с наибольшим числом просмотров входят следующие сессии (перечислены в порядке убывания числа просмотров):


1. Сессия 119.

На эту тему представлено 12 докладов. Около 550 зрителей.


2. Сессия 118.

На эту тему представлено 13 докладов. Около 480 зрителей.


3. Сессия 122.

На эту тему представлено 11 докладов. Около 440 зрителей.


4. Сессия 125.

На эту тему представлено 2 доклада. Около 400 зрителей.


5. Сессия 133.

На эту тему представлено 15 докладов. Около 270 зрителей.


6. Сессия 127.

На эту тему представлено 12 докладов. Около 330 зрителей.


7. Сессия 51.

На эту тему представлено 25 докладов. Около 300 зрителей.


8. Сессия 124.

На эту тему представлено 14 доклада. Около 300 зрителей.


9. Сессия 50.

На эту тему представлено 43 доклада. Около 300 зрителей.


10. Сессия 117.

На эту тему представлено 14 докладов. Около 330 зрителей.


##### Направления докладов с наибольшим числом просмотров. Популярные направления докладов


В текущем исследовании рассматриваются данные о просмотрах трансляций 37-ми направлений. В десятку направлений с наибольшим числом просмотров входят следующие направления докладов (перечислены в порядке убывания числа просмотров):


1. Направление 1.

По этому направлению представлено 506 докладов. Около 1700 зрителей.


2. Направление 7.

По этому направлению представлено 363 доклада. Около 1600 зрителей.


3. Направление 3.

По этому направлению представлено 435 докладов. Около 1600 зрителей.


4. Направление 2.

По этому направлению представлено 266 докладов. Около 1400 зрителей.


5. Направление 16.

По этому направлению представлено 104 доклада. Около 830 зрителей.


6. Направление 4.

По этому направлению представлено 241 доклад. Около 960 зрителей.


7. Направление 13.

По этому направлению представлено 77 докладов. Около 630 зрителей.


8. Направление 22.

По этому направлению представлено 83 доклада. Около 540 зрителей.


9. Направление 6.

По этому направлению представлено 99 докладов. Около 520 зрителей.


10. Направление 14.

По этому направлению представлено 78 докладов. Около 660 зрителей.


##### Направления докладов с наибольшим относительным числом просмотров. Востребованные направления докладов


По результатам исследования, формирую список десяти наиболее востребованных направлений докладов.


1. Направление 32.


2. Направление 29.


3. Направление 28.


4. Направление 8.


5. Направление 3.


6. Направление 1.


7. Направление 31.


8. Направление 19.


9. Направление 35.


10. Направление 10.	


Стоит отметить, что в списке востребованных направлений докладов присутсвуют два направления из популярных (с большим числом докладов): Направление 1, Направление 3.


## Рекомендации
	

В ходе анализа данных трансляций докладов НМИЦ был выявлен ряд технических сложностей, затрудняющих исследование и не позволяющих сделать более детальные выводы. Ниже приведено описание этих сложностей и предложения по их исправлению/устранению.


* Разноформатная запись дат и времён в датасетах. В части строк отсутствуют данные о времени окончания мероприятия, или вовсе отсутствуют данные и о начале, и об окончании мероприятия. Необходимо выбрать единый формат записи дат и времени, и придерживаться его при записи данных во всех датасетах. 


* В данных наименований мероприятий присутствует множество вариаций обозначения перерывов, начала и окончания мероприятий. Необходимо унифицировать способ записи таких мероприятий. Возможно, ввести единую для всех подобных мероприятий метку/обозначение о том, что это не непосредственно доклад. 


* В данных после докладов следуют дискуссии, чаще всего длительностью 5 минут. В этом исследовании такие дискуссии рассматривались как часть доклада, им приписывалось имя предшествующего докладчика. Если предположение верно, то имеет смысл в расписании мероприятий объединять доклады и дискуссии, имея в виду, что последние 5 минут доклада - это завершающая дискуссия. Если же нужно сохранить отдельную форму записи, то необходимо унифицировать способ записи таких дискуссий, т.к. встречаются обозначения, типа: "вопросы и ответы", "ответы на вопросы" и пр.


* В обозначениях потоков дискуссий встречаются записи вида: "Поток 5", "7", "10.1", "Тренинг-комната". Такая разноформатная запись также затрудняет анализ. В этом исследовании Поток 10.1 был выделен в ранее несуществующий Поток 11, а Поток 6.1 объединён с Потоком 6. Необхоимо придерживаться формы записи потока трансляций одним целым числом без текста.


* В текущем виде данные о наименовании доклада и соответствующей докладу сессии/направлении/коде классификатора разделены на отдельные датасеты. В таком виде анализ данных затруднён, т.к. при объединении часть данных дублируется, сложно вычислять число просмотров, не для всех докладов есть данные о сессии/направлении. Как следствие, численные данные о количестве просмотров являются ориентировочными и не могут быть использованы для точных расчётов. Также, замечены неясности со временем начала и окончания рубрик относительно времени начала и окончания докладов. 


Рекомендую заносить данные о соответствующей докладу сессии/направлении/коде классификатора в данные с расписаниями докладов.


* В датасете с просмотрами зрителей каждый зритель идентифицируется тремя идентификаторами: user_id, o_id, barcode. При этом один зритель может иметь (и указать) более одного o_id, barcode, или не указать никаких данных о себе, или иметь o_id, barcode, но не иметь user_id. Как следствие, при анализе существующих данных невозможно указать точное число зрителей каждого доклада, направления и др. Необходимо сократить число индентификаторов до двух: user_id как обязательная идентификация зрителя, barcode как идентификация проявленного зрителем интереса к конкретному дню конференции, или направлению/сессии и т.д. Одновременно с этим не должно быть возможности регистрации зрителя без этих двух идентификаторов.


* В датасете с просмотрами зрителей каждый зритель самообозначается вводом своей страны, региона, города, профессии, специализации. При этом зритель имеет возможность написать эти данные в произвольной форме или не указать данные вовсе. В текущем виде присутствует излишняя вариабельность профессий и специализаций. Существует путаница с тем, что является профессией человека, а что его специализацией. В данных присутствуют такие общие понятия, как: "Врач", "Исследователь", "Представитель медицинской компании" и др. Всё это вкупе значительно снижают качество исследования. Необходимо ввести обязательное внесение данных о себе для каждого зрителя путём выбора из предложенных вариантов. Зритель без внесённых данных не должен получать доступ к трансляциям. 

Точность выводов этого исследования можно улучшить путём составления словаря Врач: Специализация.
Т.к., в данные зрители часто вводили общее понятие только в один из столбцов, имеется возможность восстановить профессию из специализации, или специализацию из профессии. Например, многие зрители обозначили свою профессию, как "Врач", указав в специализации более точное определение. Работу по составлению словаря может проделать человек с соответствующими знаниями.  


* В датасете с просмотрами зрителей присутствует дополнительная информация, не проанализированная в этом исследовании. Часть из таких дополнительных данных не имеет ценности. Например: в описании базы данных написано, что: параметр "Кол-во кликов" - "Количество кликов для подтверждения просмотра трансляции. Необходим для оценки присутствия на мероприятии, если пользователь не нажал на всплывающее окно, то сессия подключения прерывается". При первоначальном анализе были выявлены, в том числе, такие строки, где зритель смотрел вебинар 4 часа подряд и не сделал ни одного клика. Имеет смысл пересмотреть набор собираемых данных и способ их сбора.



## Дашборд

In [ ]:
# Датасет для дашборда

data_dash = data.copy()

data_dash = data_dash.query('organization != "no_organization"')

In [ ]:
# Введение вспомогательных столбцов

data_dash['date_start'] = pd.to_datetime(data_dash['user_date_start'])

data_dash['date_end'] = pd.to_datetime(data_dash['user_date_end'])

data_dash.head(1)

In [ ]:
# !pip install jupyter-dash

In [ ]:
import dash
import dash_core_components as dcc
import dash_html_components as html
import plotly.express as px
import pandas as pd
from dash import Dash, dcc, html, Input, Output

In [ ]:
app = Dash(__name__)

app.layout = html.Div([
   html.H1('Дашборд по трансляциям докладов НМИЦ Минздрава России'),
   html.Br(),
   html.H3('''
       Дашборд построен на основе данных по просмотрам трансляций докладов
   '''), 
   html.Br(),
   html.H3('''
       Выбор мероприятия
   '''),
   html.Div(
   dcc.Dropdown(
        id="events-dropdown",
        options=data_dash['event'].unique(),
        value=data_dash['event'].unique(),
        clearable=True,
        multi=True
    ), style={'width': '49%', 'display': 'flex'}),
    html.Br(),
    html.H3('''
       Выбор дат
   '''),
    html.Div(
    dcc.DatePickerRange(
        id='date-picker',
        min_date_allowed=data_dash['date_start'].min(),
        max_date_allowed=data_dash['date_end'].max(),
        initial_visible_month=data_dash['date_start'].min(),
        start_date=data_dash['date_start'].min(),
        end_date=data_dash['date_end'].max()
    ), style={'width': '49%', 'display':  'flex'}),
   html.Div([
   dcc.Graph(
       id='date-users',
       figure=px.bar(data_dash.groupby('date')['user_id'].nunique().reset_index(),
                     x="date", y = 'user_id')
   )  
]
), # количество зрителей на мероприятиях
html.Div([ 
  dcc.Graph(
       id='event-users-h',
       figure=px.bar(data_dash.groupby('event')['user_id'].nunique().reset_index()\
                     .sort_values(by='user_id', ascending = True), x="user_id", y="event",
                     orientation='h', title = 'Топ мероприятий')
   ) 
]
), # количество зрителей по городам
  html.Div([
  dcc.Graph(
       id='city-users-h',
       figure=px.bar(data_dash.groupby('user_city')['user_id'].nunique().reset_index()\
                     .sort_values(by='user_id', ascending = True).tail(10),
                     x="user_id", y="user_city",
                     orientation='h', title = 'Топ городов')
   ) 
]
), # количество зрителей по профессиям
  html.Div([
  dcc.Graph(
       id='prof-users-h',
       figure=px.bar(data_dash.groupby('profession')['user_id'].nunique().reset_index()\
                     .sort_values(by='user_id', ascending = True).tail(10),
                     x="user_id", y="profession",
                     orientation='h', title = 'Топ профессий зрителей')
   ) 
]
), # количество просмотров по докладчикам
   html.Div([
  dcc.Graph(
       id='speakers-views-h',
       figure=px.bar(data_dash.groupby('speaker_name')['user_id'].count().reset_index()\
                     .sort_values(by='user_id', ascending = True).tail(10),
                     x="user_id", y="speaker_name",
                     orientation='h', title = 'Топ докладчиков по количеству просмотров')
   ) 
]
),       
    
])

# фильтр для количества зрителей на мероприятиях

@app.callback(
    Output("event-users-h", "figure"), [
    Input("events-dropdown", "value"),
    Input("date-picker", "start_date"),
    Input("date-picker", "end_date")]
)
def update_event_bar_chart(event, start_date, end_date):
    df_tmp = data_dash
    mask = (df_tmp['event'].isin(event)) & (df_tmp['date_start'] >= start_date) &\
                                        (df_tmp['date_end']<=end_date)
    fig_tmp = px.bar(df_tmp[mask].groupby('event')['user_id'].nunique().reset_index()\
                     .sort_values(by='user_id', ascending = True), x="user_id", y="event",
                     orientation='h', title = 'Топ мероприятий')
    return fig_tmp

# фильтр для количества зрителей по городам

@app.callback(
    Output("city-users-h", "figure"), [
    Input("events-dropdown", "value"),
    Input("date-picker", "start_date"),
    Input("date-picker", "end_date")]
)
def update_city_bar_chart(event, start_date, end_date):
    df_tmp = data_dash
    mask = (df_tmp['event'].isin(event)) & (df_tmp['date_start'] >= start_date) &\
                                        (df_tmp['date_end']<=end_date)
    fig_tmp = px.bar(df_tmp[mask].groupby('user_city')['user_id'].nunique().reset_index()\
                     .sort_values(by='user_id', ascending = True).tail(10),
                     x="user_id", y="user_city",
                     orientation='h', title = 'Топ городов')
    return fig_tmp

# фильтр для количества зрителей по профессиям

@app.callback(
    Output("prof-users-h", "figure"), [
    Input("events-dropdown", "value"),
    Input("date-picker", "start_date"),
    Input("date-picker", "end_date")]
)
def update_prof_bar_chart(event, start_date, end_date):
    df_tmp = data_dash
    mask = (df_tmp['event'].isin(event)) & (df_tmp['date_start'] >= start_date) &\
                                        (df_tmp['date_end']<=end_date)
    fig_tmp = px.bar(df_tmp[mask].groupby('profession')['user_id'].nunique().reset_index()\
                     .sort_values(by='user_id', ascending = True).tail(10),
                     x="user_id", y="profession",
                     orientation='h', title = 'Топ профессий зрителей')
    return fig_tmp

# фильтр для количества просмотров по докладчикам

@app.callback(
    Output("speakers-views-h", "figure"), [
    Input("events-dropdown", "value"),
    Input("date-picker", "start_date"),
    Input("date-picker", "end_date")]
)
def update_speaker_bar_chart(event, start_date, end_date):
    df_tmp = data_dash
    mask = (df_tmp['event'].isin(event)) & (df_tmp['date_start'] >= start_date) &\
                                        (df_tmp['date_end']<=end_date)
    fig_tmp = px.bar(df_tmp[mask].groupby('speaker_name')['user_id'].count().reset_index()\
                     .sort_values(by='user_id', ascending = True).tail(10),
                     x="user_id", y="speaker_name",
                     orientation='h', title = 'Топ докладчиков по количеству просмотров')
    return fig_tmp

# фильтр для количества зрителей по датам

@app.callback(
    Output("date-users", "figure"), [
    Input("events-dropdown", "value"),
    Input("date-picker", "start_date"),
    Input("date-picker", "end_date")]
)
def update_date_bar_chart(event, start_date, end_date):
    df_tmp = data_dash
    mask = (df_tmp['event'].isin(event)) & (df_tmp['date_start'] >= start_date) &\
                                        (df_tmp['date_end']<=end_date)
    fig_tmp = px.bar(df_tmp[mask].groupby('date')['user_id'].nunique().reset_index(),
                     x="date", y="user_id", title = 'Участники по дням')
    return fig_tmp


app.run(jupyter_mode="external")